In [38]:
try:    
    import numpy as np
    import pandas as pd
    import math
    import copy
    import random
    from pandas import ExcelWriter
    from pandas import ExcelFile
    from gurobipy import *
    import datetime
    import sys
    import os
    import time
    from openpyxl import load_workbook
    import chart_studio
    import chart_studio.plotly as py
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import chart_studio.tools as tls
    import plotly.io as pio
    import numpy as np
    from scipy.spatial import HalfspaceIntersection
    from scipy.spatial import ConvexHull
    from pyhull.convex_hull import ConvexHull as ConvexHullPy
    from pyhull import qconvex as qconvexPy
    from scipy.optimize import linprog
except Exception as e:
    print(e)

In [2]:
username = 'citilbotan' # your username
api_key = 'gcTlUyms0tQYznqo9ZdQ' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [3]:
testNumber=10
testFormat='{0:04}'.format(testNumber)
print(testFormat)
dfTestData = pd.read_excel('dataSetLedger.xlsx')
#print(dfTestData['1'][0])
#print(dfTestData)
print(dfTestData['dimension1'][0])
dfTestData['dimension1'][0]=1
dfTestData.to_excel('dataSetLedger.xlsx', index=False)
dfTestData1 = pd.read_excel('dataSetLedger.xlsx')
print(dfTestData1['dimension1'][0])

0010
1
1


In [4]:
def generateRandomNDataVariableValue(nObservations):
    #This code generates n random points with a given decimal out of a random one of the set of distribions.
    #The support of the random numbers generated randomly as well.
    #Additional parameters required to generate random numbers from the set of distributions provided in their corresponding if code block.
    
    distributionPool = ['Normal','Laplace','Uniform','Standard Cauchy', 'Triangular']
    #tempRandom will be used to select the distribution 
    temp = np.random.uniform(0,5, 1)
    tempRandom = int(temp[0])
    decimalTemp = np.random.uniform(0,4)
    decimal = int(decimalTemp)
    #Calculate the support of the distribution
    support1Temp = np.random.uniform(-200, 100, 1)
    support1=round(support1Temp[0],decimal)
    incrementOnSupport1Temp = np.random.uniform(10, 300, 1)
    incrementOnSupport1 = incrementOnSupport1Temp[0]
    support2Temp = np.random.uniform(support1, support1+incrementOnSupport1, 1)
    support2 = round(support2Temp[0],decimal)
    supportMiddle = (support2+support1)/2
    
    # Generate random location (i.e. mu)  and scale parameters (i.e. stdev) (for the scale default is 1. Must be non- negative.).
    locTemp = np.random.uniform(-100, 100, 1)
    normalScaleTemp = np.random.uniform(0, 20, 1)
    laplaceScaleTemp = np.random.uniform(0, 20, 1)
    loc = round(locTemp[0],decimal)
    normalScale = round(normalScaleTemp[0], decimal)
    laplaceScale = round(laplaceScaleTemp[0], decimal)
    
    returnSetOfRandomNumbers = []
    distributionSupport=[] 
    if distributionPool[tempRandom] == 'Normal':
        distribution = 'Normal'
        distributionSupport=[loc, normalScale]
        returnSetOfRandomNumbers = np.round(np.random.default_rng().normal(loc, normalScale, nObservations),decimal)
    elif distributionPool[tempRandom] == 'Laplace':
        distribution = 'Laplace'
        distributionSupport=[loc, laplaceScale]
        returnSetOfRandomNumbers = np.round(np.random.default_rng().laplace(loc, laplaceScale, nObservations),decimal)
    elif distributionPool[tempRandom] == 'Uniform':
        distribution = 'Uniform'
        distributionSupport=[support1, support2]
        returnSetOfRandomNumbers = np.round(np.random.default_rng().uniform(support1, support2, nObservations), decimal)
    elif distributionPool[tempRandom] == 'Standard Cauchy':
        distribution = 'Standard Cauchy'
        distributionSupport=[0,1]
        returnSetOfRandomNumbers = np.round(np.random.default_rng().standard_cauchy(nObservations),decimal)
    elif distributionPool[tempRandom] == 'Triangular':
        distribution = 'Triangular'
        distributionSupport=[support1, supportMiddle ,support2]
        returnSetOfRandomNumbers = np.round(np.random.default_rng().triangular(support1, supportMiddle ,support2, nObservations),decimal)
    else:
        print('generateNDataVariableValue/tempRandom: ', tempRandom, 'which should be in [0,4].')
    #distributionInfo = [distribution, distributionSupport]
    return returnSetOfRandomNumbers, distribution, distributionSupport

In [5]:
# New data created in this block
def randomDataPoints(nObservations,nExplanatorys):
    
    explanatoryTraining = []
    distributionSet = []
    distributionSupportSet=[]
    responseTraining, distribution, distributionSupport = generateRandomNDataVariableValue(nObservations)
    distributionSet.append(distribution)
    distributionSupportSet.append(distributionSupport)
    for i in range(nExplanatorys):
        explanatoryTrainingAppend, distribution, distributionSupport = generateRandomNDataVariableValue(nObservations)
        distributionSet.append(distribution)
        distributionSupportSet.append(distributionSupport)
        explanatoryTraining.append(explanatoryTrainingAppend)
    #----- Solve L1    
    manualBetaValues=[]
    solution, indexesOfFittedPoints, positionOfObservations, OFV, dualVariableValues =\
    L1RegressionModel(explanatoryTraining,responseTraining,manualBetaValues)
    diffAboveBelow = calculateDiffAboveBelow(positionOfObservations)  
    #---------------
    nExplanatories = len(explanatoryTraining)
    dimensionL1 = nExplanatories + 1
    nPoints = len(explanatoryTraining[0])
    
    dataLedger = pd.read_excel('dataSetLedger.xlsx')
    dataForL1 = {'Response':responseTraining}
    for i in range(nExplanatories):
        dataForL1[f'Explanatory{i+1}']=explanatoryTraining[i]
    df = pd.DataFrame(dataForL1)
    dimensionL1DataName='{0:02}'.format(dimensionL1)
    nPointsDataName = '{0:04}'.format(nPoints)
    dataSetAdditionalIndex='NA'
    try:
        dataLedgerWrite=dataLedger[f'dimension{dimensionL1}'][nPoints-1] +1
        dataLedger[f'dimension{dimensionL1}'][nPoints-1]=dataLedgerWrite
        dataSetAdditionalIndex='{0:02}'.format(dataLedgerWrite)
    except Exception as e:
        print('Error in data ledger try: ', e)
    
    
    differentialWrite = differentialToDifferentialWrite(diffAboveBelow, dimensionL1)
    dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}-{differentialWrite}.csv"
    
    df.to_csv(dataSetName)
    dataLedger.to_excel('dataSetLedger.xlsx', index=False)
    
    return responseTraining, explanatoryTraining, dataSetName, df, distributionSet, distributionSupportSet

In [6]:
def createDataExcel(xlsxFileName,nObservations,nExplanatorys):
    explanatoryTraining = []
    df = pd.read_excel(xlsxFileName, sheet_name="Data")
    #print('number of observations: ', len(df['Response']))
    nObservations = len(df['Response'])
    nExplanatorys = len(df.columns)-1
    responseTraining = df['Response'].tolist()
    for i in range(1,len(df.columns)):
        explanatoryTraining.append(df[f'Explanatory{i}'].tolist())
            
    return explanatoryTraining, responseTraining, df

In [7]:
def L1RegressionModel (explanatoryTraining,responseTraining,manualBetaValues): #This function has two if statement in it, it should not be used for ultimate speed comparison
    #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
    nObservations=len(responseTraining)
    nExplanatorys=len(explanatoryTraining)    
#--------------------------------------------------------------------------------------------------------------------------        
    #print('explanatoryTraining: ', explanatoryTraining)
    #print('responseTraining: ', responseTraining)
    modelPrimalWagner59 = Model('L1RegressionModel') #Create a model object.
    modelPrimalWagner59.Params.LogToConsole=0
    
    #Below variables are created.
    # variables named 'a' stand for beta values
    a = modelPrimalWagner59.addVars(range(len(explanatoryTraining)+1) , lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="aValues")
    zPlus = modelPrimalWagner59.addVars(range(len(explanatoryTraining[0])), vtype=GRB.CONTINUOUS, name="zPlus")
    zMinus = modelPrimalWagner59.addVars(range(len(explanatoryTraining[0])), vtype=GRB.CONTINUOUS, name="zMinus")
        
    temp_sum = LinExpr() #This paragrapgh of code is the objective function.
    for i in range(nObservations):
        temp_sum.add(zPlus[i]+zMinus[i])
    modelPrimalWagner59.setObjective(temp_sum, GRB.MINIMIZE)
    
    for i in range(nObservations):# This one will be the constraints.
        temp_sum = LinExpr()
        temp_sum.add(a[0])
        for j in range(nExplanatorys):
            temp_sum.add(a[j+1]*explanatoryTraining[j][i])
        temp_sum.add(zPlus[i]-zMinus[i])
        modelPrimalWagner59.addConstr(temp_sum == responseTraining[i])
   #!!!!MANUALLY SET CONSTRAINT !!!!!!!
    if len(manualBetaValues) != 0:
        for i in range(len(explanatoryTraining)+1):
            modelPrimalWagner59.addConstr(a[i] == manualBetaValues[i])
   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    
    modelPrimalWagner59.optimize()#Solves the model
    #modelPrimalWagner59.write(f'L1.lp')
#--------------------------------------------------------------------------------------------------------------    
    #Extracts the variables from different data type to a vector consist of reel numbered values.
    decisionVariableValues =[] 
    zPlusValues = []
    zMinusValues = []
    positionOfObservations = []
    indexesOfFittedPoints=[]
    warmStartVariableInfo=[]
    
    for j in range(nExplanatorys+1):
        decisionVariableValues.append(a[j].x)

    for i in range(nObservations):
        zPlusValues.append(zPlus[i].x)
        zMinusValues.append(zMinus[i].x)
        if zPlusValues[i]>0.000001:
            positionOfObservations.append(1)
        elif zMinusValues[i]>0.000001:
            positionOfObservations.append(-1)
        else:
            indexesOfFittedPoints.append(i)
            positionOfObservations.append(0)
    dualVariableValues=[]        
    for c in modelPrimalWagner59.getConstrs():
        dualVariableValues.append(c.pi)
            

    OFV= modelPrimalWagner59.objVal
    #the following line returns the basis of the model
    #modelPrimalWagner59.write("out.bas")
    solution = {'betaValues':decisionVariableValues, 'zPlusValues':zPlusValues,'zMinusValues':zMinusValues}
    #print('dualVariableValues: ', dualVariableValues)
    return solution,indexesOfFittedPoints,positionOfObservations, OFV, dualVariableValues

In [8]:
#3/28/22
def calculateBasis(indexesOfFittedPoints,positionOfObservations,explanatoryTraining):
    #general structre
    basis=[]
    basisX1=[]
    numberOfPoints=len(explanatoryTraining[0])
    numberOfFittedPoints=len(explanatoryTraining)+1
    numberOfExplanatories=len(explanatoryTraining)
    basis= np.empty((0,numberOfPoints))
    basisX1 = np.empty((0,numberOfFittedPoints))
    setOfPoints=[i for i in range(numberOfPoints)]
    setOfUnfittedPoints=[i for i in range(numberOfPoints)]
    counter=0
    for i in indexesOfFittedPoints:
        setOfUnfittedPoints.pop(i-counter)
        #setOfUnfittedPoints.pop(i)
        counter+=1
    counterUnfitted=0
    for g in range(numberOfPoints):
        if g in indexesOfFittedPoints:
            #for i in indexesOfFittedPoints:
            appendArray=[]
            appendArray.append(1)
            for j in range(numberOfExplanatories):
                appendArray.append(explanatoryTraining[j][g])
            basisX1=np.append(basisX1,[appendArray], axis = 0)
            for k in range(numberOfPoints-numberOfExplanatories-1):
                appendArray.append(0)
            basis=np.append(basis,[appendArray], axis = 0)
        elif g in setOfUnfittedPoints:
            
            #for i in setOfUnfittedPoints:
            appendArray=[]
            appendArray.append(1)
            for j in range(numberOfExplanatories):
                appendArray.append(explanatoryTraining[j][g])
            for k in range(numberOfPoints-len(indexesOfFittedPoints)):
                if counterUnfitted==k:
                    appendArray.append(positionOfObservations[g])
                else:
                    appendArray.append(0)
            counterUnfitted+=1
            basis=np.append(basis,[appendArray], axis = 0)
            
    return basis,basisX1

In [9]:
#Ax+b<=0 structre
def calculateInertZoneInequalityCoefficients(dualValuesFromGUROBI,indexesOfFittedPoints,positionOfObservations,explanatoryTraining):
    aboveInequalitiesCoefficients=[]
    belowInequalitiesCoefficients=[]
    returnInequalitiesCoefficients=[]
    basis,basisX1=calculateBasis(indexesOfFittedPoints,positionOfObservations,explanatoryTraining)
    basisX1Inv=np.linalg.inv(basisX1)
    
    for loc in (-1,1):
        if loc==-1 :
            for i in range(len(explanatoryTraining)+1):
                tempAbove=[]
                tempBelow=[]
                for j in range(1,len(explanatoryTraining)+1):
                    #append x1,...,xm here in above and below
                    tempAbove.append(basisX1Inv[j][i])
                    tempBelow.append(-basisX1Inv[j][i])
                tempAbove.append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]+basisX1Inv[0][i])
                tempBelow.append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]-basisX1Inv[0][i])
                aboveInequalitiesCoefficients.append(tempAbove)
                belowInequalitiesCoefficients.append(tempBelow)
        else:
            for i in range(len(explanatoryTraining)+1):
                tempAbove=[]
                tempBelow=[]
                for j in range(1,len(explanatoryTraining)+1):
                    #append x1,...,xm here in above and below
                    tempAbove.append(-basisX1Inv[j][i])
                    tempBelow.append(basisX1Inv[j][i])
                tempAbove.append(-(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]+basisX1Inv[0][i]))
                tempBelow.append(-(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]-basisX1Inv[0][i]))
                aboveInequalitiesCoefficients.append(tempAbove)
                belowInequalitiesCoefficients.append(tempBelow)

    returnInequalitiesCoefficients=[aboveInequalitiesCoefficients,belowInequalitiesCoefficients]
    #first[0,1]:0 for above, 1 for below inequalities
    #second[0,1]:0 for >-1, 1 for <1
    #third[0,...,m]=[[RHS],[x1],[xm]]: contains m+1 sets starts with the coeff. RHS;then, x1,..,xm.
    #fourth[0,...,m]= each contains coeffs. from m+1 inequalities.
    #returnInequalitiesCoefficients[ [ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]],[ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]]]
    #aboveInequalitiesCoefficients=[ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]]
    #the structre is the same for below.
    returnInequalitiesCoefficientsNP = np.array(returnInequalitiesCoefficients)
    return returnInequalitiesCoefficientsNP    

In [10]:
def calculateAbsoluteErrorOfNewPoint (explanatoryNewDataObservation,responseNewDataObservation,solutionErrors):
    numberOfExplanatory=len(explanatoryNewDataObservation)
    distanceOfNewPointToCurrentHyperlane = solutionErrors['betaValues'][0] - responseNewDataObservation[0]
    for j in range(numberOfExplanatory):
        distanceOfNewPointToCurrentHyperlane += solutionErrors['betaValues'][j+1]*explanatoryNewDataObservation[j]
    return abs(distanceOfNewPointToCurrentHyperlane)

In [11]:
def calculatePositionOfNewPoint(explanatoryNewDataObservation,responseNewDataObservation,solutionErrors):
    numberOfExplanatory=len(explanatoryNewDataObservation)
    distanceOfNewPointToCurrentHyperlane = solutionErrors['betaValues'][0] - responseNewDataObservation[0]
    for j in range(numberOfExplanatory):
        distanceOfNewPointToCurrentHyperlane += solutionErrors['betaValues'][j+1]*explanatoryNewDataObservation[j] 


    returnPosition=0
    if distanceOfNewPointToCurrentHyperlane > 0:
        returnPosition=-1
    if distanceOfNewPointToCurrentHyperlane < 0:
        returnPosition=1
    return returnPosition

In [12]:
#general structre
def calculateInertZoneInequalityCoefficientsOriginal(dualValuesFromGUROBI,indexesOfFittedPoints,positionOfObservations,explanatoryTraining):
    aboveInequalitiesCoefficients=[[],[]]
    belowInequalitiesCoefficients=[[],[]]
    returnInequalitiesCoefficients=[]
    basis,basisX1=calculateBasis(indexesOfFittedPoints,positionOfObservations,explanatoryTraining)
    basisX1Inv=np.linalg.inv(basisX1)
    for i in range(len(explanatoryTraining)+1):
        aboveInequalitiesCoefficients[0].append(list())
        aboveInequalitiesCoefficients[1].append(list())
        belowInequalitiesCoefficients[0].append(list())
        belowInequalitiesCoefficients[1].append(list())
    
    for loc in (-1,1):
        if loc==-1 :
            for i in range(len(explanatoryTraining)+1):
                #append RHS of inequalities here which is the 0th list of above and below
                aboveInequalitiesCoefficients[0][0].append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]+basisX1Inv[0][i])
                belowInequalitiesCoefficients[0][0].append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]-basisX1Inv[0][i])
                for j in range(1,len(explanatoryTraining)+1):
                    #append x1,...,xm here in above and below
                    aboveInequalitiesCoefficients[0][j].append(-basisX1Inv[j][i])
                    belowInequalitiesCoefficients[0][j].append(basisX1Inv[j][i])
        else:
            for i in range(len(explanatoryTraining)+1):
                #append RHS of inequalities here which is the 0th list of above and below
                aboveInequalitiesCoefficients[1][0].append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]+basisX1Inv[0][i])
                belowInequalitiesCoefficients[1][0].append(loc-dualValuesFromGUROBI[indexesOfFittedPoints[i]]-basisX1Inv[0][i])
                for j in range(1,len(explanatoryTraining)+1):
                    #append x1,...,xm here in above and below
                    aboveInequalitiesCoefficients[1][j].append(-basisX1Inv[j][i])
                    belowInequalitiesCoefficients[1][j].append(basisX1Inv[j][i])


    returnInequalitiesCoefficients=[aboveInequalitiesCoefficients,belowInequalitiesCoefficients]
    #first[0,1]:0 for above, 1 for below inequalities
    #second[0,1]:0 for >-1, 1 for <1
    #third[0,...,m]=[[RHS],[x1],[xm]]: contains m+1 sets starts with the coeff. RHS;then, x1,..,xm.
    #fourth[0,...,m]= each contains coeffs. from m+1 inequalities.
    #returnInequalitiesCoefficients[ [ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]],[ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]]]
    #aboveInequalitiesCoefficients=[ [ [ RHSfor'>-1'],[ x1for'>-1'],[ xmfor'>-1']],[[RHSfor'<1'],[x1for'<1'],[xmfor'<1']]]
    #the structre is the same for below.
    return returnInequalitiesCoefficients    

In [13]:
def fittedPointForInertZone(aboveOrBelow,dualVariableValuesOriginal,indexesOfFittedPointsOriginal):
    returnPointIndex=None
    if aboveOrBelow == 'above':
        for i in indexesOfFittedPointsOriginal:
            if dualVariableValuesOriginal[i]>=0:
                returnPointIndex=i
            
    elif aboveOrBelow == 'below':
        for i in indexesOfFittedPointsOriginal:
            if dualVariableValuesOriginal[i]<=0:
                returnPointIndex=i
    else:
        print('Error. Please pass(the first parameter) above or below as string')
    return returnPointIndex

In [14]:
def reorderHalfspaceIntersections(vertexConnectionIndices,halfspaceInersections):#Works for 3D
    orderedVertexList=[]
    for i in range(len(halfspaceInersections[0])):
        orderedVertexList.append([])
    
    controlIndex=0
    for k in range(len(halfspaceInersections[0])):
        orderedVertexList[k].append(halfspaceInersections[controlIndex][k])
    #print('controlIndex: ', controlIndex)
    whileBreak=False
    while True:
        for individualFaces in vertexConnectionIndices:
            if individualFaces[0]==controlIndex:
                for k in range(len(halfspaceInersections[0])):
                    orderedVertexList[k].append(halfspaceInersections[individualFaces[1]][k])
                controlIndex=individualFaces[1]
                #print('controlIndex: ', controlIndex)
            #print('len(orderedVertexList[0]): ', len(orderedVertexList[0]),'len(halfspaceInersections): ', len(halfspaceInersections))
            if len(orderedVertexList[0])==len(halfspaceInersections):
                whileBreak=True
                break
        if whileBreak==True:
            break
            
    return orderedVertexList        

In [15]:
def findInertLimits2D(indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal,dualValuesFittedPoints):

    dualValues=dualValuesFittedPoints
        
    basis,basisX1 = calculateBasis(indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
    basisX1Inv = np.linalg.inv(basisX1)
    
    
    aboveGreaterThanMinusOneVertexSet1=(-1-dualValues[0]+basisX1Inv[0][0])/-basisX1Inv[1][0]
    aboveLessThanOneVertexSet1=(1-dualValues[0]+basisX1Inv[0][0])/-basisX1Inv[1][0]
    
    aboveGreaterThanMinusOneVertexSet2=(-1-dualValues[1]+basisX1Inv[0][1])/-basisX1Inv[1][1]
    aboveLessThanOneVertexSet2=(1-dualValues[1]+basisX1Inv[0][1])/-basisX1Inv[1][1]
    
    belowGreaterThanMinusOneVertexSet1=(-1-dualValues[0]-basisX1Inv[0][0])/basisX1Inv[1][0]
    belowLessThanOneVertexSet1=(1-dualValues[0]-basisX1Inv[0][0])/basisX1Inv[1][0]
    
    belowGreaterThanMinusOneVertexSet2=(-1-dualValues[1]-basisX1Inv[0][1])/basisX1Inv[1][1]
    belowLessThanOneVertexSet2=(1-dualValues[1]-basisX1Inv[0][1])/basisX1Inv[1][1]
    #Should assign them on lessthanAbove and morethanAbove sets, later will miminimum maximum applied.
    
    if - basisX1Inv[1][0]<0:
        adjustedAboveGreaterThanMinusOneVertexSet1 = aboveLessThanOneVertexSet1
        adjustedAboveLessThanOneVertexSet1 = aboveGreaterThanMinusOneVertexSet1
        adjustedBelowGreaterThanMinusOneVertexSet1 = belowGreaterThanMinusOneVertexSet1
        adjustedBelowLessThanOneVertexSet1 = belowLessThanOneVertexSet1
    else:
        adjustedAboveGreaterThanMinusOneVertexSet1 = aboveGreaterThanMinusOneVertexSet1
        adjustedAboveLessThanOneVertexSet1 = aboveLessThanOneVertexSet1
        adjustedBelowGreaterThanMinusOneVertexSet1 = belowLessThanOneVertexSet1
        adjustedBelowLessThanOneVertexSet1 =  belowGreaterThanMinusOneVertexSet1        

    if - basisX1Inv[1][1]<0:
        adjustedAboveGreaterThanMinusOneVertexSet2 = aboveLessThanOneVertexSet2
        adjustedAboveLessThanOneVertexSet2 = aboveGreaterThanMinusOneVertexSet2
        adjustedBelowGreaterThanMinusOneVertexSet2 = belowGreaterThanMinusOneVertexSet2
        adjustedBelowLessThanOneVertexSet2 = belowLessThanOneVertexSet2
    else:
        adjustedAboveGreaterThanMinusOneVertexSet2 = aboveGreaterThanMinusOneVertexSet2
        adjustedAboveLessThanOneVertexSet2 = aboveLessThanOneVertexSet2
        adjustedBelowGreaterThanMinusOneVertexSet2 = belowLessThanOneVertexSet2
        adjustedBelowLessThanOneVertexSet2 =  belowGreaterThanMinusOneVertexSet2   
    
    #Take minimum of maximums; and maximum of minimums among the two defining inequality tuples of a zone 
    vertexAboveMax=min(adjustedAboveLessThanOneVertexSet1,adjustedAboveLessThanOneVertexSet2)
    vertexAboveMin=max(adjustedAboveGreaterThanMinusOneVertexSet1,adjustedAboveGreaterThanMinusOneVertexSet2)
    vertexBelowMax=min(adjustedBelowLessThanOneVertexSet1,adjustedBelowLessThanOneVertexSet2)
    vertexBelowMin=max(adjustedBelowGreaterThanMinusOneVertexSet1,adjustedBelowGreaterThanMinusOneVertexSet2)    
    
    
    return vertexAboveMax,vertexAboveMin,vertexBelowMax,vertexBelowMin

In [16]:
def calculateDiffAboveBelow(positionOfObservations):
    diffAboveBelowReturn=0
    for position in positionOfObservations:
        diffAboveBelowReturn+=position
    return diffAboveBelowReturn

In [17]:
def calculateDualVariableValues (basis, indexesOfFittedPointsOriginal):
    dualVariableValueList=[]
    unfittedPointIndexList=[]
    basisInv = np.linalg.inv(basis)
    for i in range(len(basisInv)):
        if (i not in indexesOfFittedPointsOriginal):
            unfittedPointIndexList.append(i)
    for i in range(len(basisInv)):
        temp=0
        for j in range(len(indexesOfFittedPointsOriginal),len(basisInv)):
            temp+=basisInv[j][i]
            #temp+=basisInv[i][j]
        #dualVariableValueList.append(np.round(temp,6))
        dualVariableValueList.append(temp)
    return dualVariableValueList  

In [18]:
def findBarycenterFromVertexes (definingInequalities,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,aboveOrBelowAsText,explanatoryValues):
    #-------
    xofPoints=[]
    for i in range(len(explanatoryValues)):
        xofPoints.append(list())
        for j in range(len(explanatoryValues[i])):
            xofPoints[i].append(explanatoryValues[i][j])    
    halfspaces = definingInequalities
    
    #feasible_pointIndex = fittedPointForInertZone(aboveOrBelowAsText,dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    #feasible_point=np.array([])
    feasiblePointFromLP = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(definingInequalities)
    #print("Chebychef center of a simplex at ", aboveOrBelowAsText," is: ", feasiblePointFromLP)
    #for i in range(len(indexesOfFittedPointsOriginal)-1):
    #    feasible_point = np.append(feasible_point, xofPoints[i][feasible_pointIndex])
#-------
    halfspaceIntersections = HalfspaceIntersection(halfspaces, np.array(feasiblePointFromLP))
    #-------
    vertexCoordinates = halfspaceIntersections.intersections
    print('findBarycenterFromVertexes/vertexCoordinates: ', vertexCoordinates)
    #print('TEST, findBarycenterFromVertexes, vertexCoordinates :', vertexCoordinates)
    dimensionSimplex = len(vertexCoordinates[0])
    numberOfVertexes = len(vertexCoordinates)
    lambdaMultiplier = 1/(numberOfVertexes)    
    baryCenter = []
    for j in range(dimensionSimplex):
        baryCenter.append(0)
        for i in range(numberOfVertexes): 
            baryCenter[j]+=vertexCoordinates[i][j]*(lambdaMultiplier)
            
    return baryCenter        

In [19]:
def generateVertexFromDefiningInequalityCoefficients (definingInequalities):
    # This function takes defining inequalities of a simplex removed one of its rows.
    # Normally there are one more defining inequality then there are variables in the defining inequality expressions.
    hurra= np.transpose(definingInequalities)
    lenExpression=len(definingInequalities[0])
    hurra1=-hurra[lenExpression-1]
    hurra2=np.transpose(np.delete(hurra,lenExpression-1,0))
    #print('hurra1: ', hurra1)
    #print('hurra2: ', hurra2)
    #print('np.transpose(hurra1): ', np.transpose(hurra1))
    #print('np.transpose(hurra2): ', np.transpose(hurra2))
    #print(np.linalg.solve(np.transpose(hurra2), np.transpose(hurra1)))
    #print(np.linalg.solve(hurra2, hurra1))
    decisionTemp = np.linalg.solve(hurra2, hurra1)
    decisionVariableValues = decisionTemp.tolist()
    return decisionVariableValues

In [20]:
def generateFeasiblePointFromDefiningInequalityCoefficients (definingInequalities):
    #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
    nExpressions=len(definingInequalities)
    nDimension=len(definingInequalities[0])-1    
#--------------------------------------------------------------------------------------------------------------------------        
    modelFeasiblePoint = Model('FeasiblePointFromExpressions') #Create a model object.
    modelFeasiblePoint.Params.LogToConsole=0
    
    aValues = modelFeasiblePoint.addVars(range(nDimension) , lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="varA")
        
    temp_sum = LinExpr() #This paragrapgh of code is the objective function.
    for i in range(nDimension):
        temp_sum.add(aValues[i])
    modelFeasiblePoint.setObjective(0, GRB.MAXIMIZE)
    
    for i in range(nExpressions):# This one will be the constraints.
        temp_sum = LinExpr()
        temp_sum.add(definingInequalities[i][nDimension])
        for j in range(nDimension):
            temp_sum.add(aValues[j]*definingInequalities[i][j])
        modelFeasiblePoint.addConstr(temp_sum <= 0)
    
    modelFeasiblePoint.optimize()#Solves the model
    #modelFeasiblePoint.write(f'L1.lp')
#--------------------------------------------------------------------------------------------------------------    
    #Extracts the variables from different data type to a vector consist of reel numbered values.
    decisionVariableValues =[] 
 
    for v in modelFeasiblePoint.getVars():
        decisionVariableValues.append(v.x)
    
    OFV= modelFeasiblePoint.objVal
    modelFeasiblePoint.update()
    return decisionVariableValues

DID NOT WORK!!!!!!
See halfspace intersection for precision issues related to qhalf.

If you do not know a feasible point for the halfspaces, use linear programming to find one. Assume, n halfspaces defined by: aj*x1+bj*x2+cj*x3+dj<=0, j=1..n. Perform the following linear program:

max(x5) aj*x1+bj*x2+cj*x3+dj*x4+x5<=0, j=1..n

Then, if [x1,x2,x3,x4,x5] is an optimal solution with x4>0 and x5>0 we get:

aj*(x1/x4)+bj*(x2/x4)+cj*(x3/x4)+dj<=(-x5/x4) j=1..n and (-x5/x4)<0,

and conclude that the point [x1/x4,x2/x4,x3/x4] is inside all the halfspaces. Since x5 is optimal, this feasible point is "clearly inside" the halfspaces (good for precision errors).

After finding a feasible point, the rest of the intersection algorithm is from Preparata & Shamos ['85, p. 316, "A simple case ..."]. Translate the halfspaces so that the feasible point is the origin. Calculate the dual polytope. The dual polytope is the convex hull of the vertices dual to the original faces in regard to the unit sphere (i.e., halfspaces at distance d from the origin are dual to vertices at distance 1/d). Then calculate the resulting polytope, which is the dual of the dual polytope, and translate the origin back to the feasible point [S. Spitz, S. Teller, D. Strawn].

In [21]:
def generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (definingInequalities):
    try:
        #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
        norm_vector = np.reshape(np.linalg.norm(definingInequalities[:, :-1], axis=1),
            (definingInequalities.shape[0], 1))
        c = np.zeros((definingInequalities.shape[1],))
        c[-1] = -1
        A = np.hstack((definingInequalities[:, :-1], norm_vector))
        b = - definingInequalities[:, -1:]
        res = linprog(c, A_ub=A, b_ub=b, bounds=(None, None))
        x = res.x[:-1]
        y = res.x[-1]
        return x
    except Exception as expt:
        print(expt)
        #passExcept=np.delete(definingInequalities, 0, 0)
        print('TEST AREA ENTERED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        #aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
        print('TEST AREA PASSED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        return aPass

The two simplexes are a different size but one is entirely contained in the other. Three types: <br>
A1: Interior simplex collapses to a point <br>
A2: Interior simplex is entirely contained in the interior <br> 
A3: Extreme points of interior simplex are on the facets of the exterior simplex <br>
B: The two simplexes are a different size but the smaller simplex protrudes: a truncated simplex <br>
C: The two simplexes are the same size (an orthoplex; e.g.  an  octahedron in 3D) <br>
D: Some other unanticipated shape. 

In [22]:
def generateFeasiblePointFromDefiningInequalityCoefficients (definingInequalities):
    #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
    nExpressions=len(definingInequalities)
    nDimension=len(definingInequalities[0])-1    
#--------------------------------------------------------------------------------------------------------------------------        
    modelFeasiblePoint = Model('FeasiblePointFromExpressions') #Create a model object.
    modelFeasiblePoint.Params.LogToConsole=0
    
    aValues = modelFeasiblePoint.addVars(range(nDimension) , lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="varA")
        
    temp_sum = LinExpr() #This paragrapgh of code is the objective function.
    for i in range(nDimension):
        temp_sum.add(aValues[i])
    modelFeasiblePoint.setObjective(0, GRB.MAXIMIZE)
    
    for i in range(nExpressions):# This one will be the constraints.
        temp_sum = LinExpr()
        temp_sum.add(definingInequalities[i][nDimension])
        for j in range(nDimension):
            temp_sum.add(aValues[j]*definingInequalities[i][j])
        modelFeasiblePoint.addConstr(temp_sum <= 0)
    
    modelFeasiblePoint.optimize()#Solves the model
    #modelFeasiblePoint.write(f'L1.lp')
#--------------------------------------------------------------------------------------------------------------    
    #Extracts the variables from different data type to a vector consist of reel numbered values.
    decisionVariableValues =[] 
 
    for v in modelFeasiblePoint.getVars():
        decisionVariableValues.append(v.x)
    
    OFV= modelFeasiblePoint.objVal
    modelFeasiblePoint.update()
    return decisionVariableValues

In [23]:
def generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (definingInequalities):
    try:
        #Below different options for data creation are created depending on the parameters passed with L1RegressionModel.
        norm_vector = np.reshape(np.linalg.norm(definingInequalities[:, :-1], axis=1),
            (definingInequalities.shape[0], 1))
        c = np.zeros((definingInequalities.shape[1],))
        c[-1] = -1
        A = np.hstack((definingInequalities[:, :-1], norm_vector))
        b = - definingInequalities[:, -1:]
        res = linprog(c, A_ub=A, b_ub=b, bounds=(None, None))
        x = res.x[:-1]
        y = res.x[-1]
        return x
    except Exception as expt:
        print(expt)
        #passExcept=np.delete(definingInequalities, 0, 0)
        print('TEST AREA ENTERED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        #aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
        print('TEST AREA PASSED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        return aPass

In [24]:
def classifyIneriorSimplex (definingInequalitiesCouple, differential,aboveOrBelow): 
    #Identify vertexes of Simplex 1
    #Identify vertexes of Simplex 2
    unusualCaseCheckAbove=False
    unusualCaseCheckBelow=False
    definingInequalitiesSet1=np.array([definingInequalitiesCouple[i] for i in range(int(len(definingInequalitiesCouple)/2))])
    definingInequalitiesSet2=np.array([definingInequalitiesCouple[i] for i in range(int(len(definingInequalitiesCouple)/2),int(len(definingInequalitiesCouple)))])
    definingInequalitySets=[definingInequalitiesSet1,definingInequalitiesSet2]
    testVertexSet=[[],[]]
    lenExpressions = len(definingInequalitiesCouple[0])
    dimensionL1 = lenExpressions
    vertexes = getSimplexVertexesFromDefiningInequalities(definingInequalitiesCouple)
    #vertexCenters = np.round(np.add(np.multiply(vertexes[0],0.5),np.multiply(vertexes[1],0.5)),2)
    #vertexCenters = np.float64(np.add(np.multiply(vertexes[0],0.5),np.multiply(vertexes[1],0.5)))
    vertexCenters = np.add(np.multiply(vertexes[0],0.5),np.multiply(vertexes[1],0.5))
    #print('vertexCenters: ', vertexCenters)
    #print('-----------------------------------------------------')
    #print('If vertex centers are same resulting shape is an orthoplex: C')
    #print('classifyIneriorSimplex/vertexCenters: ', vertexCenters)
    #print('-----------------------------------------------------')
    #resultOrthoplex = np.all(vertexCenters == vertexCenters[0])
    resultOrthoplex=True
    for i in range(len(vertexCenters)):
        for j in range(len(vertexCenters[0])):
            if abs(vertexCenters[i][j]-vertexCenters[0][j]) > abs(vertexCenters[0][j])/1000:
                resultOrthoplex=False
    #resultOrthoplex = np.array_equiv(np.longdouble(vertexCenters),np.longdouble(vertexCenters[0]))
    #print('classifyIneriorSimplex/resultOrthoplex: ', resultOrthoplex)
    #
    testResult = 'N/A'
    if differential < -(dimensionL1-2) and (aboveOrBelow=='below' or aboveOrBelow=='Below'):
        unusualCaseCheckBelow=True
    elif differential > (dimensionL1-2) and (aboveOrBelow=='above' or aboveOrBelow=='Above'):
        unusualCaseCheckAbove=True
    if unusualCaseCheckBelow:
        if differential == - dimensionL1:
            testResult = 'A0 - infeasible simplex - no inert zone'
        elif differential == -(dimensionL1-1):
            testResult = 'A1 - Interior simplex collapses to a point - cross section is a point - half line inert zone'
    elif unusualCaseCheckAbove:
        if differential == dimensionL1:
            testResult = 'A0 - infeasible simplex at above - no inert zone'
        elif differential == (dimensionL1-1):
            testResult = 'A1 - Interior simplex collapses to a point at above - cross section is a point - half line inert zone'
    else:
        for i in range(len(vertexes)):
            for j in range(len(vertexes[i])):
                testVertexSet[i].append(checkIfVertexInSimplex(definingInequalitySets[1-i], vertexes[i][j])) 
        #print('resultOrthoplex: ', resultOrthoplex)
        if resultOrthoplex:
            testResult = 'C - The two simplexes are the same size - cross-section is an orthoplex; e.g. an octahedron in 3D)'
        #print('-----------------------------------------------------')
        #print('If testVertexSet are same resulting shape is an orthoplex: A2 or A3')
        #print('classifyIneriorSimplex/testVertexSet: ' ,testVertexSet)
        #print('-----------------------------------------------------')        
        
        if (testVertexSet[0]!=testVertexSet[1]):
            classifiedPointSet1 = classifyVertexWRTSimplex (definingInequalitiesSet2, vertexes[0][0])
            classifiedPointSet2 = classifyVertexWRTSimplex (definingInequalitiesSet1, vertexes[1][0])
            #print('classifiedPointSet1: ', classifiedPointSet1)
            #print('classifiedPointSet2: ',classifiedPointSet2)
            if classifiedPointSet1 != 'on' and classifiedPointSet2 != 'on':
                testResult= 'A2 - Interior simplex is entirely contained in the interior - cross-section is a simplex'
            else:
                testResult= 'A3 - Extreme points of interior simplex are on the facets of the exterior simplex - cross-section is a simplex'
        elif resultOrthoplex:
            testResult = 'C - The two simplexes are the same size - cross-section is an orthoplex; e.g. an octahedron in 3D)'
        else: 
            testResult = 'B - The two simplexes are a different size but the smaller simplex protrudes- cross-section is a truncated simplex'

    return testResult

In [25]:
def classifyVertexWRTSimplex (definingInequalities, vertexCoordinates):
    #Function assumes that defining inequality set of a simplex is passed as an argument.
    lenExpressions = len(definingInequalities[0])
    testResult = "N/A"
    counter=0
    for aDefiningInequality in definingInequalities:
        LHS=aDefiningInequality[lenExpressions-1]
        for i in range(lenExpressions-1):
            LHS += aDefiningInequality[i]*vertexCoordinates[i]
        #print('---')
        #print('vertex: ', vertexCoordinates, '- position to hyperplane: ', aDefiningInequality)
        #print('LHS: ', round(LHS,3))
        #print('---')
        #print('!@#$%^&*(________classifyVertexWRTSimplex/LHS: ',LHS)
        if round(LHS,3) > 0:
            testResult = "outside" 
        elif round(LHS,3) == 0:
            testResult = "on"
        else:
            counter+=1
    if counter == lenExpressions:
        testResult = "inside"
    return testResult

In [41]:
def getSimplexVertexesFromDefiningInequalities (halfspaces):
    #This function takes defining inequalities of a simplex couple and return their vertexes.
    halfspacesSet1=np.array([halfspaces[i] for i in range(int(len(halfspaces)/2))])
    halfspacesSet2=np.array([halfspaces[i] for i in range(int(len(halfspaces)/2),int(len(halfspaces)))])
    #halfSpacesSets=np.concatenate((halfspacesSet1, halfspacesSet2))
    halfSpacesSets=[halfspacesSet1, halfspacesSet2]
    #print(halfSpacesSets)
    numberOfInequalities=len(halfspacesSet1)
    vertexSet=[[],[]]
    for i in range(2):
        halfspacesSetOfIteration = halfSpacesSets [i]
        #vertexSet = np.append(vertexSet, [])
        for j in range(numberOfInequalities):
            tempIneqs = np.delete(halfspacesSetOfIteration, j, 0)
            #subSetOfDefiningInequalities = np.delete(halfspacesAboveSet1, 1, 0)
            vertexCoordinates = generateVertexFromDefiningInequalityCoefficients(tempIneqs)
            vertexSet[i].append(vertexCoordinates)

    return vertexSet

In [169]:
def calculateSubSAE (xofPoints, yofPoints, beta1MeshB1B0):
    numberOfPoints=len(yofPoints)
    subSAEExpressions=[]
    for i in range(numberOfPoints):
        numberList=[k for k in range(numberOfPoints)]
        numberList.pop(i)
        beta0ValueMakesABSValue0=yofPoints[i]-xofPoints[0][i]*xLineMashes[i]
        tempSubSAE=0
        for j in numberList:
            tempSubSAE+=np.abs(xofPoints[0][i]*beta1MeshB1B0-yofPoints[i]+beta0ValueMakesABSValue0)
        subSAEExpressions.append(tempSubSAE)    
        
    return subSAEExpressions

In [27]:
def checkIfVertexInSimplex (definingInequalities, vertexCoordinates):
    #Function assumes that defining inequality set of a simplex is passed as an argument.
    lenExpressions = len(definingInequalities[0])
    testResult = 1
    for aDefiningInequality in definingInequalities:
        LHS=aDefiningInequality[lenExpressions-1]
        for i in range(lenExpressions-1):
            LHS += aDefiningInequality[i]*vertexCoordinates[i]
        if LHS > 0.0001:
            testResult=0
        #print('-----------------------------------------------------')
        #print('One of LHS should be close to zero.')
        #print('checkIfVertexInSimplex/LHS: ', LHS)
        #print('-----------------------------------------------------')  
        #print('checkIfVertexInSimplex/LHS: ', LHS)
    return testResult

In [28]:
chebychefCenterAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (definingInequalities)
pointOnSimplexFacetAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (definingInequalities)

NameError: name 'definingInequalities' is not defined

In [28]:
def convexCombinationOfSetOfPoints (pointSet):
    nPoints=len(pointSet)
    nDimension=len(pointSet[0])
    
    tempUB=1
    convexCombinationMultipliers=[]
    #Multipliers are created in a diminising way from 1.
    totalMultipliers=0
    for i in range(nPoints-1):
        randomUniformPoint= np.random.uniform(0,tempUB,1)
        totalMultipliers+=randomUniformPoint[0]
        convexCombinationMultipliers.append(randomUniformPoint[0])
        tempUB-=max(0,tempUB-randomUniformPoint[0])
    if tempUB == 0:
        convexCombinationMultipliers.append(0)
    else:
        convexCombinationMultipliers.append(1-totalMultipliers)
        
    pointFromConvexCombination=[]
    for j in range(nDimension):
        tempValue=0
        for i in range(nPoints):
            tempValue+=pointSet[i][j]*convexCombinationMultipliers[i]
        pointFromConvexCombination.append(tempValue)
            
    #print('convexCombinationOfSetOfPoints/convexCombinationMultipliers: ', convexCombinationMultipliers)
    return pointFromConvexCombination

In [29]:
pt1=[1,2,3,4,5]
pt2=[5,4,3,2,1]
#pt3=[6,7,8,9,10]
setOfPoints=[pt1,pt2]
convexCombOfPts123=convexCombinationOfSetOfPoints (setOfPoints)
print(convexCombOfPts123)

[3.5638359221338427, 3.281917961066921, 3.0, 2.718082038933079, 2.4361640778661573]


In [93]:
dataLedger = pd.read_excel('dataSetLedger.xlsx')
dataLedgerWrite=1+dataLedger[f'dimension{1}'][1]
dataLedgerWriteStr=str(dataLedgerWrite)
dataSetAdditionalIndex='{0:01}'.format(dataLedgerWrite)
#dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}.xlsx"
dataLedger[f'dimension{1}'][1]=dataLedgerWrite
dataLedger.to_excel('dataSetLedger.xlsx', index=False)

In [ ]:
    dimensionL1DataName='{0:02}'.format(dimensionL1)
    nPointsDataName = '{0:04}'.format(nPoints)
    dataSetAdditionalIndex='{0:01}'.format(dataLedger[f'dimension{dimensionL1}'][nPoints-1])
    dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}.xlsx"
    dataSetAdditionalIndex+=1
    dataLedger[f'dimension{dimensionL1}'][nPoints-1]=dataSetAdditionalIndex
    df.to_excel(dataSetName, index=False, sheet_name="Data")
    dataLedger.to_excel('dataSetLedger.xlsx', index=False)

In [29]:
# Creates a Gucci Bag out of an existing data set.
def writeL1SolutionParametersToExcel (dataSetName,diffAboveBelow,solutionOriginal, OFVOriginal, indexesOfFittedPointsOriginal,dualVariableValuesOriginal,\
returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2):
    
    nPoints=len(dfData.index)
    nDimension=len(solutionOriginal['betaValues'])
    current_time = datetime.datetime.now()
    current_year = current_time.year
    current_month = current_time.year
    current_day = current_time.day
    #Watch the creation time, it might be updated each time the initial data set is called by analyzeCreateSeriesOfData...
    shortNarrativeForDataSets = f'Dataset: {dataSetName} contains {nPoints} points with {nDimension-1} explanatory variables and a response variable. This data set is created on (MM/DD/YY): {current_month}\ {current_day}\ {current_year}. This data set is used to be applied L1 regression. It contains the following sheets along with their description: <\br> Data: This sheet contains all explanatory and response variables. Rensonse variables is collected at the first column of Data set and explanatory variables are collected to the right columns of the respose variable.' 
    
    dfDifferential = pd.DataFrame({'Differential': [diffAboveBelow]})
    dfBetaValues = pd.DataFrame({'Beta Values': solutionOriginal['betaValues']})
    dfObjectiveValue = pd.DataFrame({'Objective Value': [OFVOriginal]})
    dfIndexesOfFittedPoints = pd.DataFrame({'Fitted Point Indexes': indexesOfFittedPointsOriginal})
    dfDualVariableValues = pd.DataFrame({'Dual Variable Values': dualVariableValuesOriginal})
    dictReturnInequalitiesCoefficientsAbove={}
    transposedReturnInequalitiesCoefficientsAbove = np.transpose(returnInequalitiesCoefficients[0])
    for i in range(len(transposedReturnInequalitiesCoefficientsAbove)-1):
        dictReturnInequalitiesCoefficientsAbove[f'a{i}'] = list(transposedReturnInequalitiesCoefficientsAbove[i])
    dictReturnInequalitiesCoefficientsAbove['b'] = \
    list(np.negative(transposedReturnInequalitiesCoefficientsAbove[len(transposedReturnInequalitiesCoefficientsAbove)-1]))
    dfReturnInequalitiesCoefficientsAbove = pd.DataFrame(dictReturnInequalitiesCoefficientsAbove)
    dictReturnInequalitiesCoefficientsBelow={}
    transposedReturnInequalitiesCoefficientsBelow = np.transpose(returnInequalitiesCoefficients[1])
    for i in range(len(transposedReturnInequalitiesCoefficientsBelow)-1):
        dictReturnInequalitiesCoefficientsBelow[f'a{i}'] = list(transposedReturnInequalitiesCoefficientsBelow[i])
    dictReturnInequalitiesCoefficientsBelow['b'] = \
    list(np.negative(transposedReturnInequalitiesCoefficientsBelow[len(transposedReturnInequalitiesCoefficientsBelow)-1]))
    dfReturnInequalitiesCoefficientsBelow = pd.DataFrame(dictReturnInequalitiesCoefficientsBelow)
    #---
    writer = pd.ExcelWriter(dataSetName, engine='xlsxwriter')
    #---
    dfData.to_excel(writer, index=False, sheet_name="Data")
    dfDifferential.to_excel(writer, index=False, sheet_name="Differential")
    dfBetaValues.to_excel(writer, index=False, sheet_name="BetaValues")
    dfObjectiveValue.to_excel(writer, index=False, sheet_name="ObjectiveValue")
    dfIndexesOfFittedPoints.to_excel(writer, index=False, sheet_name="IndexesOfFittedPoints")
    dfDualVariableValues.to_excel(writer, index=False, sheet_name="DualVariableValues")
    dfReturnInequalitiesCoefficientsAbove.to_excel(writer, index=False, sheet_name="DefiningInequalitiesAbove")
    dfReturnInequalitiesCoefficientsBelow.to_excel(writer, index=False, sheet_name="DefiningInequalitiesBelow")
    #writer.save()
    #---------------------------------------------------------------------------------------
    #---
    dfShapes = pd.DataFrame({'Above Simplex Shapes': [simplexShapesAbove],'Below Simplex Shapes':[simplexShapesBelow]})
    dfShapes.to_excel(writer, index=False, sheet_name="SimplexShapes")
    #writer.save()
    #---
    Barycenters={}
    if len(baryCenterOfSimplexesAboveSet1) > 0:
        Barycenters['BarycenterAboveSimplex1']=baryCenterOfSimplexesAboveSet1
    if len(baryCenterOfSimplexesAboveSet2) > 0:
        Barycenters['BarycenterAboveSimplex2']=baryCenterOfSimplexesAboveSet2
    if len(baryCenterOfSimplexesBelowSet1) > 0:    
        Barycenters['BarycenterBelowSimplex1']=baryCenterOfSimplexesBelowSet1
    if len(baryCenterOfSimplexesBelowSet2) > 0:     
        Barycenters['BarycenterBelowSimplex2']=baryCenterOfSimplexesBelowSet2
    #---
    dfBarycenters = pd.DataFrame(Barycenters)
    dfBarycenters.to_excel(writer, index=False, sheet_name="Barycenters")
    writer.save()

In [30]:
def differentialToDifferentialWrite (diffAboveBelow, dimensionL1):
    if diffAboveBelow < 0:
        differential = dimensionL1 - diffAboveBelow 
        differentialWrite = '{0:02}'.format(differential)
    else:
        differential = diffAboveBelow
        differentialWrite = '{0:02}'.format(differential)
        
    return differentialWrite

In [64]:
#WIP
# New data created in this block
def analyzeCreateSeriesOfDataFromZeroDifferentialData (dataSetName,aboveOrBelow):
    #above or below should be passed as 'above' or 'below'. i.e. as a string
    #This code creates series of data sets up to the differential equal to the dimension of the data set. That is #of explanatory +1
    #Data set names 05by0071-01...05by0076-01; 05by007-01... 05by0012-01; 04by0016-01...04by0021-01; 06by0062-01...06by0068-01
    # 07by0043-01...07by0050-01; 07by0053-01...07by0060-01; 08by0072-01...08by0080-01; 09by0083-01...09by0092-01
    # 10by0082-01...10by0092-01; 04by0018-01...04by0022-01; 03by0017-01...03by0020-01; 02by0010-01...02by0012-01
    # 09by0035-01...09by0044-01
    explanatoryTrainingOriginal, responseTrainingOriginal, df = createDataExcel(dataSetName,None,None)
    numberOfExplanatoryVariables=len(explanatoryTrainingOriginal)
    nExplanatories=len(explanatoryTrainingOriginal)
    dimensionL1=numberOfExplanatoryVariables+1
    numberOfPoints=len(responseTrainingOriginal)
    manualBetaValues=list()
    solutionOriginal, indexesOfFittedPointsOriginal, positionOfObservationsOriginal, OFVOriginal,\
    dualVariableValuesOriginal = L1RegressionModel(explanatoryTrainingOriginal,responseTrainingOriginal,manualBetaValues)

    for stepCounter in range(dimensionL1+1):
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        #04by0015-01: manualBetaValues= [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #03by0012-01: manualBetaValues= [1.191011236,-0.146067416,-0.123595506]
        #You can automate this process just by passing a manual beta values.
        manualBetaValuesLoop=solutionOriginal['betaValues']
        #Beta values of  03by0012-01
        #manualBetaValues=[1.191011236, -0.146067416, -0.123595506]
        #Beta values of 04by0014-01
        #manualBetaValues = [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #-------------------------------------------------
        dataSetNameLoop = dataSetName


        print('dataSetNameLoop: ', dataSetNameLoop)    
        explanatoryTrainingLoop, responseTrainingLoop, dfData = createDataExcel(dataSetNameLoop,None,None)
        numberOfPoints=len(responseTrainingLoop)
        solutionLoop, indexesOfFittedPointsLoop, positionOfObservationsLoop, OFVLoop, dualVariableValuesLoop =\
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesLoop)
        diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsLoop)
        basisLoop,basisX1Loop = calculateBasis(indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        basisX1Inv = np.linalg.inv(basisX1Loop)
        basisInv = np.linalg.inv(basisLoop)
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        dualVariableValuesLoop = calculateDualVariableValues(basisLoop, indexesOfFittedPointsLoop)
        #03by0012-01: dualVariableValuesOriginal=[-1,-1,-1,1,1,-1,1,-1,-1,1]
        #04by0015-01: dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #-------------------------------------------------
        print('Data set name: ', dataSetName)
        print('Differential: ', diffAboveBelow)
        print('Beta values: ', solutionLoop['betaValues'])
        print('Objective value: ', OFVLoop)
        #print('position of observations:', positionOfObservationsOriginal)
        print('indexesOfFittedPointsLoop: ', indexesOfFittedPointsLoop)
        #print('dualVariableValues: ', np.round(dualVariableValuesOriginal,3))
        returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients\
        (dualVariableValuesLoop,indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        #print('-------------------------------------------------')
        print('-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------')
        manualBetaValuesTest=list()
        solutionTest, indexesOfFittedPointsTest, positionOfObservationsTest, OFVTest, dualVariableValuesTest = \
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesTest)
        #print('Data set name TEST: ', dataSetName)
        print('Objective value TEST: ', OFVTest)
        print('-------------------------------------------------------------------------------------------------------------------')
        
        baryCenterOfSimplexesAboveSet1=np.array([])
        baryCenterOfSimplexesAboveSet2=np.array([])
        baryCenterOfSimplexesBelowSet1=np.array([])
        baryCenterOfSimplexesBelowSet2=np.array([])

        halfspacesAbove = returnInequalitiesCoefficients[0]
        halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
        halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
        halfspacesBelow = returnInequalitiesCoefficients[1]
        halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
        halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
        simplexShapesAbove = classifyIneriorSimplex (halfspacesAbove, diffAboveBelow, 'above')
        print('SHAPES ABOVE: ', simplexShapesAbove)
        #print('*****************************')
        simplexShapesBelow = classifyIneriorSimplex (halfspacesBelow, diffAboveBelow, 'below')
        print('SHAPES BELOW: ', simplexShapesBelow)
        print('-------------------------------------------------------------------------------------------------------------------')
        #print('CHECK HERE WITH A MANUAL SOLUTION:::')
        #print(halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop)
        barryCenterToAppendAbove=list()
        barryCenterToAppendBelow=list()
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
            #chebychefCenterAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
            baryCenterOfSimplexesAboveSet1 = findBarycenterFromVertexes\
            (halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            #print('barycenter of the simplexes above set 1: ', np.round(baryCenterOfSimplexesAboveSet1,3))
            try:
                pointOnSimplexFacetAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet1,baryCenterOfSimplexesAboveSet1])
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet1: ', interiorPointAboveSet1)
                barryCenterToAppendAbove=interiorPointAboveSet1
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet1: ', pointOnSimplexFacetAboveSet1)                
            except Exception as e:
                print(e)
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 1.')
            print(e)
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),\
                                                                            int(len(halfspacesAbove)))])
            #chebychefCenterAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
            baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes\
            (halfspacesAboveSet2,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            try:
                pointOnSimplexFacetAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet2=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet2,baryCenterOfSimplexesAboveSet2])
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet2: ', interiorPointAboveSet2)
                barryCenterToAppendAbove=interiorPointAboveSet2
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet2: ', pointOnSimplexFacetAboveSet2)
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes above set 2: ', np.round(baryCenterOfSimplexesAboveSet2,3))        
            #print('barycenter of the simplexes above are the same: ',\
                  #np.array_equal(np.round(baryCenterOfSimplexesAboveSet1,3), np.round(baryCenterOfSimplexesAboveSet2,3)))
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 2.')
            print(e)    
        print('-')
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
            baryCenterOfSimplexesBelowSet1 = findBarycenterFromVertexes\
            (halfspacesBelowSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'below',\
             explanatoryTrainingLoop)
            #chebychefCenterBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            try:
                pointOnSimplexFacetBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet1: ', pointOnSimplexFacetBelowSet1)
                interiorPointBelowSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet1,baryCenterOfSimplexesBelowSet1])            
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointBelowSet1: ', interiorPointBelowSet1)
                barryCenterToAppendBelow=interiorPointBelowSet1
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes below set 1: ', np.round(baryCenterOfSimplexesBelowSet1,3))
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 1.')
            print(e)
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),\
                                                                            int(len(halfspacesBelow)))])
            #chebychefCenterBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            baryCenterOfSimplexesBelowSet2 = findBarycenterFromVertexes\
            (halfspacesBelowSet2,dualVariableValuesLoop, indexesOfFittedPointsLoop,'below',explanatoryTrainingLoop)            
            try:
                pointOnSimplexFacetBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)
                interiorPointBelowSet2 = convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet2,baryCenterOfSimplexesBelowSet2])            
                barryCenterToAppendBelow = interiorPointBelowSet2
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet2: ', pointOnSimplexFacetBelowSet2)
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes below set 2: ', np.round(baryCenterOfSimplexesBelowSet2,3))
            #print('barycenter of the simplexes below are the same: ',\
                  #np.array_equal(np.round(baryCenterOfSimplexesBelowSet1,3), np.round(baryCenterOfSimplexesBelowSet2,3)))
            
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 2.')
            print(e)
  
     
        dataLedger = pd.read_excel('dataSetLedger.xlsx')
        try:
            if aboveOrBelow == 'above' or aboveOrBelow == 'Above':
                # New data set is formed here. There is explanatorytrainingloop data that grow gradually. 
                # After this step a new data set is created by appending the barycenter of the corresponding side of L1 ...
                # ... hyperplane on the previous data set.
                # You can create a new sheet on the data set to be written and write: defining inequalities, solution for the data set ...
                for j in range(nExplanatories):
                    explanatoryTrainingLoop[j].append(barryCenterToAppendAbove[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], barryCenterToAppendAbove)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane+deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(nExplanatories):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                #----------------------
                dfData = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                print('TEST/NumberOfPoints: ', numberOfPoints)
                dimensionL1DataName='{0:02}'.format(dimensionL1)
                nPointsDataName = '{0:04}'.format(numberOfPoints)
                dataLedgerWrite=1+dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]
                dataSetAdditionalIndex='{0:02}'.format(dataLedgerWrite)
                differentialWrite = differentialToDifferentialWrite (diffAboveBelow, dimensionL1)
                #dataSetAdditionalIndex+=1
                dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}-{differentialWrite}.xlsx"
                dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]=dataLedgerWrite
                #dfData.to_excel(dataSetName, index=False, sheet_name="Data")
                dataLedger.to_excel('dataSetLedger.xlsx', index=False)
        except Exception as e:
            print('error obtaining baryCenter at above')
            print(e)
        try:
            if aboveOrBelow == 'below' or aboveOrBelow == 'Below':
                for j in range(nExplanatories):
                    explanatoryTrainingLoop[j].append(feasiblePointToAppendBelow[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], feasiblePointToAppendBelow)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane-deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(nExplanatories):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                print('TEST/:len(dataForL1)', len(dataForL1))
                #----------------------
                dfData = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                print('TEST/NumberOfPoints: ', numberOfPoints)
                dimensionL1DataName='{0:02}'.format(dimensionL1)
                nPointsDataName = '{0:04}'.format(numberOfPoints)
                dataLedgerWrite=1+dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]
                dataSetAdditionalIndex='{0:02}'.format(dataLedgerWrite)
                differentialWrite = differentialToDifferentialWrite (diffAboveBelow, dimensionL1)
                dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}-{differentialWrite}.xlsx"
                dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]=dataLedgerWrite
                #dfData.to_excel(dataSetName, index=False, sheet_name="Data")
                dataLedger.to_excel('dataSetLedger.xlsx', index=False)
                #dataSetName = f"0{dimensionL1}by00{numberOfPoints}-02.xlsx"
                #dfData.to_excel(dataSetName, index=False, sheet_name="Data")
        except Exception as e:
            print('error obtaining baryCenter at below')
            print(e)  
    #create a new data set
        #if stepCounter != 0:
        dataSetNameLoop = dataSetName
        print('analyzeCreateSeriesOfDataFromZeroDifferentialData/dataSetNameLoop: ',dataSetNameLoop)
        writeL1SolutionParametersToExcel (dataSetNameLoop,diffAboveBelow,solutionLoop, OFVLoop, indexesOfFittedPointsLoop,dualVariableValuesLoop,\
        returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
        baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2) 
        print('!!!!!!!!!!!!!!!!!!!!!!!************** ITERATION ENDS*****************!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

In [65]:
analyzeCreateSeriesOfDataFromZeroDifferentialData ('03by0007-01-00.xlsx','above')

dataSetNameLoop:  03by0007-01-00.xlsx
Data set name:  03by0007-01-00.xlsx
Differential:  1
Beta values:  [-96.91312514573492, -0.22504043261702744, -0.019930673218749497]
Objective value:  51.925452644067946
indexesOfFittedPointsLoop:  [1, 2, 6]
-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------
Objective value TEST:  51.925452644067946
-------------------------------------------------------------------------------------------------------------------
SHAPES ABOVE:  A2 - Interior simplex is entirely contained in the interior - cross-section is a simplex
SHAPES BELOW:  C - The two simplexes are the same size - cross-section is an orthoplex; e.g. an octahedron in 3D)
-------------------------------------------------------------------------------------------------------------------
findBarycenterFromVertexes/vertexCoordinates:  [[-77.44325249 -16.41503956]
 [-78.52458307  16.09541968]
 [-76.64262444  16.60922071]]
findBarycenter

In [60]:
#Writing this function is cancelled because of the unnecessary wieldy
#When 0 differential data is passed, the function will creates data to limit differential from above and below
def analyzeCreateSeriesOfDataFromData (dataSetName):
    
    #Create general parameters and solve initial L1 
    dataSetNameOriginal=dataSetName
    explanatoryTrainingOriginal, responseTrainingOriginal, df = createDataExcel(dataSetName,None,None)
    numberOfExplanatoryVariables=len(explanatoryTrainingOriginal)
    dimensionL1=numberOfExplanatoryVariables+1
    numberOfPoints=len(responseTrainingOriginal)
    manualBetaValues=list()
    solutionOriginal, indexesOfFittedPointsOriginal, positionOfObservationsOriginal, OFVOriginal,\
    dualVariableValuesOriginal = L1RegressionModel(explanatoryTrainingOriginal,responseTrainingOriginal,manualBetaValues)
    diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsLoop)
    
    if diffAboveBelow >= 0:
        for i in range(dimensionL1-diffAboveBelow):
            #THIS CHUNK WILL BE COPY PASTED TO ALL LOOPS---------------------------------------------------------------------------
            manualBetaValuesLoop=solutionOriginal['betaValues']
            dataSetNameLoop = dataSetName
            explanatoryTrainingLoop, responseTrainingLoop, dfData = createDataExcel(dataSetNameLoop,None,None)
            numberOfPoints=len(responseTrainingLoop)
            solutionLoop, indexesOfFittedPointsLoop, positionOfObservationsLoop, OFVLoop, dualVariableValuesLoop =\
            L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesLoop)
            diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsLoop)
            basisLoop,basisX1Loop = calculateBasis(indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
            basisX1Inv = np.linalg.inv(basisX1Loop)
            basisInv = np.linalg.inv(basisLoop)
            dualVariableValuesLoop = calculateDualVariableValues(basisLoop, indexesOfFittedPointsLoop)
            #-------------------------------------------------
            print('Data set name: ', dataSetName)
            print('Differential: ', diffAboveBelow)
            print('Beta values: ', solutionLoop['betaValues'])
            print('Objective value: ', OFVLoop)
            #print('position of observations:', positionOfObservationsOriginal)
            print('indexesOfFittedPointsLoop: ', indexesOfFittedPointsLoop)
            #print('dualVariableValues: ', np.round(dualVariableValuesOriginal,3))
            returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients\
            (dualVariableValuesLoop,indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
            #print('-------------------------------------------------')
            print('-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------')
            manualBetaValuesTest=list()
            solutionTest, indexesOfFittedPointsTest, positionOfObservationsTest, OFVTest, dualVariableValuesTest = \
            L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesTest)
            #print('Data set name TEST: ', dataSetName)
            print('Objective value TEST: ', OFVTest)
            
            #CHUNK ENDS---------------------------------------------------------------------------
            
            #ABOVE
            print('-------------------------------------------------------------------------------------------------------------------')
            baryCenterOfSimplexesAboveSet1=np.array([])
            baryCenterOfSimplexesAboveSet2=np.array([])
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
            halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
            simplexShapesAbove = classifyIneriorSimplex (halfspacesAbove, diffAboveBelow, 'above')
            print('SHAPES ABOVE: ', simplexShapesAbove)
            print('-------------------------------------------------------------------------------------------------------------------')
            feasiblePointToAppendAbove=list()
            try:
                halfspacesAbove = returnInequalitiesCoefficients[0]
                halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
                #chebychefCenterAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
                baryCenterOfSimplexesAboveSet1 = findBarycenterFromVertexes\
                (halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
                 explanatoryTrainingLoop)
                #print('barycenter of the simplexes above set 1: ', np.round(baryCenterOfSimplexesAboveSet1,3))
                try:
                    pointOnSimplexFacetAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                    interiorPointAboveSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet1,baryCenterOfSimplexesAboveSet1])
                    #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet1: ', interiorPointAboveSet1)
                    feasiblePointToAppendAbove=interiorPointAboveSet1
                    #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet1: ', pointOnSimplexFacetAboveSet1)                
                except Exception as e:
                    print(e)
            except Exception as e:
                print('Problem in obtaining the barycenters of "above".')
                print(e)
            try:
                halfspacesAbove = returnInequalitiesCoefficients[0]
                halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),\
                                                                                int(len(halfspacesAbove)))])
                #chebychefCenterAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
                baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes\
                (halfspacesAboveSet2,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
                 explanatoryTrainingLoop)
                try:
                    pointOnSimplexFacetAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                    interiorPointAboveSet2=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet2,baryCenterOfSimplexesAboveSet2])
                    #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet2: ', interiorPointAboveSet2)
                    feasiblePointToAppendBelow=interiorPointAboveSet2
                    #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet2: ', pointOnSimplexFacetAboveSet2)
                except Exception as e:
                    print(e)
                #print('barycenter of the simplexes above set 2: ', np.round(baryCenterOfSimplexesAboveSet2,3))        
                #print('barycenter of the simplexes above are the same: ',\
                      #np.array_equal(np.round(baryCenterOfSimplexesAboveSet1,3), np.round(baryCenterOfSimplexesAboveSet2,3)))
            except Exception as e:
                print('Problem in obtaining the barycenters of "above" set 2.')
                print(e)    
            print('-')            
    #create a new data set
        if i != 0:
            print('analyzeCreateSeriesOfDataFromZeroDifferentialData/dataSetNameLoop: ',dataSetNameLoop)
            writeL1SolutionParametersToExcel (dataSetNameLoop,diffAboveBelow,solutionLoop, OFVLoop, indexesOfFittedPointsLoop,dualVariableValuesLoop,\
            returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
            baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2)            
            
            #Obtain from above
            #Create feasible point
            #Update loop parameters
            #Create a gucci bag from the updated data set
                #Update GucciBags function so that it writes simplex vertexes and the barycenters
        for i in range(diffAboveBelow):
            #Obtain from below
    elif diffAboveBelow <= 0:
        for i in range(-diffAboveBelow):
            #Obtain from above
        for i in range(dimensionL1+diffAboveBelow):
            #Obtain from below    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    #############################################################################################################
    #############################################################################################################
    #############################################################################################################
    ##################################### BUILD YOUR FUNCTION FROM THIS MASS ####################################
    #############################################################################################################
    #############################################################################################################
    #############################################################################################################
    #############################################################################################################

    for stepCounter in range(dimensionL1+1):
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        #04by0015-01: manualBetaValues= [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #03by0012-01: manualBetaValues= [1.191011236,-0.146067416,-0.123595506]
        #You can automate this process just by passing a manual beta values.
        manualBetaValuesLoop=solutionOriginal['betaValues']
        #Beta values of  03by0012-01
        #manualBetaValues=[1.191011236, -0.146067416, -0.123595506]
        #Beta values of 04by0014-01
        #manualBetaValues = [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #-------------------------------------------------
        dataSetNameLoop = dataSetName


        print('dataSetNameLoop: ', dataSetNameLoop)    
        explanatoryTrainingLoop, responseTrainingLoop, dfData = createDataExcel(dataSetNameLoop,None,None)
        numberOfPoints=len(responseTrainingLoop)
        solutionLoop, indexesOfFittedPointsLoop, positionOfObservationsLoop, OFVLoop, dualVariableValuesLoop =\
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesLoop)
        diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsLoop)
        basisLoop,basisX1Loop = calculateBasis(indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        basisX1Inv = np.linalg.inv(basisX1Loop)
        basisInv = np.linalg.inv(basisLoop)
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        dualVariableValuesLoop = calculateDualVariableValues(basisLoop, indexesOfFittedPointsLoop)
        #03by0012-01: dualVariableValuesOriginal=[-1,-1,-1,1,1,-1,1,-1,-1,1]
        #04by0015-01: dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #-------------------------------------------------
        print('Data set name: ', dataSetName)
        print('Differential: ', diffAboveBelow)
        print('Beta values: ', solutionLoop['betaValues'])
        print('Objective value: ', OFVLoop)
        #print('position of observations:', positionOfObservationsOriginal)
        print('indexesOfFittedPointsLoop: ', indexesOfFittedPointsLoop)
        #print('dualVariableValues: ', np.round(dualVariableValuesOriginal,3))
        returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients\
        (dualVariableValuesLoop,indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        #print('-------------------------------------------------')
        print('-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------')
        manualBetaValuesTest=list()
        solutionTest, indexesOfFittedPointsTest, positionOfObservationsTest, OFVTest, dualVariableValuesTest = \
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesTest)
        #print('Data set name TEST: ', dataSetName)
        print('Objective value TEST: ', OFVTest)
        print('-------------------------------------------------------------------------------------------------------------------')
        
        baryCenterOfSimplexesAboveSet1=np.array([])
        baryCenterOfSimplexesAboveSet2=np.array([])
        baryCenterOfSimplexesBelowSet1=np.array([])
        baryCenterOfSimplexesBelowSet2=np.array([])

        halfspacesAbove = returnInequalitiesCoefficients[0]
        halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
        halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
        halfspacesBelow = returnInequalitiesCoefficients[1]
        halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
        halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
        simplexShapesAbove = classifyIneriorSimplex (halfspacesAbove, diffAboveBelow, 'above')
        print('SHAPES ABOVE: ', simplexShapesAbove)
        #print('*****************************')
        simplexShapesBelow = classifyIneriorSimplex (halfspacesBelow, diffAboveBelow, 'below')
        print('SHAPES BELOW: ', simplexShapesBelow)
        print('-------------------------------------------------------------------------------------------------------------------')
        #print('CHECK HERE WITH A MANUAL SOLUTION:::')
        #print(halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop)
        feasiblePointToAppendAbove=list()
        feasiblePointToAppendBelow=list()
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
            #chebychefCenterAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
            baryCenterOfSimplexesAboveSet1 = findBarycenterFromVertexes\
            (halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            #print('barycenter of the simplexes above set 1: ', np.round(baryCenterOfSimplexesAboveSet1,3))
            try:
                pointOnSimplexFacetAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet1,baryCenterOfSimplexesAboveSet1])
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet1: ', interiorPointAboveSet1)
                feasiblePointToAppendAbove=interiorPointAboveSet1
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet1: ', pointOnSimplexFacetAboveSet1)                
            except Exception as e:
                print(e)
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 1.')
            print(e)
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),\
                                                                            int(len(halfspacesAbove)))])
            #chebychefCenterAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
            baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes\
            (halfspacesAboveSet2,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            try:
                pointOnSimplexFacetAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet2=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet2,baryCenterOfSimplexesAboveSet2])
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet2: ', interiorPointAboveSet2)
                feasiblePointToAppendBelow=interiorPointAboveSet2
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet2: ', pointOnSimplexFacetAboveSet2)
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes above set 2: ', np.round(baryCenterOfSimplexesAboveSet2,3))        
            #print('barycenter of the simplexes above are the same: ',\
                  #np.array_equal(np.round(baryCenterOfSimplexesAboveSet1,3), np.round(baryCenterOfSimplexesAboveSet2,3)))
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 2.')
            print(e)    
        print('-')
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
            baryCenterOfSimplexesBelowSet1 = findBarycenterFromVertexes\
            (halfspacesBelowSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'below',\
             explanatoryTrainingLoop)
            #chebychefCenterBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            try:
                pointOnSimplexFacetBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet1: ', pointOnSimplexFacetBelowSet1)
                interiorPointBelowSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet1,baryCenterOfSimplexesBelowSet1])            
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointBelowSet1: ', interiorPointBelowSet1)
                feasiblePointToAppendBelow=interiorPointBelowSet1
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes below set 1: ', np.round(baryCenterOfSimplexesBelowSet1,3))
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 1.')
            print(e)
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),\
                                                                            int(len(halfspacesBelow)))])
            #chebychefCenterBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            baryCenterOfSimplexesBelowSet2 = findBarycenterFromVertexes\
            (halfspacesBelowSet2,dualVariableValuesLoop, indexesOfFittedPointsLoop,'below',explanatoryTrainingLoop)            
            try:
                pointOnSimplexFacetBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)            
                interiorPointBelowSet2 = convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet2,baryCenterOfSimplexesBelowSet2])            
                feasiblePointToAppendBelow = interiorPointBelowSet2
                #print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet2: ', pointOnSimplexFacetBelowSet2)
            except Exception as e:
                print(e)
            #print('barycenter of the simplexes below set 2: ', np.round(baryCenterOfSimplexesBelowSet2,3))
            #print('barycenter of the simplexes below are the same: ',\
                  #np.array_equal(np.round(baryCenterOfSimplexesBelowSet1,3), np.round(baryCenterOfSimplexesBelowSet2,3)))
            
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 2.')
            print(e)
  
    #create a new data set
        if stepCounter != 0:
            print('analyzeCreateSeriesOfDataFromZeroDifferentialData/dataSetNameLoop: ',dataSetNameLoop)
            writeL1SolutionParametersToExcel (dataSetNameLoop,diffAboveBelow,solutionLoop, OFVLoop, indexesOfFittedPointsLoop,dualVariableValuesLoop,\
            returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
            baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2)      
        dataLedger = pd.read_excel('dataSetLedger.xlsx')
        try:
            if aboveOrBelow == 'above' or aboveOrBelow == 'Above':
                # New data set is formed here. There is explanatorytrainingloop data that grow gradually. 
                # After this step a new data set is created by appending the barycenter of the corresponding side of L1 ...
                # ... hyperplane on the previous data set.
                # You can create a new sheet on the data set to be written and write: defining inequalities, solution for the data set ...
                for j in range(numberOfExplanatoryVariables):
                    explanatoryTrainingLoop[j].append(feasiblePointToAppendAbove[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], feasiblePointToAppendAbove)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane+deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(numberOfExplanatoryVariables):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                #----------------------
                dfData = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                print('TEST/NumberOfPoints: ', numberOfPoints)
                dimensionL1DataName='{0:02}'.format(dimensionL1)
                nPointsDataName = '{0:04}'.format(numberOfPoints)
                dataLedgerWrite=1+dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]
                dataSetAdditionalIndex='{0:02}'.format(dataLedgerWrite)
                differentialWrite = differentialToDifferentialWrite (diffAboveBelow, dimensionL1)
                #dataSetAdditionalIndex+=1
                dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}-{differentialWrite}.xlsx"
                #dfData.to_excel(dataSetName, index=False, sheet_name="Data")
        except Exception as e:
            print('error obtaining baryCenter at above')
            print(e)
        try:
            if aboveOrBelow == 'below' or aboveOrBelow == 'Below':
                for j in range(numberOfExplanatoryVariables):
                    explanatoryTrainingLoop[j].append(feasiblePointToAppendBelow[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], feasiblePointToAppendBelow)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane-deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(numberOfExplanatoryVariables):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                #print('TEST/:len(dataForL1)', len(dataForL1))
                #----------------------
                dfData = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                print('TEST/NumberOfPoints: ', numberOfPoints)
                dimensionL1DataName='{0:02}'.format(dimensionL1)
                nPointsDataName = '{0:04}'.format(numberOfPoints)
                dataLedgerWrite=1+dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]
                dataSetAdditionalIndex='{0:02}'.format(dataLedgerWrite)
                differentialWrite = differentialToDifferentialWrite (diffAboveBelow, dimensionL1)
                dataSetName = f"{dimensionL1DataName}by{nPointsDataName}-{dataSetAdditionalIndex}-{differentialWrite}.xlsx"
                #dataLedger[f'dimension{dimensionL1}'][numberOfPoints-1]=dataLedgerWrite
                dfData.to_excel(dataSetName, index=False, sheet_name="Data")
                #dataLedger.to_excel('dataSetLedger.xlsx', index=False)
                #dataSetName = f"0{dimensionL1}by00{numberOfPoints}-02.xlsx"
                #dfData.to_excel(dataSetName, index=False, sheet_name="Data")
        except Exception as e:
            print('error obtaining baryCenter at below')
            print(e)  
        print('!!!!!!!!!!!!!!!!!!!!!!!************** ITERATION ENDS*****************!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

IndentationError: expected an indented block (Temp/ipykernel_6644/1027011586.py, line 117)

In [32]:
def calculateLocationOfAPointOnL1Hyperplane (betaValues, pointLocation):
    pointLocationOnL1Hyperplane=betaValues[0]
    for i in range(1,len(betaValues)):
        pointLocationOnL1Hyperplane+= betaValues[i]*pointLocation[i-1]
    return pointLocationOnL1Hyperplane

In [63]:
analyzeCreateSeriesOfDataFromZeroDifferentialData ('03by0007-01-00.xlsx','above')

dataSetNameLoop:  03by0007-01-00.xlsx
Data set name:  03by0007-01-00.xlsx
Differential:  0
Beta values:  [-96.91312514573492, -0.22504043261702744, -0.019930673218749497]
Objective value:  8.098452644067933
indexesOfFittedPointsLoop:  [1, 2, 6]
-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------
Objective value TEST:  8.098452644067931
-------------------------------------------------------------------------------------------------------------------
SHAPES ABOVE:  A3 - Extreme points of interior simplex are on the facets of the exterior simplex - cross-section is a simplex
SHAPES BELOW:  A3 - Extreme points of interior simplex are on the facets of the exterior simplex - cross-section is a simplex
-------------------------------------------------------------------------------------------------------------------
findBarycenterFromVertexes/vertexCoordinates:  [[-77.03647867 -29.57164108]
 [-79.19913983  35.44927739]
 [-75.435222

FileNotFoundError: [Errno 2] No such file or directory: '03by0008-06-00.xlsx'

In [ ]:
# Create multiple lists
technologies =  ['Spark','Pandas','Java','Python', 'PHP']
fee = [25000,20000,15000,15000,18000]
duration = ['5o Days','35 Days',np.nan,'30 Days', '30 Days']
discount = [2000,1000,800,500,800]
columns=['Courses','Fee','Duration','Discount']

# Create DataFrame from multiple lists
df = pd.DataFrame(list(zip(technologies,fee,duration,discount)), columns=columns)
df.to_excel('Courses.xlsx')
# Write DataFrame to Excel file with sheet name
df.to_excel('Courses.xlsx', sheet_name='Technologies')
# Write to Multiple Sheets
with pd.ExcelWriter('Courses.xlsx') as writer:
    df.to_excel(writer, sheet_name='Technologies')
    df2.to_excel(writer, sheet_name='Schedule')
# Append DataFrame to existing excel file
with pd.ExcelWriter('Courses.xlsx',mode='a') as writer:  
    df.to_excel(writer, sheet_name='Technologies')

In [464]:
np.array_equiv(vertexCenters , vertexCenters[0])

[[ 0.01123596 -0.06741573 -0.16853933]
 [ 0.16853933 -0.01123596 -1.52808989]
 [-0.17977528  0.07865169  0.69662921]
 [-0.01123596  0.06741573 -1.83146067]
 [-0.16853933  0.01123596 -0.47191011]
 [ 0.17977528 -0.07865169 -2.69662921]]


[[-6.000000000000002, 29.0, -1.0000000000000018], [-48.00000000000001, 32.0, 27.0]]


In [594]:
a=[[2,3,4],[2,3,4],[2,3,4]]
b=a[0]
print(vertexCoupleCentersA)
print(np.array_equiv(a,b))
print(np.array_equiv(vertexCoupleCentersA,vertexCoupleCentersA[0]))
a1=vertexCoupleCentersA
b1=vertexCoupleCentersA[0]
print(a1)
print(b1)
print(np.array_equiv(a1,b1))

[[162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]]
True
True
[[162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]
 [162.459 -90.289   8.41 ]]
[162.459 -90.289   8.41 ]
True


In [589]:
print(vertexCoupleCentersA[0]==vertexCoupleCentersA[1])
print(vertexCoupleCentersA[0])
print(vertexCoupleCentersA[1])

[ True  True False]
[162.459 -90.289   8.41 ]
[162.459 -90.289   8.41 ]


Vertexes Set 1 :  [[-5.39  5.53]
 [-3.39 35.53]
 [24.61 -6.47]]
-----
Vertexes Set 2 :  [[ 26.61  23.53]
 [ 22.61 -36.47]
 [-33.39  47.53]]
-----
[[10.61 14.53]
 [ 9.61 -0.47]
 [-4.39 20.53]]
False
1
1
1
0
0
0
[ 3.17512512 10.92213094]
[ 9.47974976 12.74573811]
[ 6.32743744 11.83393453]


Vertexes Set 1 :  [[ 25.39  22.47]
 [ 21.39 -37.53]
 [-34.61  46.47]]
-----
Vertexes Set 2 :  [[-6.61  4.47]
 [-4.61 34.47]
 [23.39 -7.53]]
-----
[[ 9.39 13.47]
 [ 8.39 -1.53]
 [-5.61 19.47]]
0
0
0
1
1
1


[[ 6.63488200e-03  4.01319219e-03  1.16294914e-03 -2.21882366e+00]
 [ 2.08548004e-03 -3.94560553e-03 -1.54250631e-02 -1.64877270e+00]
 [-2.17842770e-03 -6.52007731e-03  9.86219785e-03 -2.11336748e+00]
 [-6.54193434e-03  6.45249065e-03  4.39991615e-03 -2.01903616e+00]]


In [475]:
vertexSet=np.array([])
print(vertexSet)
vertexSet = np.append(vertexSet, [])
print(vertexSet)

[]
[]


In [391]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
arr
np.delete(arr, 1, 0)
print(type(arr))

<class 'numpy.ndarray'>


In [414]:
a=halfspacesAboveSet1
print(a)
b=np.delete(halfspacesAboveSet1, 1, 0)
print(type(a))
print(b)
print('-------')
print(halfspacesAboveSet1)

[[-0.00663488 -0.00401319 -0.00116295  0.21882366]
 [-0.00208548  0.00394561  0.01542506 -0.3512273 ]
 [ 0.00217843  0.00652008 -0.0098622   0.11336748]
 [ 0.00654193 -0.00645249 -0.00439992  0.01903616]]
<class 'numpy.ndarray'>
[[-0.00663488 -0.00401319 -0.00116295  0.21882366]
 [ 0.00217843  0.00652008 -0.0098622   0.11336748]
 [ 0.00654193 -0.00645249 -0.00439992  0.01903616]]
-------
[[-0.00663488 -0.00401319 -0.00116295  0.21882366]
 [-0.00208548  0.00394561  0.01542506 -0.3512273 ]
 [ 0.00217843  0.00652008 -0.0098622   0.11336748]
 [ 0.00654193 -0.00645249 -0.00439992  0.01903616]]


In [329]:
print(x)

[-4.51590065 35.9410805  -6.82788933]


In [324]:
print(y)

12.097586340392041


In [372]:
feasiblePointFromLP = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
print(feasiblePointFromLP)

[22.755 10.17  23.245]


In [46]:
dualVariableValueList=calculateDualVariableValues (basis, indexesOfFittedPointsOriginal)
print(dualVariableValueList)

[-1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [36]:
basis,basisX1 = calculateBasis(indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
basisX1Inv = np.linalg.inv(basisX1)
basisInv = np.linalg.inv(basis)
print(basisInv)

[[-2.41043903e-01  8.52277559e-01  1.28388434e-01  2.60377910e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 6.31976188e-02 -4.57239290e-02  2.98447964e-02 -4.73184862e-02
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [ 3.19442968e-02 -4.72919103e-02 -7.20208355e-03  2.25496970e-02
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [-1.86031679e-02 -3.06952270e-02  3.74720952e-02  1.18262996e-02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-6.85234400e-01 -4.30636760e-01 -7.34027852e-01  8.49899011e-01
  

In [431]:
passExcept=halfspacesAboveSet1
aPass = generateFeasiblePointFromDefiningInequalityCoefficients(passExcept)
print(aPass)

[2.3000000000000003, 6.0, 0.49999999999999784]


In [432]:
passExcept=halfspacesAboveSet1
aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
print(aPass)

[2.3 6.  0.5]


In [421]:
passExcept=np.delete(halfspacesAboveSet1, 0, 0)
aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
print(aPass)

[ 9.08500521e+11  7.50722868e+11 -1.60608644e+12]


In [422]:
passExcept=np.delete(halfspacesAboveSet1, 1, 0)
aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
print(aPass)

[-1.51388886e+10 -9.74274956e+10 -4.63389231e+10]


In [423]:
passExcept=np.delete(halfspacesAboveSet1, 2, 0)
aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
print(aPass)

[ 1.50396983e+11 -3.90199097e+11  5.47358747e+11]


In [424]:
passExcept=np.delete(halfspacesAboveSet1, 3, 0)
aPass = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(passExcept)
print(aPass)

[-2.31044669e+11  2.40078481e+11  6.28528406e+10]


In [425]:
print(returnInequalitiesCoefficients)

[[[ 0.06319762  0.0319443  -0.01860317 -0.32771872]
  [-0.04572393 -0.04729191 -0.03069523  0.40426411]
  [ 0.0298448  -0.00720208  0.0374721  -0.04416658]
  [-0.04731849  0.0225497   0.0118263  -0.03237881]
  [-0.06319762 -0.0319443   0.01860317 -1.67228128]
  [ 0.04572393  0.04729191  0.03069523 -2.40426411]
  [-0.0298448   0.00720208 -0.0374721  -1.95583342]
  [ 0.04731849 -0.0225497  -0.0118263  -1.96762119]]

 [[-0.06319762 -0.0319443   0.01860317  0.15436909]
  [ 0.04572393  0.04729191  0.03069523 -1.30029101]
  [-0.0298448   0.00720208 -0.0374721  -0.30094345]
  [ 0.04731849 -0.0225497  -0.0118263  -0.55313463]
  [ 0.06319762  0.0319443  -0.01860317 -2.15436909]
  [-0.04572393 -0.04729191 -0.03069523 -0.69970899]
  [ 0.0298448  -0.00720208  0.0374721  -1.69905655]
  [-0.04731849  0.0225497   0.0118263  -1.44686537]]]


In [369]:
feasiblePointFromLP = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
print(feasiblePointFromLP)

[22.755      10.17000001 23.245     ]


In [ ]:
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})
df.to_excel(dataSetName, index=False, sheet_name="data")

In [ ]:
responseTraining, explanatoryTraining, dataSetName = randomDataPoints(20,4,[0,30])

In [ ]:
import pandas as pd
technologies= {
    'Courses':["Spark","PySpark","Hadoop","Python","Pandas"],
    'Fee' :[22000,25000,23000,24000,26000],
    'Duration':['30days','50days','35days', '40days','55days'],
    'Discount':[1000,2300,1000,1200,2500]
          }
df = pd.DataFrame(technologies)
print(df)

In [ ]:
# Create multiple lists
technologies =  ['Spark','Pandas','Java','Python', 'PHP']
fee = [25000,20000,15000,15000,18000]
duration = ['5o Days','35 Days',np.nan,'30 Days', '30 Days']
discount = [2000,1000,800,500,800]
columns=['Courses','Fee','Duration','Discount']

# Create DataFrame from multiple lists
df = pd.DataFrame(list(zip(technologies,fee,duration,discount)), columns=columns)
df.to_excel('Courses.xlsx')
# Write DataFrame to Excel file with sheet name
df.to_excel('Courses.xlsx', sheet_name='Technologies')
# Write to Multiple Sheets
with pd.ExcelWriter('Courses.xlsx') as writer:
    df.to_excel(writer, sheet_name='Technologies')
    df2.to_excel(writer, sheet_name='Schedule')
# Append DataFrame to existing excel file
with pd.ExcelWriter('Courses.xlsx',mode='a') as writer:  
    df.to_excel(writer, sheet_name='Technologies')

In [ ]:
print('check')

In [170]:
#Solve L1 regression on a data set - Babayaga
dataSetName="BurnPatientsData.xlsx"
explanatoryTrainingOriginal, responseTrainingOriginal, dfData = createDataExcel(dataSetName,None,None)
numberOfExplanatoryVariables=len(explanatoryTrainingOriginal)
numberOfPoints=len(responseTrainingOriginal)
manualBetaValues=list()
#manualBetaValues=betaValuesPass
solutionOriginal, indexesOfFittedPointsOriginal, positionOfObservationsOriginal, OFVOriginal, dualVariableValuesOriginal = L1RegressionModel(explanatoryTrainingOriginal,responseTrainingOriginal,manualBetaValues)
diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsOriginal)
basis,basisX1 = calculateBasis(indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
basisX1Inv = np.linalg.inv(basisX1)
basisInv = np.linalg.inv(basis)
dualVariableValuesOriginal = calculateDualVariableValues (basis, indexesOfFittedPointsOriginal)
#-------------------------------------------------
returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#-------------------------------------------------
print('Data set name: ', dataSetName)
print('Differential: ', diffAboveBelow)
print('Beta values: ', solutionOriginal['betaValues'])
print('Objective value: ', OFVOriginal)
print('indexesOfFittedPointsOriginal: ', indexesOfFittedPointsOriginal)
print('-------------------------------------------------')

baryCenterOfSimplexesAboveSet1=np.array([])
baryCenterOfSimplexesAboveSet2=np.array([])
baryCenterOfSimplexesBelowSet1=np.array([])
baryCenterOfSimplexesBelowSet2=np.array([])

halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
halfspacesBelow = returnInequalitiesCoefficients[1]
halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
simplexShapesAbove = classifyIneriorSimplex (halfspacesAbove, diffAboveBelow, 'above')
print('SHAPES ABOVE: ',simplexShapesAbove)
print('---*****************************-----------------')
simplexShapesBelow = classifyIneriorSimplex (halfspacesBelow, diffAboveBelow, 'below')
print('SHAPES BELOW: ',simplexShapesBelow)
print('-------------------------------------------------')
#-----
try:
    halfspacesAbove = returnInequalitiesCoefficients[0]
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    baryCenterOfSimplexesAboveSet1 = findBarycenterFromVertexes(halfspacesAboveSet1,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'above',explanatoryTrainingOriginal)
    print('barycenter of the simplexes above set 1: ', baryCenterOfSimplexesAboveSet1)
except Exception as e:
    print('Problem in obtaining the barycenters of "above" set 1.')
    print(e)
try:
    halfspacesAbove = returnInequalitiesCoefficients[0]
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes(halfspacesAboveSet2,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'above',explanatoryTrainingOriginal)
    print('barycenter of the simplexes above set 2: ', baryCenterOfSimplexesAboveSet2)        
    #print('barycenter of the simplexes above are the same: ', np.array_equal(np.round(baryCenterOfSimplexesAboveSet1,3), np.round(baryCenterOfSimplexesAboveSet2,3)))
except Exception as e:
    print('Problem in obtaining the barycenters of "above" set 2.')
    print(e)    
print('-')
try:
    halfspacesBelow = returnInequalitiesCoefficients[1]
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
    baryCenterOfSimplexesBelowSet1 = findBarycenterFromVertexes(halfspacesBelowSet1,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'below',explanatoryTrainingOriginal)
    print('barycenter of the simplexes below set 1: ', baryCenterOfSimplexesBelowSet1)
except Exception as e:
    print('Problem in obtaining the barycenters of "below" set 1.')
    print(e)
try:
    halfspacesBelow = returnInequalitiesCoefficients[1]
    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    baryCenterOfSimplexesBelowSet2 = findBarycenterFromVertexes(halfspacesBelowSet2,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'below',explanatoryTrainingOriginal)
    print('barycenter of the simplexes below set 2: ', baryCenterOfSimplexesBelowSet2)
    #print('barycenter of the simplexes below are the same: ', np.array_equal(np.round(baryCenterOfSimplexesBelowSet1,3), np.round(baryCenterOfSimplexesBelowSet2,3)))
except Exception as e:
    print('Problem in obtaining the barycenters of "below" set 2.')
    print(e)
    
#---------------------------------
#writeL1SolutionParametersToExcel (dataSetName,diffAboveBelow,solutionOriginal, OFVOriginal, indexesOfFittedPointsOriginal,dualVariableValuesOriginal,\
#returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
#baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2)
#---------------------------------
#WRITE vertexes of the simplexes as well
#print('positionOfObservationsOriginal: ',positionOfObservationsOriginal)

Data set name:  BurnPatientsData.xlsx
Differential:  2
Beta values:  [5.946966022682898, 0.5755301769740022, 2.662266770405915, -2.6332539461274944]
Objective value:  3226.6444972814697
indexesOfFittedPointsOriginal:  [142, 179, 337, 341]
-------------------------------------------------
SHAPES ABOVE:  A2 - Interior simplex is entirely contained in the interior - cross-section is a simplex
---*****************************-----------------
SHAPES BELOW:  B - The two simplexes are a different size but the smaller simplex protrudes- cross-section is a truncated simplex
-------------------------------------------------
findBarycenterFromVertexes/vertexCoordinates:  [[24.          7.         -0.85405345]
 [14.          2.          1.39919082]
 [-1.          7.          1.27705342]
 [29.          7.          1.37803316]]
barycenter of the simplexes above set 1:  [16.499999999999844, 5.75000000000002, 0.8000559893644381]
findBarycenterFromVertexes/vertexCoordinates:  [[-36.          -3.      

In [79]:
betaValuesPass=solutionOriginal['betaValues']

In [ ]:
##############################################################################
#
# An example of writing multiple dataframes to worksheets using Pandas and
# XlsxWriter.
#
# SPDX-License-Identifier: BSD-2-Clause
# Copyright 2013-2022, John McNamara, jmcnamara@cpan.org
#

import pandas as pd


# Create some Pandas dataframes from some data.
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})
df2 = pd.DataFrame({'Data': [21, 22, 23, 24]})
df3 = pd.DataFrame({'Data': [31, 32, 33, 34]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Sheet1')
df2.to_excel(writer, sheet_name='Sheet2')
df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [1065]:
print(returnInequalitiesCoefficients)

[[[ 3.62817982e-17 -5.88235294e-01  4.88235259e+00]
  [ 1.35135135e+01  1.57392687e+00 -2.28608906e+01]
  [-1.35135135e+01 -9.85691574e-01  1.49785370e+01]
  [-3.62817982e-17  5.88235294e-01 -6.88235259e+00]
  [-1.35135135e+01 -1.57392687e+00  2.08608906e+01]
  [ 1.35135135e+01  9.85691574e-01 -1.69785370e+01]]

 [[-3.62817982e-17  5.88235294e-01 -7.47058859e+00]
  [-1.35135135e+01 -1.57392687e+00  2.11645466e+01]
  [ 1.35135135e+01  9.85691574e-01 -1.86939590e+01]
  [ 3.62817982e-17 -5.88235294e-01  5.47058859e+00]
  [ 1.35135135e+01  1.57392687e+00 -2.31645466e+01]
  [-1.35135135e+01 -9.85691574e-01  1.66939590e+01]]]


In [835]:
print(basisX1)
print(np.round(basisX1Inv,2))
#print(basisInv)

[[  1.    -33.729  98.989  87.434 -78.802]
 [  1.    -49.489  90.541 -82.853 -81.424]
 [  1.     21.739 -22.395  13.243 -24.375]
 [  1.    -49.907  11.503  69.976  27.936]
 [  1.     94.22  -24.024  60.133  53.867]
 [  1.    -33.729  98.989  87.434 -78.802]
 [  1.    -49.489  90.541 -82.853 -81.424]
 [  1.     21.739 -22.395  13.243 -24.375]
 [  1.    -49.907  11.503  69.976  27.936]
 [  1.     94.22  -24.024  60.133  53.867]
 [  1.    -33.729  98.989  87.434 -78.802]
 [  1.    -49.489  90.541 -82.853 -81.424]
 [  1.     21.739 -22.395  13.243 -24.375]
 [  1.    -49.907  11.503  69.976  27.936]
 [  1.     94.22  -24.024  60.133  53.867]
 [  1.    -33.729  98.989  87.434 -78.802]
 [  1.    -49.489  90.541 -82.853 -81.424]
 [  1.     21.739 -22.395  13.243 -24.375]
 [  1.    -49.907  11.503  69.976  27.936]
 [  1.     94.22  -24.024  60.133  53.867]
 [  1.    -33.729  98.989  87.434 -78.802]
 [  1.    -49.489  90.541 -82.853 -81.424]
 [  1.     21.739 -22.395  13.243 -24.375]
 [  1.    -

In [844]:
print(basis)
print(np.round(basisInv,2))
#print(basisInv)

[[  1.    -33.729  98.989  87.434 -78.802   0.      0.      0.      0.
    0.   ]
 [  1.    -49.489  90.541 -82.853 -81.424   0.      0.      0.      0.
    0.   ]
 [  1.     21.739 -22.395  13.243 -24.375   0.      0.      0.      0.
    0.   ]
 [  1.    -33.648  44.682 -72.384  64.772  -1.      0.      0.      0.
    0.   ]
 [  1.    -49.907  11.503  69.976  27.936   0.      0.      0.      0.
    0.   ]
 [  1.     14.672  -6.585 -54.356  30.042   0.     -1.      0.      0.
    0.   ]
 [  1.     60.962  45.916  21.749 -24.526   0.      0.     -1.      0.
    0.   ]
 [  1.     94.22  -24.024  60.133  53.867   0.      0.      0.      0.
    0.   ]
 [  1.     83.518 -20.588  71.554 -34.725   0.      0.      0.     -1.
    0.   ]
 [  1.     95.073  22.753 -97.64   50.912   0.      0.      0.      0.
    1.   ]]
[[-0.17  0.29  0.38  0.    0.33  0.    0.    0.18 -0.    0.  ]
 [ 0.    0.   -0.    0.   -0.01  0.    0.    0.01 -0.    0.  ]
 [ 0.    0.01 -0.02 -0.   -0.   -0.   -0.    0.01  0.

In [530]:
feasiblePointFromLP = generateFeasiblePointFromDefiningInequalityCoefficients(halfspacesBelowSet1)
print(feasiblePointFromLP)

[-13.760999999999962, 2.0220000000000353, 147.73999999999992]


In [896]:
print(returnInequalitiesCoefficients)

[[[-6.01631378e-03  5.52431800e-03  5.55748604e-03  2.85977123e-02]
  [ 5.49635442e-03 -3.03476582e-03  7.13602578e-03  3.62663856e-01]
  [ 2.57956261e-03  2.17006596e-03 -8.28194775e-03 -2.42914896e-01]
  [-2.05960325e-03 -4.65961814e-03 -4.41156407e-03 -1.48346672e-01]
  [ 6.01631378e-03 -5.52431800e-03 -5.55748604e-03 -2.02859771e+00]
  [-5.49635442e-03  3.03476582e-03 -7.13602578e-03 -2.36266386e+00]
  [-2.57956261e-03 -2.17006596e-03  8.28194775e-03 -1.75708510e+00]
  [ 2.05960325e-03  4.65961814e-03  4.41156407e-03 -1.85165333e+00]]

 [[ 6.01631378e-03 -5.52431800e-03 -5.55748604e-03 -4.57997712e-01]
  [-5.49635442e-03  3.03476582e-03 -7.13602578e-03 -1.01929586e+00]
  [-2.57956261e-03 -2.17006596e-03  8.28194775e-03 -1.77961104e-01]
  [ 2.05960325e-03  4.65961814e-03  4.41156407e-03 -3.44745328e-01]
  [-6.01631378e-03  5.52431800e-03  5.55748604e-03 -1.54200229e+00]
  [ 5.49635442e-03 -3.03476582e-03  7.13602578e-03 -9.80704144e-01]
  [ 2.57956261e-03  2.17006596e-03 -8.28194775

In [898]:
vertListA= getSimplexVertexesFromDefiningInequalities (halfspacesAbove)
print(vertListA)

[[[-18.693880541999985, 7.778432691999997, -33.11506277800001], [-18.693880541999995, 7.778432692000003, -33.115062778], [-18.693880542, 7.778432691999995, -33.115062777999995], [-18.693880541999985, 7.778432692000014, -33.115062777999995]], [[-439.309880542, 416.8384326920001, 184.54893722199995], [512.298119458, -145.0175673079999, 333.724937222], [482.17011945800016, 605.5264326920002, -444.955062778], [-629.9338805419999, -846.2335673079999, -205.77906277800005]]]


In [899]:
baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes(halfspacesAboveSet2,dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'above',explanatoryTrainingOriginal)
print(baryCenterOfSimplexesAboveSet2)

[-24.925174055999946, 10.371243589333318, -44.15341703733327]


In [148]:
hullPyBelow

['4', '0 3 2', '3 1 2', '1 0 2', '0 1 3']

number of observations:  50434
len(df.columns):  4
feasible_pointBelow:  [3.81412952 0.         0.        ]
feasible_pointAbove:  [-5.  20.   4.4]


In [240]:
print(returnInequalitiesCoefficients)

[[[-9.46832003e-03 -1.34987906e-02 -4.83819891e-03 -1.15371427e+00]
  [-9.19519290e-03  1.21918849e-02 -1.29567020e-03 -1.62518498e+00]
  [ 1.05729733e-02 -1.39040536e-02  1.18091937e-02 -1.60877695e+00]
  [ 8.09053968e-03  1.52109593e-02 -5.67532455e-03 -1.61232380e+00]
  [ 9.46832003e-03  1.34987906e-02  4.83819891e-03 -8.46285732e-01]
  [ 9.19519290e-03 -1.21918849e-02  1.29567020e-03 -3.74815019e-01]
  [-1.05729733e-02  1.39040536e-02 -1.18091937e-02 -3.91223049e-01]
  [-8.09053968e-03 -1.52109593e-02  5.67532455e-03 -3.87676199e-01]]

 [[ 9.46832003e-03  1.34987906e-02  4.83819891e-03 -1.44555903e+00]
  [ 9.19519290e-03 -1.21918849e-02  1.29567020e-03 -2.00288561e+00]
  [-1.05729733e-02  1.39040536e-02 -1.18091937e-02 -2.30952092e+00]
  [-8.09053968e-03 -1.52109593e-02  5.67532455e-03 -2.24203445e+00]
  [-9.46832003e-03 -1.34987906e-02 -4.83819891e-03 -5.54440974e-01]
  [-9.19519290e-03  1.21918849e-02 -1.29567020e-03  2.88561154e-03]
  [ 1.05729733e-02 -1.39040536e-02  1.18091937

In [235]:
print(feasible_pointBelow)

[-177.51805 -196.24919   24.71498]


feasible_pointAbove:  []
feasible_pointBelow:  [ 5.564  0.582 -6.926 54.07 ]


In [41]:
halfspacesAbove

array([[ 0.06319762,  0.0319443 , -0.01860317, -0.32771872],
       [-0.04572393, -0.04729191, -0.03069523,  0.40426411],
       [ 0.0298448 , -0.00720208,  0.0374721 , -0.04416658],
       [-0.04731849,  0.0225497 ,  0.0118263 , -0.03237881],
       [-0.06319762, -0.0319443 ,  0.01860317, -1.67228128],
       [ 0.04572393,  0.04729191,  0.03069523, -2.40426411],
       [-0.0298448 ,  0.00720208, -0.0374721 , -1.95583342],
       [ 0.04731849, -0.0225497 , -0.0118263 , -1.96762119]])

number of observations:  50434
len(df.columns):  4
feasible_pointAbove:  [-5.  20.   4.4]
feasible_pointBelow:  [ 9.  -1.  15.9]


feasible_pointAboveIndex:  0
feasible_pointBelowIndex:  2
feasible_pointAbove:  [10. 14.]
feasible_pointBelow:  [ 9. -1.]


In [64]:
halfspacesAbove

array([[-0.14285714,  0.14285714],
       [ 0.14285714, -1.14285714],
       [ 0.14285714, -2.14285714],
       [-0.14285714, -0.85714286]])

In [52]:
indexesOfFittedPointsOriginal

[0, 1, 5, 6, 7, 8]

In [42]:
dualVariableValuesOriginal

[-1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -0.0, -1.0, 1.0, -1.0]

feasible_pointAbove:  [10. 14.]
feasible_pointBelow:  [-5. 20.]


In [72]:
vertexTextListAbove

['-3.4,-5.4,24.6', '35.5,5.5,-6.5']

In [71]:
print(hsAbove.intersections)

[[-3.39 35.53]
 [-5.39  5.53]
 [24.61 -6.47]]


In [70]:
returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesBelow = returnInequalitiesCoefficients[1]
print(halfspacesAbove)
print(halfspacesBelow)

[[ 0.09090909  0.06060606 -1.84515152]
 [-0.02597403 -0.06493506  0.21909091]
 [-0.06493506  0.004329   -0.37393939]
 [-0.09090909 -0.06060606 -0.15484848]
 [ 0.02597403  0.06493506 -2.21909091]
 [ 0.06493506 -0.004329   -1.62606061]]
[[-0.09090909 -0.06060606 -0.33      ]
 [ 0.02597403  0.06493506 -2.11857143]
 [ 0.06493506 -0.004329   -1.55142857]
 [ 0.09090909  0.06060606 -1.67      ]
 [-0.02597403 -0.06493506  0.11857143]
 [-0.06493506  0.004329   -0.44857143]]


In [87]:
#Automated 2D Graphing

xs = np.linspace(-50, 50, 4000)

x1ofInertPoints=[]
yofInertPoints=[]
x1ofPoints=[]
yofPoints=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
colorList=[]
textList=[]
vertexAboveMax,vertexAboveMin,vertexBelowMax,vertexBelowMin=\
findInertLimits2D(indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal,dualVariableValuesOriginal)
xs_vertexAboveMax = np.linspace(vertexAboveMax, vertexAboveMax, 4000)
xs_vertexAboveMin = np.linspace(vertexAboveMin, vertexAboveMin, 4000)
xs_vertexBelowMax = np.linspace(vertexBelowMax, vertexBelowMax, 4000)
xs_vertexBelowMin = np.linspace(vertexBelowMin, vertexBelowMin, 4000)
ys_vertexAboveMax= np.linspace((beta0ofL1line+beta1ofL1line*vertexAboveMax), (beta0ofL1line+beta1ofL1line*vertexAboveMax)+ 100, 4000)
ys_vertexAboveMin= np.linspace((beta0ofL1line+beta1ofL1line*vertexAboveMin), (beta0ofL1line+beta1ofL1line*vertexAboveMin)+ 100+\
                            ((beta0ofL1line+beta1ofL1line*vertexAboveMax)-(beta0ofL1line+beta1ofL1line*vertexAboveMin)), 4000)
ys_vertexBelowMax= np.linspace((beta0ofL1line+beta1ofL1line*vertexBelowMax)-50+\
                            ((beta0ofL1line+beta1ofL1line*vertexBelowMin)-(beta0ofL1line+beta1ofL1line*vertexBelowMax)),\
                            (beta0ofL1line+beta1ofL1line*vertexBelowMax), 4000)
ys_vertexBelowMin= np.linspace((beta0ofL1line+beta1ofL1line*vertexBelowMin)-50, (beta0ofL1line+beta1ofL1line*vertexBelowMin), 4000)

L1Line=beta0ofL1line+beta1ofL1line*xs

for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
for i in range(len(explanatoryTrainingOriginal)):
    for j in range(len(explanatoryTrainingOriginal[i])):
        x1ofPoints.append(explanatoryTrainingOriginal[i][j])
        textList.append(f'({explanatoryTrainingOriginal[i][j]},{responseTrainingOriginal[j]})')

for i in range(len(responseTrainingOriginal)):
    locOfPoint=(beta0ofL1line + beta1ofL1line*x1ofPoints[i])
    #print('locOfPoint: ', locOfPoint, f'responseTrainingOriginal[{i}]: ' ,responseTrainingOriginal[i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        #print(locOfPoint,responseTrainingOriginal[i])
        #below of regression hyperplane
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        #print(locOfPoint,responseTrainingOriginal[i])
        #above of regression hyperplane
        colorList.append('green') 
    else:
        #print(locOfPoint,responseTrainingOriginal[i])
        colorList.append('blue')       
    
#text=textList, showlegend=False, textposition="top left", textfont=dict(size=40)),
#text=textList, textposition="top left", textfont=dict(size=20),    
fig=go.Figure(data=[
    go.Scatter(x=xs, y=L1Line, mode='lines', name='L1 line', line=dict(color='black', width=2), showlegend=False),
    go.Scatter(x=x1ofPoints, y=yofPoints, mode='markers+text',marker=dict(color=colorList, size=8), showlegend=False),
    go.Scatter(x=xs_vertexAboveMax, y=ys_vertexAboveMax, mode='lines',marker=dict(color='lightgreen', size=4),  showlegend=False),
    go.Scatter(x=xs_vertexAboveMin, y=ys_vertexAboveMin, mode='lines',marker=dict(color='lightgreen', size=4),fill='tonexty',fillcolor='rgba(26,150,65,0.5)', showlegend=False),
    go.Scatter(x=xs_vertexBelowMax, y=ys_vertexBelowMax, mode='lines',marker=dict(color='salmon', size=4),  showlegend=False),
    go.Scatter(x=xs_vertexBelowMin, y=ys_vertexBelowMin, mode='lines',marker=dict(color='salmon', size=4),fill='tonexty',fillcolor='rgba(225,0,0,0.3)',  showlegend=False),
    #go.Scatter(x=[4], y=[-15], mode='markers+text',marker=dict(color='magenta', size=8),text='(4,-15)', textposition="top left", textfont=dict(size=20), showlegend=False),
])

fig.update_layout(
    plot_bgcolor="white",
    margin=dict(t=10,l=25,b=10,r=10),
    width=700, height=700
)

fig.update_yaxes(title_text='y', title_font = {"size": 20})
fig.update_xaxes(title_text='x',title_font = {"size": 20})
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_xaxes(zeroline=True, zerolinewidth=3, zerolinecolor='LightGray')
fig.update_yaxes(zeroline=True, zerolinewidth=3, zerolinecolor='LightGray')
fig.update_xaxes(range=[np.amin(x1ofPoints)-30,np.amax(x1ofPoints)+30])
fig.update_yaxes(range=[np.amin(yofPoints)-30,np.amax(yofPoints)+30])

#fig.show()
pio.write_html(fig, file='index.html', auto_open=True)

In [78]:
dualVariableValuesOriginal

[-0.8571428571428571, 0.8571428571428572, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0]

feasible_pointAboveIndex:  0
feasible_pointBelowIndex:  2
feasible_pointAbove:  [10. 14.]
feasible_pointBelow:  [-5. 20.]


array([[-4.61, 34.47],
       [-6.61,  4.47],
       [23.39, -7.53]])

[[1, 2], [0, 1], [2, 0]]

array([[  4.,   7.],
       [ 10.,   8.],
       [  9.,  -7.],
       [  2., -23.],
       [ -3.,  -9.],
       [ -4., -24.]])

[[-4.610000000000002, -6.610000000000003, 23.39], [34.47, 4.469999999999997, -7.530000000000001]]


In [161]:
print(len(indexesOfFittedPointsOriginal))

3


In [166]:
baryCenterSet1 = findBarycenterFromVertexes(returnInequalitiesCoefficients[0],dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'above')
baryCenterSet2 = findBarycenterFromVertexes(returnInequalitiesCoefficients[1],dualVariableValuesOriginal,indexesOfFittedPointsOriginal,'below')
print('baryCenterSet1: ', baryCenterSet1)
print('baryCenterSet2: ', baryCenterSet2)

baryCenterSet1:  [31.659999999999922, 69.18002400000012]
baryCenterSet2:  [24.339999999999996, 62.81997599999998]


In [616]:
# 3D AUTOMATED GRAPHING
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
#Set new arrival point coordinates to plot later
#set graph margins here

#--Lists and ranges are defined as empty first.
uOfCones=[]
vOfCones=[]
wOfCones=[]
x1OfCones=[]
x2OfCones=[]
yOfCones=[]
colorList3D=[]

xAxisRange=range(0)
yAxisRange=range(0)
zAxisRange=range(0)

vertexTextListAbove=[]
vertexTextListAboveSet1=[]
vertexTextListAboveSet2=[]
vertexConnectionIndicesAbove = []
vertexConnectionIndicesAboveSet1=[]
vertexConnectionIndicesAboveSet2=[]
x1pointListDefiningAbove=[]
x2pointListDefiningAbove=[]
x1pointListDefiningAboveSet1=[]
x2pointListDefiningAboveSet1=[]
x1pointListDefiningAboveSet2=[]
x2pointListDefiningAboveSet2=[]
ypointListDefiningAbove=[]
ypointListDefiningAboveAux=[]
ypointListDefiningAboveSet1=[]
ypointListDefiningAboveSet2=[]
x1MeshInertZoneAbove=[]    
x2MeshInertZoneAbove=[] 
yMeshInertZoneAbove=[]
x1MeshInertZoneAboveSet1=[]    
x2MeshInertZoneAboveSet1=[] 
yMeshInertZoneAboveSet1=[]
x1MeshInertZoneAboveSet2=[]    
x2MeshInertZoneAboveSet2=[] 
yMeshInertZoneAboveSet2=[]
x1AboveMeshCap=[]    
x2AboveMeshCap=[] 
yAboveMeshCap=[]
x1LinesAbove=[]
yLinesAbove=[]
x2LinesAbove=[]
x1LinesAboveAux=[]
x2LinesAboveAux=[]
yLinesAboveAux=[]
x1LinesAboveSet1=[]
yLinesAboveSet1=[]
x2LinesAboveSet1=[]
x1LinesAboveAuxSet1=[]
x2LinesAboveAuxSet1=[]
yLinesAboveAuxSet1=[]
x1LinesAboveSet2=[]
yLinesAboveSet2=[]
x2LinesAboveSet2=[]
x1LinesAboveAuxSet2=[]
x2LinesAboveAuxSet2=[]
yLinesAboveAuxSet2=[]

vertexTextListBelow=[]
vertexTextListBelowSet1=[]
vertexTextListBelowSet2=[]
vertexConnectionIndicesBelow = []
vertexConnectionIndicesBelowSet1=[]
vertexConnectionIndicesBelowSet2=[]
x1pointListDefiningBelow=[]
x2pointListDefiningBelow=[]
x1pointListDefiningBelowSet1=[]
x2pointListDefiningBelowSet1=[]
x1pointListDefiningBelowSet2=[]
x2pointListDefiningBelowSet2=[]
ypointListDefiningBelow=[]
ypointListDefiningBelowAux=[]
ypointListDefiningBelowSet1=[]
ypointListDefiningBelowSet2=[]
x1MeshInertZoneBelow=[]    
x2MeshInertZoneBelow=[] 
yMeshInertZoneBelow=[]
x1MeshInertZoneBelowSet1=[]    
x2MeshInertZoneBelowSet1=[] 
yMeshInertZoneBelowSet1=[]
x1MeshInertZoneBelowSet2=[]    
x2MeshInertZoneBelowSet2=[] 
yMeshInertZoneBelowSet2=[]
x1BelowMeshCap=[]    
x2BelowMeshCap=[] 
yBelowMeshCap=[]
x1LinesBelow=[]
yLinesBelow=[]
x2LinesBelow=[]
x1LinesBelowAux=[]
x2LinesBelowAux=[]
yLinesBelowAux=[]
x1LinesBelowSet1=[]
yLinesBelowSet1=[]
x2LinesBelowSet1=[]
x1LinesBelowAuxSet1=[]
x2LinesBelowAuxSet1=[]
yLinesBelowAuxSet1=[]
x1LinesBelowSet2=[]
yLinesBelowSet2=[]
x2LinesBelowSet2=[]
x1LinesBelowAuxSet2=[]
x2LinesBelowAuxSet2=[]
yLinesBelowAuxSet2=[]
#--------------------------------------------------
dimensionL1=len(solutionOriginal['betaValues'])

beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
colorList=[]
if inertPointList:
    for i in range(len(inertPointList[0][0])):
        xofInertPoints.append(list())

    for i in range(len(inertPointList)):
        yofInertPoints.append(inertPointList[i][1][0])
        for j in range(len(inertPointList[0][0])):
            xofInertPoints[j].append(inertPointList[i][0][j][0])

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])

for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
#xMinLimit=min(np.min(xofPoints[0]),np.min(x1pointListDefiningAbove), np.min(x1pointListDefiningBelow),\
#              np.min(x1pointListDefiningAboveSet1), np.min(x1pointListDefiningAboveSet2),\
#             np.min(x1pointListDefiningBelowSet1), np.min(x1pointListDefiningBelowSet2))
#xMaxLimit=max(np.max(xofPoints[0]),np.max(x1pointListDefiningAbove), np.max(x1pointListDefiningBelow),\
#              np.max(x1pointListDefiningAboveSet1), np.max(x1pointListDefiningAboveSet2),\
#             np.max(x1pointListDefiningBelowSet1), np.max(x1pointListDefiningBelowSet2))        
#ypointListDefiningBelow.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningBelow[i] +\
#                               beta2ofL1line*x2pointListDefiningBelow[i]-0.1)
zAxisRange=(min(np.min(yofPoints)*2,-10)*1.2,max(np.max(yofPoints)*2,10)*1.2)
#
#zAxisRange=(-10,50)
#------------------------------------------------------------------------------------------------
#xs = np.linspace(-60, 60, 4000)

#---
#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
#x1s = np.linspace(-50, 50, 500)
#x2s = np.linspace(-50, 50, 500)
#ys = np.linspace(-30,40,500)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
#x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
#x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
#x1meshy, ymeshx1 = np.meshgrid(x1s, ys)


#L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1
fig = go.Figure(data=[
##### SAE graph
#go.Surface(x=x1meshx2, y=x2meshx1, z=L1Hyperplane, colorscale='gray', showscale=False, opacity=1),
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=yofPoints, mode='markers+text',marker=dict(color=colorList, size=3),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=yofInertPoints, mode='markers+text',marker=dict(color='red', size=1.2), showlegend=False),
])
#--------------------------------BELOW
#feasiblePointLocation=[-18.438,-17.344,-30.123]
halfspacesBelow = returnInequalitiesCoefficients[1]
halfspacesAbove = returnInequalitiesCoefficients[0]
#---------BELOW------------------------------------
try:
    halfspacesBelow = returnInequalitiesCoefficients[1] 
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
#-------
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    hullPyBelow = qconvexPy("i",hsBelow.intersections)
    vertexConnectionIndicesBelow = []

    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1

    vertexesBelow=hsBelow.intersections
    vertexTextListBelow=[]
    vertexListBelow=reorderHalfspaceIntersections(vertexConnectionIndicesBelow,hsBelow.intersections)
    x1pointListDefiningBelow=vertexListBelow[0]
    x2pointListDefiningBelow=vertexListBelow[1]

    #x1pointListDefiningBelow=[10,16,9]
    #x2pointListDefiningBelow=[20,21,5]
    ypointListDefiningBelow=[]
    ypointListDefiningBelowAux=[]
    
    
    for i in range(len(x1pointListDefiningBelow)):
        ypointListDefiningBelow.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningBelow[i] +\
                                       beta2ofL1line*x2pointListDefiningBelow[i]-0.1)
        ypointListDefiningBelowAux.append(zAxisRange[0])
    

    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')
          
    
    x1MeshInertZoneBelow=[]    
    x2MeshInertZoneBelow=[] 
    yMeshInertZoneBelow=[]

    
    for i in range(len(x1pointListDefiningBelow)):
        x1MeshInertZoneBelow.append([])
        x2MeshInertZoneBelow.append([])
        yMeshInertZoneBelow.append([])  
    
    
    jk=0    
    for i in range(len(x1pointListDefiningBelow)):

        if i == len(x1pointListDefiningBelow)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneBelow[i].append(x1pointListDefiningBelow[i])
        x2MeshInertZoneBelow[i].append(x2pointListDefiningBelow[i])
        yMeshInertZoneBelow[i].append(ypointListDefiningBelow[i])
        #2
        x1MeshInertZoneBelow[i].append(x1pointListDefiningBelow[jk])
        x2MeshInertZoneBelow[i].append(x2pointListDefiningBelow[jk])
        yMeshInertZoneBelow[i].append(ypointListDefiningBelow[jk])
        #3
        x1MeshInertZoneBelow[i].append(x1pointListDefiningBelow[i])
        x2MeshInertZoneBelow[i].append(x2pointListDefiningBelow[i])
        yMeshInertZoneBelow[i].append(ypointListDefiningBelowAux[i])
        #4
        x1MeshInertZoneBelow[i].append(x1pointListDefiningBelow[jk])
        x2MeshInertZoneBelow[i].append(x2pointListDefiningBelow[jk])
        yMeshInertZoneBelow[i].append(ypointListDefiningBelowAux[jk])
#------        
    x1BelowMeshCap=[]    
    x2BelowMeshCap=[] 
    yBelowMeshCap=[]
    for i in range(len(x1pointListDefiningBelow)):
        x1BelowMeshCap.append(x1pointListDefiningBelow[len(x1pointListDefiningBelow)-1-i])
        x2BelowMeshCap.append(x2pointListDefiningBelow[len(x1pointListDefiningBelow)-1-i])
        yBelowMeshCap.append(ypointListDefiningBelowAux[len(x1pointListDefiningBelow)-1-i])
#------        
    x1LinesBelow=[]
    yLinesBelow=[]
    x2LinesBelow=[]
    
    x1LinesBelowAux=[]
    x2LinesBelowAux=[]
    yLinesBelowAux=[]
    jk=0
    for i in range(len(x1pointListDefiningBelow)):
        if i == len(x1pointListDefiningBelow)-1:
            jk=0
        else:
            jk=i+1
        x1LinesBelow.append([x1pointListDefiningBelow[i],x1pointListDefiningBelow[jk]])
        x2LinesBelow.append([x2pointListDefiningBelow[i],x2pointListDefiningBelow[jk]])
        yLinesBelow.append([ypointListDefiningBelow[i],ypointListDefiningBelow[jk]])
    for i in range(len(x1pointListDefiningBelow)):
        x1LinesBelowAux.append([x1pointListDefiningBelow[i],x1pointListDefiningBelow[i]])
        x2LinesBelowAux.append([x2pointListDefiningBelow[i],x2pointListDefiningBelow[i]])
        yLinesBelowAux.append([ypointListDefiningBelow[i],ypointListDefiningBelowAux[i]])

#---------
    for i in range(len(x1LinesBelowAux)):
        fig.add_trace(go.Scatter3d(x=x1LinesBelowAux[i],y=x2LinesBelowAux[i], z=yLinesBelowAux[i], mode='lines',\
                                   line=dict(color='lightpink', width=3), showlegend=False))

    for i in range(len(x1pointListDefiningBelow)):
        fig.add_trace(go.Scatter3d(x=x1MeshInertZoneBelow[i], y=x2MeshInertZoneBelow[i], z=yMeshInertZoneBelow[i],mode='markers',\
                                   marker=dict(color='lightpink', size=2),\
                               showlegend=False,surfacecolor='lightpink',surfaceaxis=1,opacity=0.5))

    for i in range(len(x1LinesBelow)):
        fig.add_trace(go.Scatter3d(x=x1LinesBelow[i],y=x2LinesBelow[i], z=yLinesBelow[i], mode='lines',\
                                   marker=dict(color='lightpink', size=5), showlegend=False))

    #Inert zone top facets. (cap) mediumslateblue

    fig.add_trace(go.Mesh3d(x=x1BelowMeshCap, y=x2BelowMeshCap, z=yBelowMeshCap,\
                            color='lightpink', opacity=0.50,showlegend=False))

    #Below
    for i in range(len(x1pointListDefiningBelow)):
            colorList3D.append('magenta')
            uOfCones.append([0])
            vOfCones.append([0])
            wOfCones.append([-2])
            x1OfCones.append([x1pointListDefiningBelow[i]])
            x2OfCones.append([x2pointListDefiningBelow[i]])
            yOfCones.append([ypointListDefiningBelowAux[i]])
except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
    #These values need to set to set axis limits
    x1pointListDefiningBelow.append(0)
    x2pointListDefiningBelow.append(0)

#---------BELOW SET 1------------------------------------

try:    
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
#-------
    hsBelowSet1 = HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelowSet1)
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)
    vertexConnectionIndicesBelowSet1=[]
    

    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1

    vertexesBelowSet1=hsBelowSet1.intersections
    vertexTextListBelowSet1=[]
    vertexListBelowSet1=reorderHalfspaceIntersections(vertexConnectionIndicesBelowSet1,hsBelowSet1.intersections)
    x1pointListDefiningBelowSet1=vertexListBelowSet1[0]
    x2pointListDefiningBelowSet1=vertexListBelowSet1[1]
    #x1pointListDefiningBelow=[10,16,9]
    #x2pointListDefiningBelow=[20,21,5]
    ypointListDefiningBelowSet1=[]
    

    for i in range(len(x1pointListDefiningBelowSet1)):
        ypointListDefiningBelowSet1.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningBelowSet1[i] +\
                                       beta2ofL1line*x2pointListDefiningBelowSet1[i]-0.1)    

  
    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')

    
    x1MeshInertZoneBelowSet1=[]    
    x2MeshInertZoneBelowSet1=[] 
    yMeshInertZoneBelowSet1=[]
    
    
    for i in range(len(x1pointListDefiningBelowSet1)):
        x1MeshInertZoneBelowSet1.append([])
        x2MeshInertZoneBelowSet1.append([])
        yMeshInertZoneBelowSet1.append([])        

    
    jk=0    
    for i in range(len(x1pointListDefiningBelowSet1)):

        if i == len(x1pointListDefiningBelowSet1)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneBelowSet1[i].append(x1pointListDefiningBelowSet1[i])
        x2MeshInertZoneBelowSet1[i].append(x2pointListDefiningBelowSet1[i])
        yMeshInertZoneBelowSet1[i].append(ypointListDefiningBelowSet1[i])
        #2
        x1MeshInertZoneBelowSet1[i].append(x1pointListDefiningBelowSet1[jk])
        x2MeshInertZoneBelowSet1[i].append(x2pointListDefiningBelowSet1[jk])
        yMeshInertZoneBelowSet1[i].append(ypointListDefiningBelowSet1[jk])
        #3
        x1MeshInertZoneBelowSet1[i].append(x1pointListDefiningBelowSet1[i])
        x2MeshInertZoneBelowSet1[i].append(x2pointListDefiningBelowSet1[i])
        #yMeshInertZoneBelowSet1[i].append(ypointListDefiningBelowAux[i])
        #4
        x1MeshInertZoneBelowSet1[i].append(x1pointListDefiningBelowSet1[jk])
        x2MeshInertZoneBelowSet1[i].append(x2pointListDefiningBelowSet1[jk])
        #yMeshInertZoneBelowSet1[i].append(ypointListDefiningBelowAux[jk])

#------
    x1LinesBelowSet1=[]
    yLinesBelowSet1=[]
    x2LinesBelowSet1=[]
    
    x1LinesBelowAuxSet1=[]
    x2LinesBelowAuxSet1=[]
    #yLinesBelowAuxSet1=[]
    jk=0
    for i in range(len(x1pointListDefiningBelowSet1)):
        if i == len(x1pointListDefiningBelowSet1)-1:
            jk=0
        else:
            jk=i+1
        x1LinesBelowSet1.append([x1pointListDefiningBelowSet1[i],x1pointListDefiningBelowSet1[jk]])
        x2LinesBelowSet1.append([x2pointListDefiningBelowSet1[i],x2pointListDefiningBelowSet1[jk]])
        yLinesBelowSet1.append([ypointListDefiningBelowSet1[i],ypointListDefiningBelowSet1[jk]])
    for i in range(len(x1pointListDefiningBelowSet1)):
        x1LinesBelowAuxSet1.append([x1pointListDefiningBelowSet1[i],x1pointListDefiningBelowSet1[i]])
        x2LinesBelowAuxSet1.append([x2pointListDefiningBelowSet1[i],x2pointListDefiningBelowSet1[i]])
        #yLinesBelowAuxSet1.append([ypointListDefiningBelowSet1[i],ypointListDefiningBelowAux[i]])

#---------

    for i in range(len(x1LinesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=x1LinesBelowSet1[i],y=x2LinesBelowSet1[i], z=yLinesBelowSet1[i], mode='lines',\
                                   marker=dict(color='indigo', size=8), showlegend=False))

except Exception as e:
    print('Problem in obtaining the inert zone below set 1.')
    print(e)
    x1pointListDefiningBelowSet1.append(0)
    x2pointListDefiningBelowSet1.append(0)
#---------BELOW SET 2------------------------------------

try:

    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    feasible_pointBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet2)
#-------
    hsBelowSet2 = HalfspaceIntersection(halfspacesBelowSet2, feasible_pointBelowSet2)
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    vertexConnectionIndicesBelowSet2=[]
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1
    vertexesBelowSet2=hsBelowSet2.intersections
    vertexTextListBelowSet2=[]
    vertexListBelowSet2=reorderHalfspaceIntersections(vertexConnectionIndicesBelowSet2,hsBelowSet2.intersections)
    x1pointListDefiningBelowSet2=vertexListBelowSet2[0]
    x2pointListDefiningBelowSet2=vertexListBelowSet2[1]
    ypointListDefiningBelowSet2=[]
    
       
    for i in range(len(x1pointListDefiningBelowSet2)):
        ypointListDefiningBelowSet2.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningBelowSet2[i] +\
                                       beta2ofL1line*x2pointListDefiningBelowSet2[i]-0.1)    

    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')
    
    x1MeshInertZoneBelowSet2=[]    
    x2MeshInertZoneBelowSet2=[] 
    yMeshInertZoneBelowSet2=[]
    
    for i in range(len(x1pointListDefiningBelowSet2)):
        x1MeshInertZoneBelowSet2.append([])
        x2MeshInertZoneBelowSet2.append([])
        yMeshInertZoneBelowSet2.append([])    
    
    
    jk=0    
    for i in range(len(x1pointListDefiningBelowSet2)):

        if i == len(x1pointListDefiningBelowSet2)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneBelowSet2[i].append(x1pointListDefiningBelowSet2[i])
        x2MeshInertZoneBelowSet2[i].append(x2pointListDefiningBelowSet2[i])
        yMeshInertZoneBelowSet2[i].append(ypointListDefiningBelowSet2[i])
        #2
        x1MeshInertZoneBelowSet2[i].append(x1pointListDefiningBelowSet2[jk])
        x2MeshInertZoneBelowSet2[i].append(x2pointListDefiningBelowSet2[jk])
        yMeshInertZoneBelowSet2[i].append(ypointListDefiningBelowSet2[jk])
        #3
        x1MeshInertZoneBelowSet2[i].append(x1pointListDefiningBelowSet2[i])
        x2MeshInertZoneBelowSet2[i].append(x2pointListDefiningBelowSet2[i])
        #yMeshInertZoneBelowSet2[i].append(ypointListDefiningBelowAux[i])
        #4
        x1MeshInertZoneBelowSet2[i].append(x1pointListDefiningBelowSet2[jk])
        x2MeshInertZoneBelowSet2[i].append(x2pointListDefiningBelowSet2[jk])
        #yMeshInertZoneBelowSet2[i].append(ypointListDefiningBelowAux[jk])

#------        
#------        
    x1LinesBelowSet2=[]
    yLinesBelowSet2=[]
    x2LinesBelowSet2=[]
    
    x1LinesBelowAuxSet2=[]
    x2LinesBelowAuxSet2=[]
    yLinesBelowAuxSet2=[]
    jk=0
    for i in range(len(x1pointListDefiningBelowSet2)):
        if i == len(x1pointListDefiningBelowSet2)-1:
            jk=0
        else:
            jk=i+1
        x1LinesBelowSet2.append([x1pointListDefiningBelowSet2[i],x1pointListDefiningBelowSet2[jk]])
        x2LinesBelowSet2.append([x2pointListDefiningBelowSet2[i],x2pointListDefiningBelowSet2[jk]])
        yLinesBelowSet2.append([ypointListDefiningBelowSet2[i],ypointListDefiningBelowSet2[jk]])
    for i in range(len(x1pointListDefiningBelowSet2)):
        x1LinesBelowAuxSet2.append([x1pointListDefiningBelowSet2[i],x1pointListDefiningBelowSet2[i]])
        x2LinesBelowAuxSet2.append([x2pointListDefiningBelowSet2[i],x2pointListDefiningBelowSet2[i]])
        #yLinesBelowAuxSet2.append([ypointListDefiningBelowSet2[i],ypointListDefiningBelowAux[i]])
#------
#---------

    for i in range(len(x1LinesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=x1LinesBelowSet2[i],y=x2LinesBelowSet2[i], z=yLinesBelowSet2[i], mode='lines',\
                                   marker=dict(color='mediumslateblue', size=8), showlegend=False))
    #Inert zone top facets. (cap) mediumslateblue

except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
    #These values need to set to set axis limits
    x1pointListDefiningBelowSet2.append(0)
    x2pointListDefiningBelowSet2.append(0)

#--------------------------------ABOVE------------------------------------------
try:
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    vertexConnectionIndicesAbove = []

    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1

    vertexesAbove=hsAbove.intersections
    vertexTextListAbove=[]
    vertexListAbove=reorderHalfspaceIntersections(vertexConnectionIndicesAbove,hsAbove.intersections)
    x1pointListDefiningAbove=vertexListAbove[0]
    x2pointListDefiningAbove=vertexListAbove[1]
    ypointListDefiningAbove=[]
    ypointListDefiningAboveAux=[]

    
    
    for i in range(len(x1pointListDefiningAbove)):
        ypointListDefiningAbove.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningAbove[i] +\
                                       beta2ofL1line*x2pointListDefiningAbove[i]+0.1)
        ypointListDefiningAboveAux.append(zAxisRange[1])
      

    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')
        
    
    x1MeshInertZoneAbove=[]    
    x2MeshInertZoneAbove=[] 
    yMeshInertZoneAbove=[]

    
    for i in range(len(x1pointListDefiningAbove)):
        x1MeshInertZoneAbove.append([])
        x2MeshInertZoneAbove.append([])
        yMeshInertZoneAbove.append([])

    
    
    jk=0    
    for i in range(len(x1pointListDefiningAbove)):

        if i == len(x1pointListDefiningAbove)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneAbove[i].append(x1pointListDefiningAbove[i])
        x2MeshInertZoneAbove[i].append(x2pointListDefiningAbove[i])
        yMeshInertZoneAbove[i].append(ypointListDefiningAbove[i])
        #2
        x1MeshInertZoneAbove[i].append(x1pointListDefiningAbove[jk])
        x2MeshInertZoneAbove[i].append(x2pointListDefiningAbove[jk])
        yMeshInertZoneAbove[i].append(ypointListDefiningAbove[jk])
        #3
        x1MeshInertZoneAbove[i].append(x1pointListDefiningAbove[i])
        x2MeshInertZoneAbove[i].append(x2pointListDefiningAbove[i])
        yMeshInertZoneAbove[i].append(ypointListDefiningAboveAux[i])
        #4
        x1MeshInertZoneAbove[i].append(x1pointListDefiningAbove[jk])
        x2MeshInertZoneAbove[i].append(x2pointListDefiningAbove[jk])
        yMeshInertZoneAbove[i].append(ypointListDefiningAboveAux[jk])

#----

#------        
    x1AboveMeshCap=[]    
    x2AboveMeshCap=[] 
    yAboveMeshCap=[]
    for i in range(len(x1pointListDefiningAbove)):
        x1AboveMeshCap.append(x1pointListDefiningAbove[len(x1pointListDefiningAbove)-1-i])
        x2AboveMeshCap.append(x2pointListDefiningAbove[len(x1pointListDefiningAbove)-1-i])
        yAboveMeshCap.append(ypointListDefiningAboveAux[len(x1pointListDefiningAbove)-1-i])
#------        
    x1LinesAbove=[]
    yLinesAbove=[]
    x2LinesAbove=[]
    
    x1LinesAboveAux=[]
    x2LinesAboveAux=[]
    yLinesAboveAux=[]
    jk=0
    for i in range(len(x1pointListDefiningAbove)):
        if i == len(x1pointListDefiningAbove)-1:
            jk=0
        else:
            jk=i+1
        x1LinesAbove.append([x1pointListDefiningAbove[i],x1pointListDefiningAbove[jk]])
        x2LinesAbove.append([x2pointListDefiningAbove[i],x2pointListDefiningAbove[jk]])
        yLinesAbove.append([ypointListDefiningAbove[i],ypointListDefiningAbove[jk]])
    for i in range(len(x1pointListDefiningAbove)):
        x1LinesAboveAux.append([x1pointListDefiningAbove[i],x1pointListDefiningAbove[i]])
        x2LinesAboveAux.append([x2pointListDefiningAbove[i],x2pointListDefiningAbove[i]])
        yLinesAboveAux.append([ypointListDefiningAbove[i],ypointListDefiningAboveAux[i]])
#------

#------
    
    #Connecting lines on the L1 hyperplane
    for i in range(len(x1LinesAbove)):
        fig.add_trace(go.Scatter3d(x=x1LinesAbove[i],y=x2LinesAbove[i], z=yLinesAbove[i], mode='lines',\
                                   marker=dict(color='lightgreen', size=5), showlegend=False))
 
    #Aux connecting lines, vertical
    for i in range(len(x1LinesAbove)):
        fig.add_trace(go.Scatter3d(x=x1LinesAboveAux[i],y=x2LinesAboveAux[i], z=yLinesAboveAux[i], mode='lines',\
                                   line=dict(color='lightgreen', width=3),\
                                   showlegend=False))
    #Inert zone side facets.
    for i in range(len(x1pointListDefiningAbove)):
        fig.add_trace(go.Scatter3d(x=x1MeshInertZoneAbove[i], y=x2MeshInertZoneAbove[i], z=yMeshInertZoneAbove[i],mode='markers',\
                                       marker=dict(color='lightgreen', size=2),\
                                   showlegend=False,surfacecolor='lightgreen',surfaceaxis=0, opacity=0.5))

    fig.add_trace(go.Mesh3d(x=x1AboveMeshCap, y=x2AboveMeshCap, z=yAboveMeshCap,\
                            color='lightgreen', opacity=0.50,showlegend=False))

    #--- set paramaters above first, below next
    for i in range(len(x1pointListDefiningAbove)):
            colorList3D.append('greens')
            uOfCones.append([0])
            vOfCones.append([0])
            wOfCones.append([2])
            x1OfCones.append([x1pointListDefiningAbove[i]])
            x2OfCones.append([x2pointListDefiningAbove[i]])
            yOfCones.append([ypointListDefiningAboveAux[i]]) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
    #These values need to set to set axis limits
    x2pointListDefiningAbove.append(0)
    x1pointListDefiningAbove.append(0)
#--------------------------------ABOVE SET 1
try:
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])

    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    
    hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    vertexConnectionIndicesAboveSet1=[]

    counter=0
    for item in hullPyAboveSet1:
        if counter>0:
            vertexConnectionIndicesAboveSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
        counter+=1

    vertexesAboveSet1=hsAboveSet1.intersections
    vertexTextListAboveSet1=[]
    vertexListAboveSet1=reorderHalfspaceIntersections(vertexConnectionIndicesAboveSet1,hsAboveSet1.intersections)

    x1pointListDefiningAboveSet1=vertexListAboveSet1[0]
    x2pointListDefiningAboveSet1=vertexListAboveSet1[1]

    ypointListDefiningAboveSet1=[]
    
    
    
    for i in range(len(x1pointListDefiningAboveSet1)):
        ypointListDefiningAboveSet1.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningAboveSet1[i] +\
                                       beta2ofL1line*x2pointListDefiningAboveSet1[i]+0.1)    

        
    for i in range(len(vertexesAboveSet1[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')

    

    x1MeshInertZoneAboveSet1=[]    
    x2MeshInertZoneAboveSet1=[] 
    yMeshInertZoneAboveSet1=[]
    

    for i in range(len(x1pointListDefiningAboveSet1)):
        x1MeshInertZoneAboveSet1.append([])
        x2MeshInertZoneAboveSet1.append([])
        yMeshInertZoneAboveSet1.append([])        

#----
    jk=0    
    for i in range(len(x1pointListDefiningAboveSet1)):

        if i == len(x1pointListDefiningAboveSet1)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneAboveSet1[i].append(x1pointListDefiningAboveSet1[i])
        x2MeshInertZoneAboveSet1[i].append(x2pointListDefiningAboveSet1[i])
        yMeshInertZoneAboveSet1[i].append(ypointListDefiningAboveSet1[i])
        #2
        x1MeshInertZoneAboveSet1[i].append(x1pointListDefiningAboveSet1[jk])
        x2MeshInertZoneAboveSet1[i].append(x2pointListDefiningAboveSet1[jk])
        yMeshInertZoneAboveSet1[i].append(ypointListDefiningAboveSet1[jk])
        #3
        x1MeshInertZoneAboveSet1[i].append(x1pointListDefiningAboveSet1[i])
        x2MeshInertZoneAboveSet1[i].append(x2pointListDefiningAboveSet1[i])
        #yMeshInertZoneAboveSet1[i].append(ypointListDefiningAboveAux[i])
        #4
        x1MeshInertZoneAboveSet1[i].append(x1pointListDefiningAboveSet1[jk])
        x2MeshInertZoneAboveSet1[i].append(x2pointListDefiningAboveSet1[jk])
        #yMeshInertZoneAboveSet1[i].append(ypointListDefiningAboveAux[jk])


#------        
    x1LinesAboveSet1=[]
    yLinesAboveSet1=[]
    x2LinesAboveSet1=[]
    
    x1LinesAboveAuxSet1=[]
    x2LinesAboveAuxSet1=[]
    yLinesAboveAuxSet1=[]
    jk=0
    for i in range(len(x1pointListDefiningAboveSet1)):
        if i == len(x1pointListDefiningAboveSet1)-1:
            jk=0
        else:
            jk=i+1
        x1LinesAboveSet1.append([x1pointListDefiningAboveSet1[i],x1pointListDefiningAboveSet1[jk]])
        x2LinesAboveSet1.append([x2pointListDefiningAboveSet1[i],x2pointListDefiningAboveSet1[jk]])
        yLinesAboveSet1.append([ypointListDefiningAboveSet1[i],ypointListDefiningAboveSet1[jk]])
    for i in range(len(x1pointListDefiningAboveSet1)):
        x1LinesAboveAuxSet1.append([x1pointListDefiningAboveSet1[i],x1pointListDefiningAboveSet1[i]])
        x2LinesAboveAuxSet1.append([x2pointListDefiningAboveSet1[i],x2pointListDefiningAboveSet1[i]])
        #yLinesAboveAuxSet1.append([ypointListDefiningAboveSet1[i],ypointListDefiningAboveAux[i]])
#--------------
    
    #Connecting lines on the L1 hyperplane
    for i in range(len(x1LinesAboveSet1)):
        fig.add_trace(go.Scatter3d(x=x1LinesAboveSet1[i],y=x2LinesAboveSet1[i], z=yLinesAboveSet1[i], mode='lines',\
                                   marker=dict(color='lime', size=8), showlegend=False))

            
except Exception as e:
    print('Problem in obtaining the inert zone above Set 1.')
    print(e)
    #These values need to set to set axis limits
    x1pointListDefiningAboveSet1.append(0)
    x2pointListDefiningAboveSet1.append(0)
#--------------------------------ABOVE SET 2
try:
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)
    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    vertexConnectionIndicesAboveSet2=[]
    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
        
    vertexesAboveSet2=hsAboveSet2.intersections
    vertexTextListAboveSet2=[]
    vertexListAboveSet2=reorderHalfspaceIntersections(vertexConnectionIndicesAboveSet2,hsAboveSet2.intersections)
    x1pointListDefiningAboveSet2=vertexListAboveSet2[0]
    x2pointListDefiningAboveSet2=vertexListAboveSet2[1]
    ypointListDefiningAboveSet2=[]
       
    for i in range(len(x1pointListDefiningAboveSet2)):
        ypointListDefiningAboveSet2.append(beta0ofL1line + beta1ofL1line*x1pointListDefiningAboveSet2[i] +\
                                       beta2ofL1line*x2pointListDefiningAboveSet2[i]+0.1)    
    for i in range(len(vertexesAboveSet2[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')
    
    x1MeshInertZoneAboveSet2=[]    
    x2MeshInertZoneAboveSet2=[] 
    yMeshInertZoneAboveSet2=[]
    
    for i in range(len(x1pointListDefiningAboveSet2)):
        x1MeshInertZoneAboveSet2.append([])
        x2MeshInertZoneAboveSet2.append([])
        yMeshInertZoneAboveSet2.append([])    
    
#----
    jk=0    
    for i in range(len(x1pointListDefiningAboveSet2)):

        if i == len(x1pointListDefiningAboveSet2)-1:
            jk=0
        else:
            jk=i+1
        #1
        x1MeshInertZoneAboveSet2[i].append(x1pointListDefiningAboveSet2[i])
        x2MeshInertZoneAboveSet2[i].append(x2pointListDefiningAboveSet2[i])
        yMeshInertZoneAboveSet2[i].append(ypointListDefiningAboveSet2[i])
        #2
        x1MeshInertZoneAboveSet2[i].append(x1pointListDefiningAboveSet2[jk])
        x2MeshInertZoneAboveSet2[i].append(x2pointListDefiningAboveSet2[jk])
        yMeshInertZoneAboveSet2[i].append(ypointListDefiningAboveSet2[jk])
        #3
        x1MeshInertZoneAboveSet2[i].append(x1pointListDefiningAboveSet2[i])
        x2MeshInertZoneAboveSet2[i].append(x2pointListDefiningAboveSet2[i])
        #yMeshInertZoneAboveSet2[i].append(ypointListDefiningAboveAux[i])
        #4
        x1MeshInertZoneAboveSet2[i].append(x1pointListDefiningAboveSet2[jk])
        x2MeshInertZoneAboveSet2[i].append(x2pointListDefiningAboveSet2[jk])
        #yMeshInertZoneAboveSet2[i].append(ypointListDefiningAboveAux[jk])

#------
    x1LinesAboveSet2=[]
    yLinesAboveSet2=[]
    x2LinesAboveSet2=[]
    
    x1LinesAboveAuxSet2=[]
    x2LinesAboveAuxSet2=[]
    yLinesAboveAuxSet2=[]
    jk=0
    for i in range(len(x1pointListDefiningAboveSet2)):
        if i == len(x1pointListDefiningAboveSet2)-1:
            jk=0
        else:
            jk=i+1
        x1LinesAboveSet2.append([x1pointListDefiningAboveSet2[i],x1pointListDefiningAboveSet2[jk]])
        x2LinesAboveSet2.append([x2pointListDefiningAboveSet2[i],x2pointListDefiningAboveSet2[jk]])
        yLinesAboveSet2.append([ypointListDefiningAboveSet2[i],ypointListDefiningAboveSet2[jk]])
    for i in range(len(x1pointListDefiningAboveSet2)):
        x1LinesAboveAuxSet2.append([x1pointListDefiningAboveSet2[i],x1pointListDefiningAboveSet2[i]])
        x2LinesAboveAuxSet2.append([x2pointListDefiningAboveSet2[i],x2pointListDefiningAboveSet2[i]])
        #yLinesAboveAuxSet2.append([ypointListDefiningAboveSet2[i],ypointListDefiningAboveAux[i]])
#------
    #Connecting lines on the L1 hyperplane
    for i in range(len(x1LinesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=x1LinesAboveSet2[i],y=x2LinesAboveSet2[i], z=yLinesAboveSet2[i], mode='lines',\
                               marker=dict(color='darkgreen', size=8), showlegend=False))    
    #Aux connecting lines, vertical
    #Inert zone side facets.
            
except Exception as e:
    print('Problem in obtaining the inert zone above Set 2.')
    print(e)
    #These values need to set to set axis limits
    x1pointListDefiningAboveSet2.append(0)
    x2pointListDefiningAboveSet2.append(0)

#Draw cones at the margins
#colorList3D=[]
#----------------------------------------------------------------------

#--- add cones (values are collected in try-except of "Above" and "Below")
for i in range(len(x1OfCones)):
    fig.add_trace(go.Cone(x=x1OfCones[i], y=x2OfCones[i], z=yOfCones[i], u=uOfCones[i], v=vOfCones[i], w=wOfCones[i],\
                      showlegend=False,showscale=False, colorscale=colorList3D[i]))
#--------------------------------------------------------------------------------------------------------------------
#zAxisRange is defined above at line 43.
yMinLimit=min(np.min(xofPoints[1]),np.min(x2pointListDefiningAbove), np.min(x2pointListDefiningBelow),\
              np.min(x2pointListDefiningAboveSet1), np.min(x2pointListDefiningAboveSet2),\
             np.min(x2pointListDefiningBelowSet1), np.min(x2pointListDefiningBelowSet2))
yMaxLimit=max(np.max(xofPoints[1]),np.max(x2pointListDefiningAbove), np.max(x2pointListDefiningBelow),\
              np.max(x2pointListDefiningAboveSet1), np.max(x2pointListDefiningAboveSet2),\
             np.max(x2pointListDefiningBelowSet1), np.max(x2pointListDefiningBelowSet2))
xMinLimit=min(np.min(xofPoints[0]),np.min(x1pointListDefiningAbove), np.min(x1pointListDefiningBelow),\
              np.min(x1pointListDefiningAboveSet1), np.min(x1pointListDefiningAboveSet2),\
             np.min(x1pointListDefiningBelowSet1), np.min(x1pointListDefiningBelowSet2))
xMaxLimit=max(np.max(xofPoints[0]),np.max(x1pointListDefiningAbove), np.max(x1pointListDefiningBelow),\
              np.max(x1pointListDefiningAboveSet1), np.max(x1pointListDefiningAboveSet2),\
             np.max(x1pointListDefiningBelowSet1), np.max(x1pointListDefiningBelowSet2))
##### SAE graph
#------------------------------------------------------------------------------------------------
xs = np.linspace(-60, 60, 4000)

#---
#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(xMinLimit, xMaxLimit, 500)
x2s = np.linspace(yMinLimit, yMaxLimit, 500)
ys = np.linspace(-30,40,500)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)
L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1
fig.add_trace(go.Surface(x=x1meshx2, y=x2meshx1, z=L1Hyperplane, colorscale='gray', showscale=False, opacity=1))

yAxisRange=(yMinLimit,yMaxLimit)
xAxisRange=(xMinLimit,xMaxLimit)
#yAxisRange=(-1,20)
#xAxisRange=(0,3)
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='y',
                    zaxis_range=zAxisRange,
                    yaxis_range=yAxisRange,
                    xaxis_range=xAxisRange,
                )
)


#Normally plotly-python has a gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(225, 225, 225)",
                         gridcolor="rgb(192,192,192)",
                         showbackground=False,
                         zerolinecolor="black",),
                    yaxis = dict(
                        backgroundcolor="rgb(225, 225, 225)",
                        gridcolor="rgb(192,192,192)",
                        showbackground=False,
                        zerolinecolor="black"),
                    zaxis = dict(
                        backgroundcolor="rgb(225, 225, 225)",
                        gridcolor="rgb(192,192,192)",
                        showbackground=False,
                        zerolinecolor="rgb(192,192,192)",),),
                    #following lines set margins for the figure.
                    width=1200,
                    margin=dict(
                    r=10, l=10,
                    b=10, t=10)
                  )

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

In [93]:
min(1,2,3)

1

In [34]:
print(halfspacesAbove)
print(len(halfspacesAbove)/2)
halfspacesAboveSet1=[halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))]
halfspacesAboveSet2=[halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))]
print(halfspacesAboveSet1)
print(halfspacesAboveSet2)


[[ 0.06319762  0.0319443  -0.01860317 -1.87546508]
 [-0.04572393 -0.04729191 -0.03069523  0.40548262]
 [ 0.0298448  -0.00720208  0.0374721   0.0634368 ]
 [-0.04731849  0.0225497   0.0118263  -1.59345434]
 [-0.06319762 -0.0319443   0.01860317 -0.12453492]
 [ 0.04572393  0.04729191  0.03069523 -2.40548262]
 [-0.0298448   0.00720208 -0.0374721  -2.0634368 ]
 [ 0.04731849 -0.0225497  -0.0118263  -0.40654566]]
4.0
[ 0.06319762  0.0319443  -0.01860317 -1.87546508]
[array([-0.06319762, -0.0319443 ,  0.01860317, -0.12453492]), array([ 0.04572393,  0.04729191,  0.03069523, -2.40548262]), array([-0.0298448 ,  0.00720208, -0.0374721 , -2.0634368 ]), array([ 0.04731849, -0.0225497 , -0.0118263 , -0.40654566])]


In [45]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!4D points AUTOMATED GRAPH INERT ZONES & SIMPLEXES!!!!!!!!!!!!!!!!!
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
#inertPointList  = getInertPointsFromExcel("Inert10pts4D001-3-inertPoints.xlsx")
#if inertPointList:
#    for i in range(len(inertPointList[0][0])):
#        xofInertPoints.append(list())

    #for i in range(len(inertPointList)):
    #    yofInertPoints.append(inertPointList[i][1][0])
    #    inertPointLocation=beta0ofL1line + beta1ofL1line*inertPointList[i][0][0][0] + beta2ofL1line*inertPointList[i][0][1][0]+ beta3ofL1line*inertPointList[i][0][2][0]
    #    if inertPointLocation > inertPointList[i][1][0]:
    #        inertPointColorList.append('purple')
    #    else:
    #        inertPointColorList.append('lightgreen')
    #        
    #    for j in range(len(inertPointList[0][0])):
    #        xofInertPoints[j].append(inertPointList[i][0][j][0])        
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=xofPoints[2], mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=6),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesBelow = returnInequalitiesCoefficients[1]
#ABOVE
try:
    
    #halfspacesAbove = 1
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    
    vertexConnectionIndicesAbove = []
    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1

    #This will be obtained with hs.intersestions
    vertexesAbove=hsAbove.intersections
    
    vertexTextListAbove=[]

    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')

#----Generate plots   
    for i in range(len(vertexesAbove)):
        fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
                 marker=dict(color='lightgreen', size=5), showlegend=False))

        
    for faceVertexesAbove in vertexConnectionIndicesAbove:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#-------------------------ABOVE SET 1---------------------------
try:
    #halfspacesAboveSet1 = 1
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    
    vertexConnectionIndicesAboveSet1 = []

    counter=0
    for item in hullPyAboveSet1:
        if counter>0:
            vertexConnectionIndicesAboveSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
        counter+=1
        
    #This will be obtained with hs.intersestions
    vertexesAboveSet1=hsAboveSet1.intersections
    
    vertexTextListAboveSet1=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet1.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[i][0]],y=[vertexesAboveSet1[i][1]], z=[vertexesAboveSet1[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
        
        

    for faceVertexesAbove in vertexConnectionIndicesAboveSet1:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[i]][0],vertexesAboveSet1[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet1[faceVertexesAbove[i]][1],vertexesAboveSet1[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet1[faceVertexesAbove[i]][2],vertexesAboveSet1[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet1[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet1[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet1[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
                
except Exception as e:
    print('Problem in obtaining the inert zone above set 1.')
    print(e)
#-------------------------ABOVE SET 2---------------------------
try:
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)

    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    #Should give the indexes of the vertexes incident to each facet

    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    

    vertexConnectionIndicesAboveSet2 = []

    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions

    vertexesAboveSet2=hsAboveSet2.intersections
    

    vertexTextListAboveSet2=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet2.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[i][0]],y=[vertexesAboveSet2[i][1]], z=[vertexesAboveSet2[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
                 
    for faceVertexesAbove in vertexConnectionIndicesAboveSet2:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[i]][0],vertexesAboveSet2[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet2[faceVertexesAbove[i]][1],vertexesAboveSet2[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet2[faceVertexesAbove[i]][2],vertexesAboveSet2[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet2[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet2[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet2[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above set 2.')
    print(e)
#BELOW
try:
    #halfspacesBelow = 1
    halfspacesBelow = returnInequalitiesCoefficients[1]
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    hullPyBelow = qconvexPy("i",hsBelow.intersections)

    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1

    vertexesBelow=hsBelow.intersections

    vertexTextListBelow=[]

    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    

#Plotting    
    for i in range(len(vertexesBelow)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
                 marker=dict(color='purple', size=5), showlegend=False))

    #print(15)
    for faceVertexesBelow in vertexConnectionIndicesBelow:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))
except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
#-------------------------BELOW SET 1---------------------------
try:
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])

    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
    hsBelowSet1= HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelowSet1)
    
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)

    

    vertexConnectionIndicesBelowSet1 = []
    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1

    vertexesBelowSet1=hsBelowSet1.intersections
    print('vertexesBelowSet1: ', vertexesBelowSet1)
    vertexTextListBelowSet1=[]

    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelowSet1.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')    

#Plotting    


    for i in range(len(vertexesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[i][0]],y=[vertexesBelowSet1[i][1]], z=[vertexesBelowSet1[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))


    #print(16)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet1:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[i]][0],vertexesBelowSet1[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet1[faceVertexesBelow[i]][1],vertexesBelowSet1[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet1[faceVertexesBelow[i]][2],vertexesBelowSet1[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet1[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet1[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet1[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))    
except Exception as e:
    print('Problem in obtaining the inert zone below set 1.')
    print(e)
#-------------------------BELOW SET 2---------------------------
try:

    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    feasible_pointBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet2)
    hsBelowSet2= HalfspaceIntersection(halfspacesBelowSet2, feasible_pointBelowSet2)
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    #print(3)
    
    vertexConnectionIndicesBelowSet2 = []
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1

    vertexesBelowSet2=hsBelowSet2.intersections
    
    vertexTextListBelowSet2=[]

    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelowSet2.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')    
#Plotting    

    for i in range(len(vertexesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[i][0]],y=[vertexesBelowSet2[i][1]], z=[vertexesBelowSet2[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))

    for faceVertexesBelow in vertexConnectionIndicesBelowSet2:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[i]][0],vertexesBelowSet2[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet2[faceVertexesBelow[i]][1],vertexesBelowSet2[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet2[faceVertexesBelow[i]][2],vertexesBelowSet2[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet2[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet2[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet2[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
except Exception as e:
    print('Problem in obtaining the inert zone below set 2.')
    print(e)
#--------
zMinLimit=min(np.min(xofPoints[2]),np.min(vertexesAbove[2]), np.min(vertexesBelow[2]),\
              np.min(vertexesAboveSet1[2]), np.min(vertexesAboveSet2[2]),\
             np.min(vertexesBelowSet1[2]), np.min(vertexesBelowSet2[2]))
zMaxLimit=max(np.max(xofPoints[2]),np.max(vertexesAbove[2]), np.max(vertexesBelow[2]),\
              np.max(vertexesAboveSet1[2]), np.max(vertexesAboveSet2[2]),\
             np.max(vertexesBelowSet1[2]), np.max(vertexesBelowSet2[2]))
#--------
yMinLimit=min(np.min(xofPoints[1]),np.min(vertexesAbove[1]), np.min(vertexesBelow[1]),\
              np.min(vertexesAboveSet1[1]), np.min(vertexesAboveSet2[1]),\
             np.min(vertexesBelowSet1[1]), np.min(vertexesBelowSet2[1]))
yMaxLimit=max(np.max(xofPoints[1]),np.max(vertexesAbove[1]), np.max(vertexesBelow[1]),\
              np.max(vertexesAboveSet1[1]), np.max(vertexesAboveSet2[1]),\
             np.max(vertexesBelowSet1[1]), np.max(vertexesBelowSet2[1]))
#--------

xMinLimit=min(np.min(xofPoints[0]),np.min(vertexesAbove[0]), np.min(vertexesBelow[0]),\
              np.min(vertexesAboveSet1[0]), np.min(vertexesAboveSet2[0]),\
             np.min(vertexesBelowSet1[0]), np.min(vertexesBelowSet2[0]))
xMaxLimit=max(np.max(xofPoints[0]),np.max(vertexesAbove[0]), np.max(vertexesBelow[0]),\
              np.max(vertexesAboveSet1[0]), np.max(vertexesAboveSet2[0]),\
             np.max(vertexesBelowSet1[0]), np.max(vertexesBelowSet2[0]))
ultimateMinLimit=min(np.min(xofPoints),np.min(vertexesAbove), np.min(vertexesBelow),\
              np.min(vertexesAboveSet1), np.min(vertexesAboveSet2),\
             np.min(vertexesBelowSet1), np.min(vertexesBelowSet2))
ultimateMaxLimit=max(np.max(xofPoints),np.max(vertexesAbove), np.max(vertexesBelow),\
              np.max(vertexesAboveSet1), np.max(vertexesAboveSet2),\
             np.max(vertexesBelowSet1), np.max(vertexesBelowSet2))


axisLimits=800
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(ultimateMinLimit,ultimateMaxLimit),
                    yaxis_range=(ultimateMinLimit,ultimateMaxLimit),
                    xaxis_range=(ultimateMinLimit,ultimateMaxLimit),
                )
)

#Manually set
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(-7,13),
                    yaxis_range=(-23,33),
                    xaxis_range=(-72,140),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

vertexesBelowSet1:  [[-4.00000000e+00 -7.00000000e+00  6.70405345e+00]
 [ 2.60000000e+01  8.00000000e+00 -5.56793740e-02]
 [-1.90000000e+01 -7.00000000e+00  7.79360792e-03]
 [ 7.10000000e+01 -7.00000000e+00  3.10732840e-01]]


In [212]:
#Babayaga
#!!!!!!!!!!!!!!!!!!!!!!!!!!!4D points AUTOMATED GRAPH - JUST SIMPLEXES ABOVE and THEIR BARYCENTER!!!!!!!!!!!!!!!!!
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
  
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=[baryCenterOfSimplexesAboveSet2[0]],y=[baryCenterOfSimplexesAboveSet2[1]], z=[baryCenterOfSimplexesAboveSet2[2]], mode='markers+text',\
             marker=dict(color='green',opacity=1, size=4),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesBelow = returnInequalitiesCoefficients[1]
#ABOVE
try:
    
    #halfspacesAbove = 1
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    
    vertexConnectionIndicesAbove = []
    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1

    #This will be obtained with hs.intersestions
    vertexesAbove=hsAbove.intersections
    
    vertexTextListAbove=[]

    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')

#----Generate plots   
    #for i in range(len(vertexesAbove)):
    #    fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
    #             marker=dict(color='lightgreen', size=5), showlegend=False))

        
    #for faceVertexesAbove in vertexConnectionIndicesAbove:
    #    for i in range(len(faceVertexesAbove)-1):
    #        fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
    #                                   y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
    #                                   z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
    #                                   mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
    #        if i == len(faceVertexesAbove)-2:
    #            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
    #                                       y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
    #                                       z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
    #                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#-------------------------ABOVE SET 1---------------------------
try:
    #halfspacesAboveSet1 = 1
    halfspacesAboveSet1 = np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    
    vertexConnectionIndicesAboveSet1 = []

    counter=0
    for item in hullPyAboveSet1:
        if counter>0:
            vertexConnectionIndicesAboveSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
        counter+=1
        
    #This will be obtained with hs.intersestions
    vertexesAboveSet1=hsAboveSet1.intersections
    
    vertexTextListAboveSet1=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet1.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')

#----Generate plots   

    #for i in range(len(vertexesAboveSet1)):
    #    fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[i][0]],y=[vertexesAboveSet1[i][1]], z=[vertexesAboveSet1[i][2]], mode='markers+text',\
    #             marker=dict(color='red', size=3), showlegend=False,surfaceaxis=1,surfacecolor="red"))
    fig.add_trace(go.Surface(x=vertexesAboveSet1[:,0],y=vertexesAboveSet1[:,1], z=vertexesAboveSet1[:,2],\
             showlegend=False,colorscale='Reds'))
        
        

    for faceVertexesAbove in vertexConnectionIndicesAboveSet1:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[i]][0],vertexesAboveSet1[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet1[faceVertexesAbove[i]][1],vertexesAboveSet1[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet1[faceVertexesAbove[i]][2],vertexesAboveSet1[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='red', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet1[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet1[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet1[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='red', size=3),showlegend=False)) 
                
except Exception as e:
    print('Problem in obtaining the inert zone above set 1.')
    print(e)
#-------------------------ABOVE SET 2---------------------------
try:
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)

    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    #Should give the indexes of the vertexes incident to each facet

    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    

    vertexConnectionIndicesAboveSet2 = []

    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions

    vertexesAboveSet2=hsAboveSet2.intersections
    

    vertexTextListAboveSet2=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet2.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[i][0]],y=[vertexesAboveSet2[i][1]], z=[vertexesAboveSet2[i][2]], mode='markers+text',\
                 marker=dict(color='blue', size=3), showlegend=False))
                 
    for faceVertexesAbove in vertexConnectionIndicesAboveSet2:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[i]][0],vertexesAboveSet2[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet2[faceVertexesAbove[i]][1],vertexesAboveSet2[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet2[faceVertexesAbove[i]][2],vertexesAboveSet2[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='blue', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet2[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet2[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet2[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='blue', size=3),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above set 2.')
    print(e)
    
    
#--------------------------
#--------
zMinLimit=min(np.min(vertexesAbove[:,2]), np.min(vertexesAboveSet1[:,2]), np.min(vertexesAboveSet2[:,2]))
zMaxLimit=max(np.max(vertexesAbove[:,2]), np.max(vertexesAboveSet1[:,2]), np.max(vertexesAboveSet2[:,2]))
#--------
yMinLimit=min(np.min(vertexesAbove[:,1]), np.min(vertexesAboveSet1[:,1]), np.min(vertexesAboveSet2[:,1]))
yMaxLimit=max(np.max(vertexesAbove[:,1]), np.max(vertexesAboveSet1[:,1]), np.max(vertexesAboveSet2[:,1]))
#--------

xMinLimit=min(np.min(vertexesAbove[:,0]), np.min(vertexesAboveSet1[:,0]), np.min(vertexesAboveSet2[:,0]))
xMaxLimit=max(np.max(vertexesAbove[:,0]), np.max(vertexesAboveSet1[:,0]), np.max(vertexesAboveSet2[:,0]))
#---------


axisLimits=800
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(zMinLimit,zMaxLimit),
                    yaxis_range=(yMinLimit,yMaxLimit),
                    xaxis_range=(xMinLimit,xMaxLimit),
                )
)

#Manually set
#fig.update_layout(
#    scene = dict(   xaxis_title='x'+u"\u2081",
#                    yaxis_title='x'+u"\u2082",
#                    zaxis_title='x'+u"\u2083",
#                    zaxis_range=(-14,23),
#                    yaxis_range=(-30,45),
#                    xaxis_range=(-90,170),
#                )
#)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

In [191]:
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
z = z_data.values
print(z.shape)
sh_0, sh_1 = z.shape

(25, 25)


In [209]:
print(vertexesAboveSet1[:,0])
print(vertexesAboveSet1[:,1])
print(vertexesAboveSet1[:,2])

[24. 14. -1. 29.]
[7. 2. 7. 7.]
[-0.85405345  1.39919082  1.27705342  1.37803316]


In [196]:
print(y)
print(x)
print(z)

[ 0.    1.25  2.5   3.75  5.    6.25  7.5   8.75 10.   11.25 12.5  13.75
 15.   16.25 17.5  18.75 20.   21.25 22.5  23.75 25.   26.25 27.5  28.75
 30.  ]
[ 0.    1.25  2.5   3.75  5.    6.25  7.5   8.75 10.   11.25 12.5  13.75
 15.   16.25 17.5  18.75 20.   21.25 22.5  23.75 25.   26.25 27.5  28.75
 30.  ]
[[ 0.000000e+00  2.780985e+01  4.961936e+01  8.308067e+01  1.166632e+02
   1.304140e+02  1.507206e+02  2.201871e+02  1.561536e+02  1.486416e+02
   2.037845e+02  2.060386e+02  1.071618e+02  6.836975e+01  4.533590e+01
   4.996142e+01  2.189279e+01  1.702552e+01  1.174317e+01  1.475226e+01
   1.366710e+01  5.677561e+00  3.312340e+00  1.156517e+00 -1.476620e-01]
 [ 1.000000e+00  2.771966e+01  4.855022e+01  6.521374e+01  9.527666e+01
   1.169964e+02  1.339056e+02  1.523412e+02  1.519340e+02  1.601139e+02
   1.795327e+02  1.476184e+02  1.703943e+02  1.218194e+02  5.258537e+01
   3.308871e+01  3.840972e+01  4.424843e+01  6.957860e+01  4.019351e+00
   3.050024e+00  3.039719e+00  2.996142e+00

In [214]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
z = vertexesAboveSet1[:,2]
sh_0 = z.shape
sh_1 = z.shape
x, y = np.linspace(np.min(vertexesAboveSet1[:,0]), np.max(vertexesAboveSet1[:,0]), sh_0), np.linspace(np.min(vertexesAboveSet1[:,1]), np.max(vertexesAboveSet1[:,1]), sh_1)
fig = go.Figure(data=[go.Surface(x=x,y=y, z=z)])
fig.update_layout(title='Mt Bruno Elevation', autosize=True,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

TypeError: 'tuple' object cannot be interpreted as an integer

In [149]:
#Babayaga
#!!!!!!!!!!!!!!!!!!!!!!!!!!!4D points AUTOMATED GRAPH - JUST SIMPLEXES BELOW and THEIR BARYCENTER!!!!!!!!!!!!!!!!!
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
  
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=[baryCenterOfSimplexesBelowSet1[0]],y=[baryCenterOfSimplexesBelowSet1[1]], z=[baryCenterOfSimplexesBelowSet1[2]], mode='markers+text',\
             marker=dict(color='green',opacity=1, size=6),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
#go.Scatter3d(x=[14.5],y=[0.75], z=[1.932534336], mode='markers+text',\
#    marker=dict(color="blue", size=3,opacity=0.3), showlegend=False)
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesBelow = returnInequalitiesCoefficients[1]

#--------------
'''
#ABOVE
try:
    
    #halfspacesAbove = 1
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    
    vertexConnectionIndicesAbove = []
    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1

    #This will be obtained with hs.intersestions
    vertexesAbove=hsAbove.intersections
    
    vertexTextListAbove=[]

    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')

#----Generate plots   
    for i in range(len(vertexesAbove)):
        fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
                 marker=dict(color='lightgreen', size=5), showlegend=False))

        
    for faceVertexesAbove in vertexConnectionIndicesAbove:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#-------------------------ABOVE SET 1---------------------------
try:
    #halfspacesAboveSet1 = 1
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    
    vertexConnectionIndicesAboveSet1 = []

    counter=0
    for item in hullPyAboveSet1:
        if counter>0:
            vertexConnectionIndicesAboveSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
        counter+=1
        
    #This will be obtained with hs.intersestions
    vertexesAboveSet1=hsAboveSet1.intersections
    
    vertexTextListAboveSet1=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet1.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[i][0]],y=[vertexesAboveSet1[i][1]], z=[vertexesAboveSet1[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
        
        

    for faceVertexesAbove in vertexConnectionIndicesAboveSet1:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[i]][0],vertexesAboveSet1[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet1[faceVertexesAbove[i]][1],vertexesAboveSet1[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet1[faceVertexesAbove[i]][2],vertexesAboveSet1[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet1[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet1[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet1[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
                
except Exception as e:
    print('Problem in obtaining the inert zone above set 1.')
    print(e)
#-------------------------ABOVE SET 2---------------------------
try:
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)

    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    #Should give the indexes of the vertexes incident to each facet

    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    

    vertexConnectionIndicesAboveSet2 = []

    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions

    vertexesAboveSet2=hsAboveSet2.intersections
    

    vertexTextListAboveSet2=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet2.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[i][0]],y=[vertexesAboveSet2[i][1]], z=[vertexesAboveSet2[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
                 
    for faceVertexesAbove in vertexConnectionIndicesAboveSet2:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[i]][0],vertexesAboveSet2[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet2[faceVertexesAbove[i]][1],vertexesAboveSet2[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet2[faceVertexesAbove[i]][2],vertexesAboveSet2[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet2[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet2[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet2[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above set 2.')
    print(e)
    
    
#--------------------------
'''
#BELOW
try:
    #halfspacesBelow = 1
    halfspacesBelow = returnInequalitiesCoefficients[1]
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    hullPyBelow = qconvexPy("i",hsBelow.intersections)

    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1

    vertexesBelow=hsBelow.intersections

    vertexTextListBelow=[]

    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    

#Plotting

#    for i in range(len(vertexesBelow)):
#        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
#                 marker=dict(color='purple', size=5), showlegend=False))

    #print(15)
#    for faceVertexesBelow in vertexConnectionIndicesBelow:
#        for i in range(len(faceVertexesBelow)-1):
#            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
#                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
#                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
#                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
#            if i == len(faceVertexesBelow)-2:
#                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
#                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
#                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
#                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))

except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
#-------------------------BELOW SET 1---------------------------
try:
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])

    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
    hsBelowSet1= HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelowSet1)
    
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)

    

    vertexConnectionIndicesBelowSet1 = []
    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1

    vertexesBelowSet1=hsBelowSet1.intersections
    print('vertexesBelowSet1: ', vertexesBelowSet1)
    vertexTextListBelowSet1=[]

    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelowSet1.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')    

#Plotting    


    for i in range(len(vertexesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[i][0]],y=[vertexesBelowSet1[i][1]], z=[vertexesBelowSet1[i][2]], mode='markers+text',\
                 marker=dict(color='red', size=3), showlegend=False))


    #print(16)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet1:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[i]][0],vertexesBelowSet1[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet1[faceVertexesBelow[i]][1],vertexesBelowSet1[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet1[faceVertexesBelow[i]][2],vertexesBelowSet1[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='red', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet1[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet1[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet1[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='red', size=3),showlegend=False))    
except Exception as e:
    print('Problem in obtaining the inert zone below set 1.')
    print(e)
#-------------------------BELOW SET 2---------------------------
try:

    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    feasible_pointBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet2)
    hsBelowSet2= HalfspaceIntersection(halfspacesBelowSet2, feasible_pointBelowSet2)
    print('hsBelowSet2: ', )
    print('hsBelowSet2.intersections: ',hsBelowSet2.intersections )
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    print('hullPyBelowSet2: ', hullPyBelowSet2)
    #print(3)
    
    vertexConnectionIndicesBelowSet2 = []
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1

    vertexesBelowSet2=hsBelowSet2.intersections
    
    vertexTextListBelowSet2=[]

    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelowSet2.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')    
#Plotting    

    for i in range(len(vertexesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[i][0]],y=[vertexesBelowSet2[i][1]], z=[vertexesBelowSet2[i][2]], mode='markers+text',\
                 marker=dict(color='blue', size=3), showlegend=False))

    for faceVertexesBelow in vertexConnectionIndicesBelowSet2:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[i]][0],vertexesBelowSet2[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet2[faceVertexesBelow[i]][1],vertexesBelowSet2[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet2[faceVertexesBelow[i]][2],vertexesBelowSet2[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='blue', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet2[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet2[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet2[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='blue', size=3),showlegend=False))
except Exception as e:
    print('Problem in obtaining the inert zone below set 2.')
    print(e)
    

#--------------------------
#--------
zMinLimit=min(np.min(xofPoints[2]),np.min(vertexesAbove[2]), np.min(vertexesBelow[2]),\
              np.min(vertexesAboveSet1[2]), np.min(vertexesAboveSet2[2]),\
             np.min(vertexesBelowSet1[2]), np.min(vertexesBelowSet2[2]))
zMaxLimit=max(np.max(xofPoints[2]),np.max(vertexesAbove[2]), np.max(vertexesBelow[2]),\
              np.max(vertexesAboveSet1[2]), np.max(vertexesAboveSet2[2]),\
             np.max(vertexesBelowSet1[2]), np.max(vertexesBelowSet2[2]))
#--------
yMinLimit=min(np.min(xofPoints[1]),np.min(vertexesAbove[1]), np.min(vertexesBelow[1]),\
              np.min(vertexesAboveSet1[1]), np.min(vertexesAboveSet2[1]),\
             np.min(vertexesBelowSet1[1]), np.min(vertexesBelowSet2[1]))
yMaxLimit=max(np.max(xofPoints[1]),np.max(vertexesAbove[1]), np.max(vertexesBelow[1]),\
              np.max(vertexesAboveSet1[1]), np.max(vertexesAboveSet2[1]),\
             np.max(vertexesBelowSet1[1]), np.max(vertexesBelowSet2[1]))
#--------

xMinLimit=min(np.min(xofPoints[0]),np.min(vertexesAbove[0]), np.min(vertexesBelow[0]),\
              np.min(vertexesAboveSet1[0]), np.min(vertexesAboveSet2[0]),\
             np.min(vertexesBelowSet1[0]), np.min(vertexesBelowSet2[0]))
xMaxLimit=max(np.max(xofPoints[0]),np.max(vertexesAbove[0]), np.max(vertexesBelow[0]),\
              np.max(vertexesAboveSet1[0]), np.max(vertexesAboveSet2[0]),\
             np.max(vertexesBelowSet1[0]), np.max(vertexesBelowSet2[0]))


#--------------------------
#--------
zMinLimit=min(np.min(vertexesBelow[:,2]), np.min(vertexesBelowSet1[:,2]), np.min(vertexesBelowSet2[:,2]))
zMaxLimit=max(np.max(vertexesBelow[:,2]), np.max(vertexesBelowSet1[:,2]), np.max(vertexesBelowSet2[:,2]))
#--------
yMinLimit=min(np.min(vertexesBelow[:,1]), np.min(vertexesBelowSet1[:,1]), np.min(vertexesBelowSet2[:,1]))
yMaxLimit=max(np.max(vertexesBelow[:,1]), np.max(vertexesBelowSet1[:,1]), np.max(vertexesBelowSet2[:,1]))
#--------

xMinLimit=min(np.min(vertexesBelow[:,0]), np.min(vertexesBelowSet1[:,0]), np.min(vertexesBelowSet2[:,0]))
xMaxLimit=max(np.max(vertexesBelow[:,0]), np.max(vertexesBelowSet1[:,0]), np.max(vertexesBelowSet2[:,0]))
#---------


axisLimits=800
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(zMinLimit,zMaxLimit),
                    yaxis_range=(yMinLimit,yMaxLimit),
                    xaxis_range=(xMinLimit,xMaxLimit),
                )
)

#Manually set
#fig.update_layout(
#    scene = dict(   xaxis_title='x'+u"\u2081",
#                    yaxis_title='x'+u"\u2082",
#                    zaxis_title='x'+u"\u2083",
#                    zaxis_range=(-14,23),
#                    yaxis_range=(-30,45),
#                    xaxis_range=(-90,170),
#                )
#)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

vertexesBelowSet1:  [[-12.5         -3.75         7.88732832]
 [ 27.5         16.25        -1.12564878]
 [-32.5         -3.75        -1.04101814]
 [ 87.5         -3.75        -0.63709916]]
hsBelowSet2: 
hsBelowSet2.intersections:  [[ 47.5          6.25        -5.3455472 ]
 [  7.5        -13.75         3.6674299 ]
 [-52.5          6.25         3.17888028]
 [ 67.5          6.25         3.58279926]]
hullPyBelowSet2:  ['4', '3 1 2', '0 3 2', '1 0 2', '0 1 3']


In [36]:
#4D points AUTOMATED GRAPH - JUST INERT ZONES
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
inertPointList  = getInertPointsFromExcel("Inert10pts4D001-3-inertPoints.xlsx")
if inertPointList:
    for i in range(len(inertPointList[0][0])):
        xofInertPoints.append(list())

    for i in range(len(inertPointList)):
        yofInertPoints.append(inertPointList[i][1][0])
        inertPointLocation=beta0ofL1line + beta1ofL1line*inertPointList[i][0][0][0] + beta2ofL1line*inertPointList[i][0][1][0]+ beta3ofL1line*inertPointList[i][0][2][0]
        if inertPointLocation > inertPointList[i][1][0]:
            inertPointColorList.append('purple')
        else:
            inertPointColorList.append('lightgreen')
            
        for j in range(len(inertPointList[0][0])):
            xofInertPoints[j].append(inertPointList[i][0][j][0])        
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=xofPoints[2], mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=3.5),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
#ABOVE
try:
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAbove =np.array([])
    for i in range(len(explanatoryTrainingOriginal)):
        feasible_pointAbove = np.append(feasible_pointAbove, xofPoints[i][feasible_pointAboveIndex])
    print('feasible_pointAbove: ', feasible_pointAbove)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    vertexConnectionIndicesAbove = []
    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions
    vertexesAbove=hsAbove.intersections
    vertexTextListAbove=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')
    for i in range(len(vertexesAbove)):
        fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
                 marker=dict(color='lightgreen', size=5), showlegend=False))


    for faceVertexesAbove in vertexConnectionIndicesAbove:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#BELOW
try:
    halfspacesBelow = returnInequalitiesCoefficients[1]
    #Write a function for that.
    feasible_pointBelowIndex = fittedPointForInertZone('below',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointBelow =np.array([])

    for i in range(len(explanatoryTrainingOriginal)):
        feasible_pointBelow = np.append(feasible_pointBelow, xofPoints[i][feasible_pointBelowIndex])
    print('feasible_pointBelow: ', feasible_pointBelow)
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)    
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    hullPyBelow = qconvexPy("i",hsBelow.intersections)

    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1

    vertexesBelow=hsBelow.intersections
    vertexTextListBelow=[]
    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    
    for i in range(len(vertexesBelow)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
                 marker=dict(color='purple', size=5), showlegend=False))
    for faceVertexesBelow in vertexConnectionIndicesBelow:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))  

except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#--------
axisLimits=75
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(-axisLimits,axisLimits),
                    yaxis_range=(-axisLimits,axisLimits),
                    xaxis_range=(-axisLimits,axisLimits),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

NameError: name 'getInertPointsFromExcel' is not defined

In [152]:
print(len(hsAbove.intersections))

12


In [199]:
print(returnInequalitiesCoefficients)

[[[-0.01123596  0.06741573 -0.34606742]
  [-0.16853933  0.01123596  1.40898876]
  [ 0.17977528 -0.07865169 -1.06292135]
  [ 0.01123596 -0.06741573 -1.65393258]
  [ 0.16853933 -0.01123596 -3.40898876]
  [-0.17977528  0.07865169 -0.93707865]]

 [[ 0.01123596 -0.06741573 -0.68314607]
  [ 0.16853933 -0.01123596 -1.64719101]
  [-0.17977528  0.07865169  0.33033708]
  [-0.01123596  0.06741573 -1.31685393]
  [-0.16853933  0.01123596 -0.35280899]
  [ 0.17977528 -0.07865169 -2.33033708]]]


In [37]:
#4D points AUTOMATED, Lines are drawn individually
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
supportRange=400
supportMultiplier=5
x1s = np.linspace(-supportRange, supportRange, 200)
x2s = np.linspace(-supportRange, supportRange, 200)
ys = np.linspace(-supportRange,supportRange,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
inertPointList  = getInertPointsFromExcel("Inert10pts4D001-3-inertPoints.xlsx")
if inertPointList:
    for i in range(len(inertPointList[0][0])):
        xofInertPoints.append(list())

    for i in range(len(inertPointList)):
        yofInertPoints.append(inertPointList[i][1][0])
        inertPointLocation=beta0ofL1line + beta1ofL1line*inertPointList[i][0][0][0] + beta2ofL1line*inertPointList[i][0][1][0]+ beta3ofL1line*inertPointList[i][0][2][0]
        if inertPointLocation > inertPointList[i][1][0]:
            inertPointColorList.append('purple')
        else:
            inertPointColorList.append('lightgreen')
            
        for j in range(len(inertPointList[0][0])):
            xofInertPoints[j].append(inertPointList[i][0][j][0])        
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 
#INERT ZONE CALCULATIONS
returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
    
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=xofPoints[2], mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=3.5),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
])
#XXXXX
zValue=0
counter=0
counterSet=0
for setOfInequalities in returnInequalitiesCoefficients:
    if counterSet==0:
        for inequality in setOfInequalities:
            #zValueLinspace=(-inequality[0]*x1s-inequality[1]*x2s-inequality[3])/inequality[2]
            zValueMashSpace=(-inequality[0]*x1meshx2-inequality[1]*x2meshx1-inequality[3])/inequality[2]
            #fig.add_trace(go.Scatter3d(x=x1s,y=x2s,z=zValue, mode='lines', marker=dict(color=f'rgb({10+10*counter},{12*counter},{6+10*counter})', size=5), showlegend=False))
            fig.add_trace(go.Surface(x=x1meshx2, y=x2meshx1, z=zValueMashSpace, colorscale='gray', showscale=False, opacity=0.3))
            counter+=1
    counterSet+=1
#XXXXX

#-------- 
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(-supportRange*supportMultiplier,supportRange*supportMultiplier),
                    yaxis_range=(-supportRange,supportRange),
                    xaxis_range=(-supportRange,supportRange),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

NameError: name 'getInertPointsFromExcel' is not defined

In [53]:
for inequality in returnInequalitiesCoefficients:
    print('-')
    print(inequality)
    print('-')

-
[[ 0.06319762  0.0319443  -0.01860317 -0.32771872]
 [-0.04572393 -0.04729191 -0.03069523  0.40426411]
 [ 0.0298448  -0.00720208  0.0374721  -0.04416658]
 [-0.04731849  0.0225497   0.0118263  -0.03237881]
 [-0.06319762 -0.0319443   0.01860317 -1.67228128]
 [ 0.04572393  0.04729191  0.03069523 -2.40426411]
 [-0.0298448   0.00720208 -0.0374721  -1.95583342]
 [ 0.04731849 -0.0225497  -0.0118263  -1.96762119]]
-
-
[[-0.06319762 -0.0319443   0.01860317  0.15436909]
 [ 0.04572393  0.04729191  0.03069523 -1.30029101]
 [-0.0298448   0.00720208 -0.0374721  -0.30094345]
 [ 0.04731849 -0.0225497  -0.0118263  -0.55313463]
 [ 0.06319762  0.0319443  -0.01860317 -2.15436909]
 [-0.04572393 -0.04729191 -0.03069523 -0.69970899]
 [ 0.0298448  -0.00720208  0.0374721  -1.69905655]
 [-0.04731849  0.0225497   0.0118263  -1.44686537]]
-


In [217]:
#Save inert points to an excel file, its name should be spesified in the code. In 3D.
import openpyxl
yofInertPoints=[]
x1ofInertPoints=[]
x2ofInertPoints=[]
for i in range(int(len(inertPointList))):
    yofInertPoints.append(inertPointList[i][1][0])
    x1ofInertPoints.append(inertPointList[i][0][0][0])
    x2ofInertPoints.append(inertPointList[i][0][1][0])
    data={'y':yofInertPoints,'x1':x1ofInertPoints,'x2':x2ofInertPoints}
dfDula = pd.DataFrame(data)
dfDula.to_excel(f'{dataSetName}-inertPoints-3.xlsx')

In [60]:
#3D points and L1 graph
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xs = np.linspace(-30, 30, 1000)

xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
colorList=[]
            
        
        
#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-19, 29, 1000)
x2s = np.linspace(-19, 29, 1000)
ys = np.linspace(-13,20,1000)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])

for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

    
L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1
fig = go.Figure(data=[
    ##### SAE graph
go.Surface(x=x1meshx2, y=x2meshx1, z=L1Hyperplane, colorscale='gray', showscale=False, opacity=0.7),
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=yofPoints, mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=4),\
             showlegend=False),
])
#-------- 
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='y',
                    zaxis_range=(-15,20),
                    yaxis_range=(-15,30),
                    xaxis_range=(-10,25),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
                  height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 2, 3, 5, 6, 7, 8, 9]


In [112]:
numberList=[k for k in range(numberOfPoints)]
print(numberList)
print(i)
numberList.pop(i)
print(numberList)

[0, 1, 2, 3, 4, 5, 6, 7]
7


In [168]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, 2, 3, 4], y=[0, 2, 3, 5], fill='tozeroy')) # fill down to xaxis
fig.add_trace(go.Scatter(x=[1, 2, 3, 4], y=[3, 5, 1, 7], fill='tonexty')) # fill to trace0 y

fig.show()

In [162]:
# Automated SAE graph on 2D data

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    
    
numberOfPoints=len(yofPoints)
beta1List = np.linspace(-0.2, 0.8, 300) #beta_1
beta0List = np.linspace(-3, 1.5, 300)   #beta_0
fBetaList = np.linspace(6, 20, 300)
#beta1List = np.linspace(-3, 13, 500)
#fBetaList = np.linspace(-2, 70, 500)
#beta0List = np.linspace(-20, 50, 500)

beta1MeshB1B0, beta0meshB1B0 = np.meshgrid(beta1List, beta0List)
placeHolder, fBMeshB1fB = np.meshgrid(beta1List, fBetaList)


lineExpressions=[]
xLineMashes=[]
for i in range(numberOfPoints):
    tempExpression=0
    tempExpression=-xofPoints[0][i]*beta1List+yofPoints[i]
    lineExpressions.append(tempExpression)
    
subSAEExpressions=calculateSubSAE(xofPoints, yofPoints, beta1MeshB1B0)    

    
SAE = 0
for i in range(numberOfPoints):
    SAE+=np.abs(beta0meshB1B0+xofPoints[0][i]*beta1MeshB1B0-yofPoints[i])

fig = go.Figure(data=[go.Surface(x=beta1MeshB1B0, y=beta0meshB1B0, z=SAE, colorscale='gray', showscale=False, opacity=0.8, cmin=0, cmax=20,
                      showlegend=False)])

##### Sections on the SAE graph ##### Lines on Beta 0 - Beta 1
for i in range(numberOfPoints):
    fig.add_trace(go.Scatter3d(x=beta1List, y=lineExpressions[i], z=subSAEExpressions[i].flatten(),\
                               mode='lines', line=dict(color='darkred', width=5),showlegend=False))
    fig.add_trace(go.Scatter3d(x=beta1List, y=lineExpressions[i], z=fBMeshB1fB.flatten(),\
                               mode='lines',line=dict(color='darkred', width=5),showlegend=False))
    
#color=f'rgb({200+i*20+np.random.uniform(-100,100,1)[0]},{200+i*20+np.random.uniform(-100,100,1)[0]},{200+i*20+np.random.uniform(-100,100,1)[0]})'

fig.update_layout(
    scene = dict(   xaxis_title=u"\u03B2"+u"\u2081",
                    yaxis_title=u"\u03B2"+u"\u2080",
                    zaxis_title='f('+u"\u03B2"+')',zaxis_range=(6,10)),
)
fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(225, 225, 225)",
                         gridcolor="rgb(192,192,192)",
                         showbackground=False,
                         zerolinecolor="black",),
                    yaxis = dict(
                        backgroundcolor="rgb(225, 225, 225)",
                        gridcolor="rgb(192,192,192)",
                        showbackground=False,
                        zerolinecolor="black"),
                    zaxis = dict(
                        backgroundcolor="rgb(225, 225, 225)",
                        gridcolor="rgb(192,192,192)",
                        showbackground=False,
                        zerolinecolor="rgb(192,192,192)",),),
                    width=1200,
                    margin=dict(
                    r=10, l=10,
                    b=10, t=10)
                  )

#fig.update_traces(contours_z=dict(
#    show=True, usecolormap=False,
#    highlightcolor="black",
#    project_z=True, width=16))

pio.write_html(fig, file='index.html', auto_open=True)

IndexError: list index out of range

In [94]:
np.random.uniform(-100,100,1)[0]

14.90827599790525

In [70]:
xofPoints[0]

[10, 3, 9, -5, 19, 14, 17, -2, 16, 6]

In [60]:
print(vertexesAboveSet2)

[[ 40.3 -64.  116.9]
 [ -7.7 -72.  -59.9]
 [-71.7 104.   24.9]
 [ 48.3  56.  -79.9]]


In [58]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!! WORK IN PROGRESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!4D points AUTOMATED GRAPH - defining inequality sets are drawn individually!!!!!!!!!!!!!!!!!
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
inertPointList  = getInertPointsFromExcel("Inert10pts4D001-3-inertPoints.xlsx")
if inertPointList:
    for i in range(len(inertPointList[0][0])):
        xofInertPoints.append(list())

    for i in range(len(inertPointList)):
        yofInertPoints.append(inertPointList[i][1][0])
        inertPointLocation=beta0ofL1line + beta1ofL1line*inertPointList[i][0][0][0] + beta2ofL1line*inertPointList[i][0][1][0]+ beta3ofL1line*inertPointList[i][0][2][0]
        if inertPointLocation > inertPointList[i][1][0]:
            inertPointColorList.append('purple')
        else:
            inertPointColorList.append('lightgreen')
            
        for j in range(len(inertPointList[0][0])):
            xofInertPoints[j].append(inertPointList[i][0][j][0])        
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=xofPoints[2], mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=3.5),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
go.Scatter3d(x=[2.3],y=[6], z=[0.5], mode='markers+text',\
    marker=dict(color='olive', size=8,opacity=1), showlegend=False),
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
#ABOVE
try:
    halfspacesAbove = returnInequalitiesCoefficients[0]
    #halfspacesAbove = 1
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    #feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    #feasible_pointAbove =np.array([])
    #for i in range(len(explanatoryTrainingOriginal)):
    #    feasible_pointAbove = np.append(feasible_pointAbove, xofPoints[i][feasible_pointAboveIndex])
    #print('feasible_pointAbove: ', feasible_pointAbove)
    #MANUAL FEASIBLE POINT CREATING
    #04by0013-01 "above" zone
    #feasible_pointAboveSet1List=[2.3,6,0.5]
    feasible_pointAboveSet2List=[0.0,0.0,0.0]
    #feasible_pointAboveSet1=np.array(feasible_pointAboveSet1List)
    feasible_pointAboveSet2=np.array(feasible_pointAboveSet2List)
    
    #hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    #hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    print(feasible_pointAboveSet2)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)
    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    print(1)
    #Should give the indexes of the vertexes incident to each facet
    #hullPyAbove = qconvexPy("i",hsAbove.intersections)
    #hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    print(2)
    #vertexConnectionIndicesAbove = []
    #vertexConnectionIndicesAboveSet1 = []
    vertexConnectionIndicesAboveSet2 = []
    #counter=0
    #for item in hullPyAbove:
    #    if counter>0:
    #        vertexConnectionIndicesAbove.append([])
    #        for subitem in item.split():
    #            if(subitem.isdigit()):
    #                vertexConnectionIndicesAbove[counter-1].append(int(subitem))
    #    counter+=1
    #counter=0
    #for item in hullPyAboveSet1:
    #    if counter>0:
    #        vertexConnectionIndicesAboveSet1.append([])
    #        for subitem in item.split():
    #            if(subitem.isdigit()):
    #                vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
    #    counter+=1
    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions
    #vertexesAbove=hsAbove.intersections
    #vertexesAboveSet1=hsAboveSet1.intersections
    vertexesAboveSet2=hsAboveSet2.intersections
    
    #vertexTextListAbove=[]
    #vertexTextListAboveSet1=[]
    vertexTextListAboveSet2=[]
    #for i in range(len(vertexesAbove[0])):
    #    vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')
    #for i in range(len(vertexesAboveSet1[0])):
    #    vertexTextListAboveSet1.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')
    for i in range(len(vertexesAboveSet2[0])):
        vertexTextListAboveSet2.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')

#----Generate plots   
    #for i in range(len(vertexesAbove)):
    #    fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
    #             marker=dict(color='lightgreen', size=5), showlegend=False))

    #for i in range(len(vertexesAboveSet1)):
    #    fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[i][0]],y=[vertexesAboveSet1[i][1]], z=[vertexesAboveSet1[i][2]], mode='markers+text',\
    #             marker=dict(color='lightgreen', size=3), showlegend=False))
        
    for i in range(len(vertexesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[i][0]],y=[vertexesAboveSet2[i][1]], z=[vertexesAboveSet2[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
        
    #for faceVertexesAbove in vertexConnectionIndicesAbove:
    #    for i in range(len(faceVertexesAbove)-1):
    #        fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
    #                                   y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
    #                                   z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
    #                                   mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
    #        if i == len(faceVertexesAbove)-2:
    #            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
    #                                       y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
    #                                       z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
    #                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
    #for faceVertexesAbove in vertexConnectionIndicesAboveSet1:
    #    for i in range(len(faceVertexesAbove)-1):
    #        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[i]][0],vertexesAboveSet1[faceVertexesAbove[i+1]][0]],\
    #                                   y=[vertexesAboveSet1[faceVertexesAbove[i]][1],vertexesAboveSet1[faceVertexesAbove[i+1]][1]],\
    #                                   z=[vertexesAboveSet1[faceVertexesAbove[i]][2],vertexesAboveSet1[faceVertexesAbove[i+1]][2]],\
    #                                   mode='lines',marker=dict(color='lightgreen', size=3),showlegend=False))
    #        if i == len(faceVertexesAbove)-2:
    #            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet1[faceVertexesAbove[0]][0]],\
    #                                       y=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet1[faceVertexesAbove[0]][1]],\
    #                                       z=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet1[faceVertexesAbove[0]][2]],\
    #                                       mode='lines',marker=dict(color='lightgreen', size=3),showlegend=False)) 
                
    for faceVertexesAbove in vertexConnectionIndicesAboveSet2:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[i]][0],vertexesAboveSet2[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet2[faceVertexesAbove[i]][1],vertexesAboveSet2[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet2[faceVertexesAbove[i]][2],vertexesAboveSet2[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet2[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet2[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet2[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#BELOW
try:
    # A wrong value is assigned to quit this process early on
    halfspacesBelow = 0
    # The correct value is below
    #halfspacesBelow = returnInequalitiesCoefficients[1]
    #print('type(halfspacesBelow) :', type(halfspacesBelow))
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
    #print('type(halfspacesBelowSet1) :', type(halfspacesBelowSet1))
    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    #print('type(halfspacesBelowSet2) :', type(halfspacesBelowSet2))
    #Write a function for that.
    feasible_pointBelowIndex = fittedPointForInertZone('below',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointBelow = np.array([])

    for i in range(len(explanatoryTrainingOriginal)):
        feasible_pointBelow = np.append(feasible_pointBelow, xofPoints[i][feasible_pointBelowIndex])
    print('feasible_pointBelow: ', feasible_pointBelow)
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
    feasible_pointBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet2)
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    #print('type(hsBelow) :', type(hsBelow))
    hsBelowSet1= HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelow)
    #print('type(hsBelowSet1) :', type(hsBelowSet1))
    hsBelowSet2= HalfspaceIntersection(halfspacesBelowSet2, feasible_pointBelow)
    #print('type(hsBelowSet2) :', type(hsBelowSet2))
    
    hullPyBelow = qconvexPy("i",hsBelow.intersections)
    #print(1)
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)
    #print(2)
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    #print(3)
    
    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1
    #print(4)
    vertexConnectionIndicesBelowSet1 = []
    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1
    #print(5)
    vertexConnectionIndicesBelowSet2 = []
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1
    #print(6)
    vertexesBelow=hsBelow.intersections
    #print(7)
    vertexesBelowSet1=hsBelowSet1.intersections
    #print(8)
    vertexesBelowSet2=hsBelowSet2.intersections
    #print(9)
    vertexTextListBelow=[]
    vertexTextListBelowSet1=[]
    vertexTextListBelowSet2=[]
    #for i in range(len(vertexesBelow[0])):
    #    vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    
    #print(10)
    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelowSet1.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')    
    #print(11)
    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelowSet2.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')    
    #print(12)
#Plotting    
    #for i in range(len(vertexesBelow)):
    #    fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
    #             marker=dict(color='purple', size=5), showlegend=False))
    #print(13)
    for i in range(len(vertexesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[i][0]],y=[vertexesBelowSet1[i][1]], z=[vertexesBelowSet1[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(14)
    for i in range(len(vertexesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[i][0]],y=[vertexesBelowSet2[i][1]], z=[vertexesBelowSet2[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(15)
    #for faceVertexesBelow in vertexConnectionIndicesBelow:
    #    for i in range(len(faceVertexesBelow)-1):
    #        fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
    #                                   y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
    #                                   z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
    #                                   mode='lines', marker=dict(color='purple', size=5), showlegend=False))
    #        if i == len(faceVertexesBelow)-2:
    #            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
    #                                       y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
    #                                       z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
    #                                       mode='lines',marker=dict(color='purple', size=5),showlegend=False))
    #print(16)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet1:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[i]][0],vertexesBelowSet1[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet1[faceVertexesBelow[i]][1],vertexesBelowSet1[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet1[faceVertexesBelow[i]][2],vertexesBelowSet1[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet1[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet1[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet1[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(17)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet2:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[i]][0],vertexesBelowSet2[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet2[faceVertexesBelow[i]][1],vertexesBelowSet2[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet2[faceVertexesBelow[i]][2],vertexesBelowSet2[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet2[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet2[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet2[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(18)           
except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
#--------
axisLimits=150
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(-axisLimits,axisLimits),
                    yaxis_range=(-axisLimits,axisLimits),
                    xaxis_range=(-axisLimits,axisLimits),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

len(df.columns):  4
[0. 0. 0.]
1
2
Problem in obtaining the inert zone below.
object of type 'int' has no len()


In [48]:
returnInequalitiesCoefficients

array([[[ 0.06319762,  0.0319443 , -0.01860317, -0.32771872],
        [-0.04572393, -0.04729191, -0.03069523,  0.40426411],
        [ 0.0298448 , -0.00720208,  0.0374721 , -0.04416658],
        [-0.04731849,  0.0225497 ,  0.0118263 , -0.03237881],
        [-0.06319762, -0.0319443 ,  0.01860317, -1.67228128],
        [ 0.04572393,  0.04729191,  0.03069523, -2.40426411],
        [-0.0298448 ,  0.00720208, -0.0374721 , -1.95583342],
        [ 0.04731849, -0.0225497 , -0.0118263 , -1.96762119]],

       [[-0.06319762, -0.0319443 ,  0.01860317,  0.15436909],
        [ 0.04572393,  0.04729191,  0.03069523, -1.30029101],
        [-0.0298448 ,  0.00720208, -0.0374721 , -0.30094345],
        [ 0.04731849, -0.0225497 , -0.0118263 , -0.55313463],
        [ 0.06319762,  0.0319443 , -0.01860317, -2.15436909],
        [-0.04572393, -0.04729191, -0.03069523, -0.69970899],
        [ 0.0298448 , -0.00720208,  0.0374721 , -1.69905655],
        [-0.04731849,  0.0225497 ,  0.0118263 , -1.44686537]]])

In [45]:
halfspacesAboveSet2

array([[-0.06319762, -0.0319443 ,  0.01860317, -1.67228128],
       [ 0.04572393,  0.04729191,  0.03069523, -2.40426411],
       [-0.0298448 ,  0.00720208, -0.0374721 , -1.95583342],
       [ 0.04731849, -0.0225497 , -0.0118263 , -1.96762119]])

In [46]:
halfspacesAbove

array([[ 0.06319762,  0.0319443 , -0.01860317, -0.32771872],
       [-0.04572393, -0.04729191, -0.03069523,  0.40426411],
       [ 0.0298448 , -0.00720208,  0.0374721 , -0.04416658],
       [-0.04731849,  0.0225497 ,  0.0118263 , -0.03237881],
       [-0.06319762, -0.0319443 ,  0.01860317, -1.67228128],
       [ 0.04572393,  0.04729191,  0.03069523, -2.40426411],
       [-0.0298448 ,  0.00720208, -0.0374721 , -1.95583342],
       [ 0.04731849, -0.0225497 , -0.0118263 , -1.96762119]])

In [ ]:
# IMAGE 3/21/22 4:25 PM !!!!!!!!!!!!!!!!!!!!!!!!!!!4D points AUTOMATED GRAPH - defining inequality sets are drawn individually and the zones!!!!!!!!!!!!!!!!!
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

xofInertPoints=[]
yofInertPoints=[]
xofPoints=[]
yofPoints=[]
yofPointsOriginal=[]
beta0ofL1line=solutionOriginal['betaValues'][0]
beta1ofL1line=solutionOriginal['betaValues'][1]
beta2ofL1line=solutionOriginal['betaValues'][2]
beta3ofL1line=solutionOriginal['betaValues'][3]
colorList=[]


#Following defines an array of, for example, 300 values between (-10,20) for beta1List.
x1s = np.linspace(-70, 70, 200)
x2s = np.linspace(-70, 70, 200)
ys = np.linspace(-70,70,200)    

#Following defines a space by taking a list of numbers as coordinate values.
#Two dimensional object requires an array of numbers, but for three dimensional objects np.meshgrid should be defined.
x2meshx1, x1meshx2 = np.meshgrid(x2s, x1s) 
x2meshy, ymeshx2 = np.meshgrid(x2s, ys)
x1meshy, ymeshx1 = np.meshgrid(x1s, ys)

L1Hyperplane = beta0ofL1line + beta1ofL1line*x1meshx2 + beta2ofL1line*x2meshx1

inertPointColorList=[]

for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
inertPointList  = getInertPointsFromExcel("Inert10pts4D001-3-inertPoints.xlsx")
if inertPointList:
    for i in range(len(inertPointList[0][0])):
        xofInertPoints.append(list())

    for i in range(len(inertPointList)):
        yofInertPoints.append(inertPointList[i][1][0])
        inertPointLocation=beta0ofL1line + beta1ofL1line*inertPointList[i][0][0][0] + beta2ofL1line*inertPointList[i][0][1][0]+ beta3ofL1line*inertPointList[i][0][2][0]
        if inertPointLocation > inertPointList[i][1][0]:
            inertPointColorList.append('purple')
        else:
            inertPointColorList.append('lightgreen')
            
        for j in range(len(inertPointList[0][0])):
            xofInertPoints[j].append(inertPointList[i][0][j][0])        
        
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=(beta0ofL1line + beta1ofL1line*explanatoryTrainingOriginal[0][i] + beta2ofL1line*explanatoryTrainingOriginal[1][i]+ beta3ofL1line*explanatoryTrainingOriginal[2][i])
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue') 

xofPoints=[]
yofPoints=[]
for i in range(len(explanatoryTrainingOriginal)):
    xofPoints.append(list())
    for j in range(len(explanatoryTrainingOriginal[i])):
        xofPoints[i].append(explanatoryTrainingOriginal[i][j])
for i in range(len(responseTrainingOriginal)):
    yofPoints.append(responseTrainingOriginal[i])
    locOfPoint=0
    locOfPoint+=solutionOriginal['betaValues'][0]
    for j in range(len(explanatoryTrainingOriginal)-1):
        locOfPoint+=solutionOriginal['betaValues'][j+1]*explanatoryTrainingOriginal[j][i]
    if np.round(locOfPoint,3)> responseTrainingOriginal[i]:
        colorList.append('magenta')    
    elif np.round(locOfPoint,3)<responseTrainingOriginal[i]:
        colorList.append('green') 
    else:
        colorList.append('blue')
        
        
fig = go.Figure(data=[   
    #points on the hyperplane
go.Scatter3d(x=xofPoints[0],y=xofPoints[1], z=xofPoints[2], mode='markers+text',\
             marker=dict(color=colorList,opacity=1, size=3.5),\
             showlegend=False),
#go.Scatter3d(x=xofInertPoints[0],y=xofInertPoints[1], z=xofInertPoints[2], mode='markers+text',\
#    marker=dict(color=inertPointColorList, size=1.2,opacity=0.3), showlegend=False),
])


returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients(dualVariableValuesOriginal,indexesOfFittedPointsOriginal,positionOfObservationsOriginal,explanatoryTrainingOriginal)
#INERT ZONE CALCULATIONS
halfspacesAbove = returnInequalitiesCoefficients[0]
halfspacesBelow = returnInequalitiesCoefficients[1]
#ABOVE
try:
    
    #halfspacesAbove = 1
    halfspacesAbove = returnInequalitiesCoefficients[0]
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAbove = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAbove)
    hsAbove = HalfspaceIntersection(halfspacesAbove, feasible_pointAbove)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAbove = qconvexPy("i",hsAbove.intersections)
    
    vertexConnectionIndicesAbove = []
    counter=0
    for item in hullPyAbove:
        if counter>0:
            vertexConnectionIndicesAbove.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAbove[counter-1].append(int(subitem))
        counter+=1

    #This will be obtained with hs.intersestions
    vertexesAbove=hsAbove.intersections
    
    vertexTextListAbove=[]

    for i in range(len(vertexesAbove[0])):
        vertexTextListAbove.append(f'{np.round(vertexesAbove[0][i],1)},{np.round(vertexesAbove[1][i],1)},{np.round(vertexesAbove[2][i],1)}')

#----Generate plots   
    for i in range(len(vertexesAbove)):
        fig.add_trace(go.Scatter3d(x=[vertexesAbove[i][0]],y=[vertexesAbove[i][1]], z=[vertexesAbove[i][2]], mode='markers+text',\
                 marker=dict(color='lightgreen', size=5), showlegend=False))

        
    for faceVertexesAbove in vertexConnectionIndicesAbove:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[i]][0],vertexesAbove[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAbove[faceVertexesAbove[i]][1],vertexesAbove[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAbove[faceVertexesAbove[i]][2],vertexesAbove[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAbove[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAbove[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAbove[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAbove[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='lightgreen', size=5),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above.')
    print(e)
#-------------------------ABOVE SET 1---------------------------
try:
    #halfspacesAboveSet1 = 1
    halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
    feasible_pointAboveIndex = fittedPointForInertZone('above',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet1)
    hsAboveSet1 = HalfspaceIntersection(halfspacesAboveSet1, feasible_pointAboveSet1)
    #Should give the indexes of the vertexes incident to each facet
    hullPyAboveSet1 = qconvexPy("i",hsAboveSet1.intersections)
    
    vertexConnectionIndicesAboveSet1 = []

    counter=0
    for item in hullPyAboveSet1:
        if counter>0:
            vertexConnectionIndicesAboveSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet1[counter-1].append(int(subitem))
        counter+=1
        
    #This will be obtained with hs.intersestions
    vertexesAboveSet1=hsAboveSet1.intersections
    
    vertexTextListAboveSet1=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet1.append(f'{np.round(vertexesAboveSet1[0][i],1)},{np.round(vertexesAboveSet1[1][i],1)},{np.round(vertexesAboveSet1[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[i][0]],y=[vertexesAboveSet1[i][1]], z=[vertexesAboveSet1[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
        
        

    for faceVertexesAbove in vertexConnectionIndicesAboveSet1:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[i]][0],vertexesAboveSet1[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet1[faceVertexesAbove[i]][1],vertexesAboveSet1[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet1[faceVertexesAbove[i]][2],vertexesAboveSet1[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet1[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet1[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet1[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet1[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
                
except Exception as e:
    print('Problem in obtaining the inert zone above set 1.')
    print(e)
#-------------------------ABOVE SET 2---------------------------
try:
    halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
    feasible_pointAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesAboveSet2)

    hsAboveSet2 = HalfspaceIntersection(halfspacesAboveSet2, feasible_pointAboveSet2)
    #Should give the indexes of the vertexes incident to each facet

    hullPyAboveSet2 = qconvexPy("i",hsAboveSet2.intersections)
    

    vertexConnectionIndicesAboveSet2 = []

    counter=0
    for item in hullPyAboveSet2:
        if counter>0:
            vertexConnectionIndicesAboveSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesAboveSet2[counter-1].append(int(subitem))
        counter+=1
    #This will be obtained with hs.intersestions

    vertexesAboveSet2=hsAboveSet2.intersections
    

    vertexTextListAboveSet2=[]
    for i in range(len(vertexesAbove[0])):
        vertexTextListAboveSet2.append(f'{np.round(vertexesAboveSet2[0][i],1)},{np.round(vertexesAboveSet2[1][i],1)},{np.round(vertexesAboveSet2[2][i],1)}')

#----Generate plots   

    for i in range(len(vertexesAboveSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[i][0]],y=[vertexesAboveSet2[i][1]], z=[vertexesAboveSet2[i][2]], mode='markers+text',\
                 marker=dict(color='darkgreen', size=3), showlegend=False))
                 
    for faceVertexesAbove in vertexConnectionIndicesAboveSet2:
        for i in range(len(faceVertexesAbove)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[i]][0],vertexesAboveSet2[faceVertexesAbove[i+1]][0]],\
                                       y=[vertexesAboveSet2[faceVertexesAbove[i]][1],vertexesAboveSet2[faceVertexesAbove[i+1]][1]],\
                                       z=[vertexesAboveSet2[faceVertexesAbove[i]][2],vertexesAboveSet2[faceVertexesAbove[i+1]][2]],\
                                       mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False))
            if i == len(faceVertexesAbove)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][0],vertexesAboveSet2[faceVertexesAbove[0]][0]],\
                                           y=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][1],vertexesAboveSet2[faceVertexesAbove[0]][1]],\
                                           z=[vertexesAboveSet2[faceVertexesAbove[len(faceVertexesAbove)-1]][2],vertexesAboveSet2[faceVertexesAbove[0]][2]],\
                                           mode='lines',marker=dict(color='darkgreen', size=3),showlegend=False)) 
except Exception as e:
    print('Problem in obtaining the inert zone above set 2.')
    print(e)
#BELOW
try:
    #halfspacesBelow = 1
    halfspacesBelow = returnInequalitiesCoefficients[1]
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    hullPyBelow = qconvexPy("i",hsBelow.intersections)

    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1

    vertexesBelow=hsBelow.intersections

    vertexTextListBelow=[]

    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    

#Plotting    
    for i in range(len(vertexesBelow)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
                 marker=dict(color='purple', size=5), showlegend=False))

    #print(15)
    for faceVertexesBelow in vertexConnectionIndicesBelow:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))
except Exception as e:
    print('Problem in obtaining the inert zone below.')
    print(e)
#-------------------------BELOW SET 1---------------------------
try:
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])

    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
    hsBelowSet1= HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelowSet1)
    
    hullPyBelow = qconvexPy("i",hsBelow.intersections)
    #print(1)
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)
    #print(2)
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    #print(3)
    
    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1
    #print(4)
    vertexConnectionIndicesBelowSet1 = []
    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1
    #print(5)
    vertexConnectionIndicesBelowSet2 = []
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1
    #print(6)
    vertexesBelow=hsBelow.intersections
    #print(7)
    vertexesBelowSet1=hsBelowSet1.intersections
    #print(8)
    vertexesBelowSet2=hsBelowSet2.intersections
    #print(9)
    vertexTextListBelow=[]
    vertexTextListBelowSet1=[]
    vertexTextListBelowSet2=[]
    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    
    #print(10)
    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelowSet1.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')    
    #print(11)
    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelowSet2.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')    
    #print(12)
#Plotting    
    for i in range(len(vertexesBelow)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
                 marker=dict(color='purple', size=5), showlegend=False))
    #print(13)
    for i in range(len(vertexesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[i][0]],y=[vertexesBelowSet1[i][1]], z=[vertexesBelowSet1[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(14)
    for i in range(len(vertexesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[i][0]],y=[vertexesBelowSet2[i][1]], z=[vertexesBelowSet2[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(15)
    for faceVertexesBelow in vertexConnectionIndicesBelow:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))
    #print(16)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet1:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[i]][0],vertexesBelowSet1[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet1[faceVertexesBelow[i]][1],vertexesBelowSet1[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet1[faceVertexesBelow[i]][2],vertexesBelowSet1[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet1[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet1[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet1[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(17)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet2:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[i]][0],vertexesBelowSet2[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet2[faceVertexesBelow[i]][1],vertexesBelowSet2[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet2[faceVertexesBelow[i]][2],vertexesBelowSet2[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet2[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet2[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet2[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(18)           
except Exception as e:
    print('Problem in obtaining the inert zone below set 1.')
    print(e)
#-------------------------BELOW SET 2---------------------------
try:
    #halfspacesBelow = 1
    halfspacesBelow = returnInequalitiesCoefficients[1]
    #print('type(halfspacesBelow) :', type(halfspacesBelow))
    halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
    #print('type(halfspacesBelowSet1) :', type(halfspacesBelowSet1))
    halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
    #print('type(halfspacesBelowSet2) :', type(halfspacesBelowSet2))
    #Write a function for that.
    feasible_pointBelowIndex = fittedPointForInertZone('below',dualVariableValuesOriginal,indexesOfFittedPointsOriginal)
    feasible_pointBelow = np.array([])

    for i in range(len(explanatoryTrainingOriginal)):
        feasible_pointBelow = np.append(feasible_pointBelow, xofPoints[i][feasible_pointBelowIndex])
    print('feasible_pointBelow: ', feasible_pointBelow)
    feasible_pointBelow = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelow)
    feasible_pointBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet1)
    feasible_pointBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative(halfspacesBelowSet2)
    hsBelow = HalfspaceIntersection(halfspacesBelow, feasible_pointBelow)
    #print('type(hsBelow) :', type(hsBelow))
    hsBelowSet1= HalfspaceIntersection(halfspacesBelowSet1, feasible_pointBelowSet1)
    #print('type(hsBelowSet1) :', type(hsBelowSet1))
    hsBelowSet2= HalfspaceIntersection(halfspacesBelowSet2, feasible_pointBelowSet2)
    #print('type(hsBelowSet2) :', type(hsBelowSet2))
    
    hullPyBelow = qconvexPy("i",hsBelow.intersections)
    #print(1)
    hullPyBelowSet1 = qconvexPy("i",hsBelowSet1.intersections)
    #print(2)
    hullPyBelowSet2 = qconvexPy("i",hsBelowSet2.intersections)
    #print(3)
    
    vertexConnectionIndicesBelow = []
    counter=0
    for item in hullPyBelow:
        if counter>0:
            vertexConnectionIndicesBelow.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelow[counter-1].append(int(subitem))
        counter+=1
    #print(4)
    vertexConnectionIndicesBelowSet1 = []
    counter=0
    for item in hullPyBelowSet1:
        if counter>0:
            vertexConnectionIndicesBelowSet1.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet1[counter-1].append(int(subitem))
        counter+=1
    #print(5)
    vertexConnectionIndicesBelowSet2 = []
    counter=0
    for item in hullPyBelowSet2:
        if counter>0:
            vertexConnectionIndicesBelowSet2.append([])
            for subitem in item.split():
                if(subitem.isdigit()):
                    vertexConnectionIndicesBelowSet2[counter-1].append(int(subitem))
        counter+=1
    #print(6)
    vertexesBelow=hsBelow.intersections
    #print(7)
    vertexesBelowSet1=hsBelowSet1.intersections
    #print(8)
    vertexesBelowSet2=hsBelowSet2.intersections
    #print(9)
    vertexTextListBelow=[]
    vertexTextListBelowSet1=[]
    vertexTextListBelowSet2=[]
    for i in range(len(vertexesBelow[0])):
        vertexTextListBelow.append(f'{np.round(vertexesBelow[0][i],1)},{np.round(vertexesBelow[1][i],1)},{np.round(vertexesBelow[2][i],1)}')    
    #print(10)
    for i in range(len(vertexesBelowSet1[0])):
        vertexTextListBelowSet1.append(f'{np.round(vertexesBelowSet1[0][i],1)},{np.round(vertexesBelowSet1[1][i],1)},{np.round(vertexesBelowSet1[2][i],1)}')    
    #print(11)
    for i in range(len(vertexesBelowSet2[0])):
        vertexTextListBelowSet2.append(f'{np.round(vertexesBelowSet2[0][i],1)},{np.round(vertexesBelowSet2[1][i],1)},{np.round(vertexesBelowSet2[2][i],1)}')    
    #print(12)
#Plotting    
    for i in range(len(vertexesBelow)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelow[i][0]],y=[vertexesBelow[i][1]], z=[vertexesBelow[i][2]], mode='markers+text',\
                 marker=dict(color='purple', size=5), showlegend=False))
    #print(13)
    for i in range(len(vertexesBelowSet1)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[i][0]],y=[vertexesBelowSet1[i][1]], z=[vertexesBelowSet1[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(14)
    for i in range(len(vertexesBelowSet2)):
        fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[i][0]],y=[vertexesBelowSet2[i][1]], z=[vertexesBelowSet2[i][2]], mode='markers+text',\
                 marker=dict(color='lightcoral', size=3), showlegend=False))
    #print(15)
    for faceVertexesBelow in vertexConnectionIndicesBelow:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[i]][0],vertexesBelow[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelow[faceVertexesBelow[i]][1],vertexesBelow[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelow[faceVertexesBelow[i]][2],vertexesBelow[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='purple', size=5), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelow[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelow[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelow[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelow[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='purple', size=5),showlegend=False))
    #print(16)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet1:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[i]][0],vertexesBelowSet1[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet1[faceVertexesBelow[i]][1],vertexesBelowSet1[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet1[faceVertexesBelow[i]][2],vertexesBelowSet1[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet1[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet1[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet1[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet1[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(17)
    for faceVertexesBelow in vertexConnectionIndicesBelowSet2:
        for i in range(len(faceVertexesBelow)-1):
            fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[i]][0],vertexesBelowSet2[faceVertexesBelow[i+1]][0]],\
                                       y=[vertexesBelowSet2[faceVertexesBelow[i]][1],vertexesBelowSet2[faceVertexesBelow[i+1]][1]],\
                                       z=[vertexesBelowSet2[faceVertexesBelow[i]][2],vertexesBelowSet2[faceVertexesBelow[i+1]][2]],\
                                       mode='lines', marker=dict(color='lightcoral', size=3), showlegend=False))
            if i == len(faceVertexesBelow)-2:
                fig.add_trace(go.Scatter3d(x=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][0],vertexesBelowSet2[faceVertexesBelow[0]][0]],\
                                           y=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][1],vertexesBelowSet2[faceVertexesBelow[0]][1]],\
                                           z=[vertexesBelowSet2[faceVertexesBelow[len(faceVertexesBelow)-1]][2],vertexesBelowSet2[faceVertexesBelow[0]][2]],\
                                           mode='lines',marker=dict(color='lightcoral', size=3),showlegend=False))
    #print(18)           
except Exception as e:
    print('Problem in obtaining the inert zone below set 2.')
    print(e)
#--------
axisLimits=150
fig.update_layout(
    scene = dict(   xaxis_title='x'+u"\u2081",
                    yaxis_title='x'+u"\u2082",
                    zaxis_title='x'+u"\u2083",
                    zaxis_range=(-axisLimits,axisLimits),
                    yaxis_range=(-axisLimits,axisLimits),
                    xaxis_range=(-axisLimits,axisLimits),
                )
)

#Normally plotly-python has gray background which I dont find aestethic. 
#Following lines update background colours and mesh on the axises.
fig.update_layout(scene = dict(
xaxis = dict(
     backgroundcolor="rgb(225, 225, 225)",
     gridcolor="rgb(192,192,192)",
     showbackground=False,
     gridwidth=1,
     zerolinecolor="black",),
yaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="black"),
zaxis = dict(
    backgroundcolor="rgb(225, 225, 225)",
    gridcolor="rgb(192,192,192)",
    showbackground=False,
    gridwidth=1,
    zerolinecolor="rgb(192,192,192)",),),
#following lines set margins for the figure.
autosize=False,
width=1000,
height=1000,
  margin=dict(
    l=10,
    r=10,
    b=10,
    t=10,
    ),
)

#fig.update_xaxes(automargin=True)
#fig.update_yaxes(automargin=True)
#fig.update_xaxes(autorange=True)
#fig.update_yaxes(autorange=True)

#fig.show()
#this cell saves 'fig' file as an HTML
pio.write_html(fig, file='index.html', auto_open=True)

In [ ]:
def analyzeCreateSeriesOfDataFromZeroDifferentialData (dataSetName,aboveOrBelow):
    #above or below should be passed as 'above' or 'below'. i.e. as a string
    #This code creates series of data sets up to the differential equal to the dimension of the data set. That is #of explanatory +1
    #Data set names 05by0071-01...05by0076-01; 05by007-01... 05by0012-01; 04by0016-01...04by0021-01; 
    # 06by0062-01...06by0068-01
    # 07by0043-01...07by0050-01; 07by0053-01...07by0060-01; 08by0072-01...08by0080-01; 09by0083-01...09by0092-01
    # 10by0082-01...10by0092-01; 04by0018-01...04by0022-01; 03by0017-01...03by0020-01; 02by0010-01...02by0012-01
    # 09by0035-01...09by0044-01
    explanatoryTrainingOriginal, responseTrainingOriginal, df = createDataExcel(dataSetName,None,None)
    numberOfExplanatoryVariables=len(explanatoryTrainingOriginal)
    nExplanatories=len(explanatoryTrainingOriginal)
    dimensionL1=numberOfExplanatoryVariables+1
    numberOfPoints=len(responseTrainingOriginal)
    manualBetaValues=list()
    solutionOriginal, indexesOfFittedPointsOriginal, positionOfObservationsOriginal, OFVOriginal,\
    dualVariableValuesOriginal = L1RegressionModel(explanatoryTrainingOriginal,responseTrainingOriginal,manualBetaValues)

    for stepCounter in range(dimensionL1+1):
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        #04by0015-01: manualBetaValues= [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #03by0012-01: manualBetaValues= [1.191011236,-0.146067416,-0.123595506]
        #You can automate this process just by passing a manual beta values.
        manualBetaValuesLoop=solutionOriginal['betaValues']
        #Beta values of  03by0012-01
        #manualBetaValues=[1.191011236, -0.146067416, -0.123595506]
        #Beta values of 04by0014-01
        #manualBetaValues = [3.938144467, -0.645304029, 0.114316466, 0.124774104]
        #-------------------------------------------------
        if stepCounter == 0:
            dataSetNameLoop = dataSetName
        else:
            if aboveOrBelow == 'below' or aboveOrBelow == 'Below':
                dataSetNameLoop = f"0{dimensionL1}by00{numberOfPoints}-02.xlsx"
            if aboveOrBelow == 'above' or aboveOrBelow == 'Above':
                dataSetNameLoop = f"0{dimensionL1}by00{numberOfPoints}-01.xlsx"
            
        explanatoryTrainingLoop, responseTrainingLoop, dfData = createDataExcel(dataSetNameLoop,None,None)
        numberOfPoints=len(responseTrainingLoop)
        solutionLoop, indexesOfFittedPointsLoop, positionOfObservationsLoop, OFVLoop, dualVariableValuesLoop =\
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesLoop)
        diffAboveBelow = calculateDiffAboveBelow(positionOfObservationsLoop)
        basisLoop,basisX1Loop = calculateBasis(indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        basisX1Inv = np.linalg.inv(basisX1Loop)
        basisInv = np.linalg.inv(basisLoop)
        #THIS PROCESS NEEDS TO BE AUTOMIZED --------------
        dualVariableValuesLoop = calculateDualVariableValues (basisLoop, indexesOfFittedPointsLoop)
        #03by0012-01: dualVariableValuesOriginal=[-1,-1,-1,1,1,-1,1,-1,-1,1]
        #04by0015-01: dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #dualVariableValuesOriginal=[-1,-1,-1,-1,1,1,-1,-1,-1,1,1,1,1,1]
        #-------------------------------------------------
        print('Data set name: ', dataSetName)
        print('Differential: ', diffAboveBelow)
        print('Beta values: ', solutionLoop['betaValues'])
        print('Objective value: ', OFVLoop)
        #print('position of observations:', positionOfObservationsOriginal)
        print('indexesOfFittedPointsLoop: ', indexesOfFittedPointsLoop)
        #print('dualVariableValues: ', np.round(dualVariableValuesOriginal,3))
        returnInequalitiesCoefficients=calculateInertZoneInequalityCoefficients\
        (dualVariableValuesLoop,indexesOfFittedPointsLoop,positionOfObservationsLoop,explanatoryTrainingLoop)
        #print('-------------------------------------------------')
        print('-----------------TEST AREA FOR NOT MANUALLY SET BETA VALUES, OBJECTIVE VALUE SHOULD BE SAME---------------------')
        manualBetaValuesTest=list()
        solutionTest, indexesOfFittedPointsTest, positionOfObservationsTest, OFVTest, dualVariableValuesTest = \
        L1RegressionModel(explanatoryTrainingLoop,responseTrainingLoop,manualBetaValuesTest)
        #print('Data set name TEST: ', dataSetName)
        print('Objective value TEST: ', OFVTest)
        print('-------------------------------------------------------------------------------------------------------------------')
        
        baryCenterOfSimplexesAboveSet1=np.array([])
        baryCenterOfSimplexesAboveSet2=np.array([])
        baryCenterOfSimplexesBelowSet1=np.array([])
        baryCenterOfSimplexesBelowSet2=np.array([])

        halfspacesAbove = returnInequalitiesCoefficients[0]
        halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
        halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),int(len(halfspacesAbove)))])
        halfspacesBelow = returnInequalitiesCoefficients[1]
        halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
        halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),int(len(halfspacesBelow)))])
        simplexShapesAbove = classifyIneriorSimplex (halfspacesAbove, diffAboveBelow, 'above')
        print('SHAPES ABOVE: ', simplexShapesAbove)
        #print('*****************************')
        simplexShapesBelow = classifyIneriorSimplex (halfspacesBelow, diffAboveBelow, 'below')
        print('SHAPES BELOW: ', simplexShapesBelow)
        print('-------------------------------------------------------------------------------------------------------------------')
        #print('CHECK HERE WITH A MANUAL SOLUTION:::')
        #print(halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop)
        barryCenterToAppendAbove=list()
        barryCenterToAppendBelow=list()
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet1=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2))])
            #chebychefCenterAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
            baryCenterOfSimplexesAboveSet1 = findBarycenterFromVertexes\
            (halfspacesAboveSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            print('barycenter of the simplexes above set 1: ', np.round(baryCenterOfSimplexesAboveSet1,3))
            try:
                pointOnSimplexFacetAboveSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet1,baryCenterOfSimplexesAboveSet1])
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet1: ', interiorPointAboveSet1)
                barryCenterToAppendAbove=interiorPointAboveSet1
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet1: ', pointOnSimplexFacetAboveSet1)                
            except Exception as e:
                print(e)
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 1.')
            print(e)
        try:
            halfspacesAbove = returnInequalitiesCoefficients[0]
            halfspacesAboveSet2=np.array([halfspacesAbove[i] for i in range(int(len(halfspacesAbove)/2),\
                                                                            int(len(halfspacesAbove)))])
            #chebychefCenterAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesAbove)
            baryCenterOfSimplexesAboveSet2 = findBarycenterFromVertexes\
            (halfspacesAboveSet2,dualVariableValuesLoop,indexesOfFittedPointsLoop,'above',\
             explanatoryTrainingLoop)
            try:
                pointOnSimplexFacetAboveSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesAbove)
                interiorPointAboveSet2=convexCombinationOfSetOfPoints([pointOnSimplexFacetAboveSet2,baryCenterOfSimplexesAboveSet2])
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointAboveSet2: ', interiorPointAboveSet2)
                barryCenterToAppendAbove=interiorPointAboveSet2
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetAboveSet2: ', pointOnSimplexFacetAboveSet2)
            except Exception as e:
                print(e)
            print('barycenter of the simplexes above set 2: ', np.round(baryCenterOfSimplexesAboveSet2,3))        
            print('barycenter of the simplexes above are the same: ',\
                  np.array_equal(np.round(baryCenterOfSimplexesAboveSet1,3), np.round(baryCenterOfSimplexesAboveSet2,3)))
        except Exception as e:
            print('Problem in obtaining the barycenters of "above" set 2.')
            print(e)    
        print('-')
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet1=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2))])
            baryCenterOfSimplexesBelowSet1 = findBarycenterFromVertexes\
            (halfspacesBelowSet1,dualVariableValuesLoop,indexesOfFittedPointsLoop,'below',\
             explanatoryTrainingLoop)
            #chebychefCenterBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            try:
                pointOnSimplexFacetBelowSet1 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet1: ', pointOnSimplexFacetBelowSet1)
                interiorPointBelowSet1=convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet1,baryCenterOfSimplexesBelowSet1])            
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/interiorPointBelowSet1: ', interiorPointBelowSet1)
                barryCenterToAppendBelow=interiorPointBelowSet1
            except Exception as e:
                print(e)
            print('barycenter of the simplexes below set 1: ', np.round(baryCenterOfSimplexesBelowSet1,3))
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 1.')
            print(e)
        try:
            halfspacesBelow = returnInequalitiesCoefficients[1]
            halfspacesBelowSet2=np.array([halfspacesBelow[i] for i in range(int(len(halfspacesBelow)/2),\
                                                                            int(len(halfspacesBelow)))])
            #chebychefCenterBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficientsAlternative (halfspacesBelow)
            baryCenterOfSimplexesBelowSet2 = findBarycenterFromVertexes\
            (halfspacesBelowSet2,dualVariableValuesLoop, indexesOfFittedPointsLoop,'below',explanatoryTrainingLoop)            
            try:
                pointOnSimplexFacetBelowSet2 = generateFeasiblePointFromDefiningInequalityCoefficients (halfspacesBelow)            
                interiorPointBelowSet2=convexCombinationOfSetOfPoints([pointOnSimplexFacetBelowSet2,baryCenterOfSimplexesBelowSet2])            
                barryCenterToAppendBelow=interiorPointBelowSet2
                print('analyzeCreateSeriesOfDataFromZeroDifferentialData/pointOnSimplexFacetBelowSet2: ', pointOnSimplexFacetBelowSet2)
            except Exception as e:
                print(e)
            print('barycenter of the simplexes below set 2: ', np.round(baryCenterOfSimplexesBelowSet2,3))
            print('barycenter of the simplexes below are the same: ',\
                  np.array_equal(np.round(baryCenterOfSimplexesBelowSet1,3), np.round(baryCenterOfSimplexesBelowSet2,3)))
            
        except Exception as e:
            print('Problem in obtaining the barycenters of "below" set 2.')
            print(e)
        writeL1SolutionParametersToExcel (dataSetNameLoop,diffAboveBelow,solutionLoop, OFVLoop, indexesOfFittedPointsLoop,dualVariableValuesLoop,\
        returnInequalitiesCoefficients,dfData,simplexShapesAbove,simplexShapesBelow,baryCenterOfSimplexesAboveSet1,baryCenterOfSimplexesAboveSet2,\
        baryCenterOfSimplexesBelowSet1,baryCenterOfSimplexesBelowSet2)    
    #create a new data set from
        try:
            if aboveOrBelow == 'above' or aboveOrBelow == 'Above':
                # New data set is formed here. There is explanatorytrainingloop data that grow gradually. 
                # After this step a new data set is created by appending the barycenter of the corresponding side of L1 ...
                # ... hyperplane on the previous data set.
                # You can create a new sheet on the data set to be written and write: defining inequalities, solution for the data set ...
                for j in range(nExplanatories):
                    explanatoryTrainingLoop[j].append(barryCenterToAppendAbove[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], barryCenterToAppendAbove)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane+deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(nExplanatories):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                #----------------------
                df = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                dataSetName = f"0{dimensionL1}by00{numberOfPoints}-01.xlsx"
                df.to_excel(dataSetName, index=False, sheet_name="Data")
        except Exception as e:
            print('error obtaining baryCenter at above')
            print(e)
        try:
            if aboveOrBelow == 'below' or aboveOrBelow == 'Below':
                for j in range(nExplanatories):
                    explanatoryTrainingLoop[j].append(barryCenterToAppendBelow[j])
                locationOfBaryCenterOnL1Hyperplane = \
                calculateLocationOfAPointOnL1Hyperplane(solutionLoop['betaValues'], barryCenterToAppendBelow)
                deltaResponse=np.round(np.random.uniform(0.1,50,1),3)
                responseTrainingLoop.append(locationOfBaryCenterOnL1Hyperplane-deltaResponse[0])
                dataForL1 = {'Response':responseTrainingLoop}
                for j in range(nExplanatories):
                    dataForL1[f'Explanatory{j+1}']=explanatoryTrainingLoop[j]
                #append new values here
                #----------------------
                df = pd.DataFrame(dataForL1)
                numberOfPoints=len(responseTrainingLoop)
                dataSetName = f"0{dimensionL1}by00{numberOfPoints}-02.xlsx"
                df.to_excel(dataSetName, index=False, sheet_name="Data")
        except Exception as e:
            print('error obtaining baryCenter at below')
            print(e)
      
        print('!!!!!!!!!!!!!!!!!!!!!!!************** ITERATION ENDS*****************!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')